In [ ]:
!pip install pandas spacy neo4j networkx scikit-learn
!python -m spacy download en_core_web_sm
!pip install --upgrade --force-reinstall numpy==1.26.4 scikit-learn==1.3.2
!pip install --force-reinstall numpy==1.26.4
!pip install --force-reinstall scipy==1.10.1
!pip install --force-reinstall scikit-learn==1.3.2

In [1]:
import pandas as pd
import spacy
from neo4j import GraphDatabase
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score,precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
import numpy as np
from collections import Counter, defaultdict

In [2]:
df = pd.read_csv('/kaggle/input/fpl-seasons-pop/fpl_two_seasons.csv')


In [3]:
af = pd.read_csv('/kaggle/input/fpl-seasons-pop/cleaned_merged_seasons.csv', 
                 dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51952 entries, 0 to 51951
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season             51952 non-null  object 
 1   name               51952 non-null  object 
 2   position           51952 non-null  object 
 3   assists            51952 non-null  int64  
 4   bonus              51952 non-null  int64  
 5   bps                51952 non-null  int64  
 6   clean_sheets       51952 non-null  int64  
 7   creativity         51952 non-null  float64
 8   element            51952 non-null  int64  
 9   fixture            51952 non-null  int64  
 10  goals_conceded     51952 non-null  int64  
 11  goals_scored       51952 non-null  int64  
 12  ict_index          51952 non-null  float64
 13  influence          51952 non-null  float64
 14  kickoff_time       51952 non-null  object 
 15  minutes            51952 non-null  int64  
 16  own_goals          519

In [5]:
af.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96169 entries, 0 to 96168
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   season_x           96169 non-null  object
 1   name               96169 non-null  object
 2   position           96169 non-null  object
 3   team_x             76317 non-null  object
 4   assists            96169 non-null  object
 5   bonus              96169 non-null  object
 6   bps                96169 non-null  object
 7   clean_sheets       96169 non-null  object
 8   creativity         96169 non-null  object
 9   element            96169 non-null  object
 10  fixture            96169 non-null  object
 11  goals_conceded     96169 non-null  object
 12  goals_scored       96169 non-null  object
 13  ict_index          96169 non-null  object
 14  influence          96169 non-null  object
 15  kickoff_time       96169 non-null  object
 16  minutes            96169 non-null  objec

In [6]:
df.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW,form
count,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,...,51952.000000,51952.000000,51952.000000,5.195200e+04,5.195200e+04,5.195200e+04,51952.000000,51952.000000,51952.000000,51952.000000
mean,0.035725,0.093875,5.276043,0.090930,4.094651,344.981637,199.273271,0.456287,0.039941,1.473866,...,1.569391,4.507488,1.215314,1.999289e+03,1.576509e+04,1.376511e+04,50.031491,0.051297,21.121073,0.119020
std,0.201135,0.459547,9.228127,0.287512,9.931570,201.805312,108.743119,0.946539,0.217700,2.819401,...,1.365193,12.189522,2.403077,7.288251e+04,6.421021e+04,5.377897e+04,11.917808,0.220606,10.951132,0.175579
min,0.000000,0.000000,-21.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-4.000000,-2.180978e+06,0.000000e+00,0.000000e+00,37.000000,0.000000,1.000000,-0.200000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,171.000000,106.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,-1.089000e+03,3.300000e+01,9.800000e+01,44.000000,0.000000,12.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,342.000000,204.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,-4.500000e+01,3.460000e+02,8.790000e+02,45.000000,0.000000,22.000000,0.025000
75%,0.000000,0.000000,8.000000,0.000000,1.500000,513.000000,294.000000,1.000000,0.000000,2.000000,...,2.000000,2.000000,1.000000,5.600000e+01,4.464000e+03,6.863250e+03,53.000000,0.000000,31.000000,0.200000
max,4.000000,3.000000,128.000000,1.000000,136.200000,778.000000,380.000000,9.000000,4.000000,31.500000,...,9.000000,169.000000,24.000000,1.983733e+06,2.104464e+06,2.233619e+06,133.000000,1.000000,38.000000,2.000000


In [7]:
df.isnull().sum()

season               0
name                 0
position             0
assists              0
bonus                0
bps                  0
clean_sheets         0
creativity           0
element              0
fixture              0
goals_conceded       0
goals_scored         0
ict_index            0
influence            0
kickoff_time         0
minutes              0
own_goals            0
penalties_missed     0
penalties_saved      0
red_cards            0
saves                0
selected             0
team_a_score         0
team_h_score         0
threat               0
total_points         0
transfers_balance    0
transfers_in         0
transfers_out        0
value                0
yellow_cards         0
GW                   0
form                 0
home_team            0
away_team            0
dtype: int64

In [8]:
df.head()

,season,name,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW,form,home_team,away_team
0,2021-22,Aaron Connolly,FWD,0,0,0,0,0.0,72,2,...,0,0,0,0,55,0,1,0.000,Burnley,Brighton
1,2021-22,Aaron Connolly,FWD,0,0,-2,0,1.1,72,12,...,1,-1682,1899,3581,55,0,2,0.000,Brighton,Watford
2,2021-22,Aaron Connolly,FWD,0,0,0,0,0.0,72,22,...,0,-737,2897,3634,55,0,3,0.050,Brighton,Everton
3,2021-22,Aaron Connolly,FWD,0,0,0,0,0.0,72,32,...,0,-3682,1857,5539,54,0,4,0.033,Brentford,Brighton
4,2021-22,Aaron Connolly,FWD,0,0,0,0,0.0,72,42,...,0,-2548,301,2849,54,0,5,0.025,Brighton,Leicester


In [9]:
# Create config.txt file in current directory
config_content = """URI=neo4j+s://1da86c19.databases.neo4j.io
USERNAME=neo4j
PASSWORD=HA4iunTOGen7RYpeISs3ZRhcWjpcokqam9przCqCuQ8"""

# Write to config.txt in current directory
with open('config.txt', 'w') as f:
    f.write(config_content)

print("config.txt created successfully!")

# Verify creation
import os
if os.path.exists('config.txt'):
    print("✓ config.txt exists in:", os.path.abspath('config.txt'))
else:
    print("✗ config.txt was not created")

config.txt created successfully!
✓ config.txt exists in: /kaggle/working/config.txt


In [11]:
#!/usr/bin/env python3
"""
Create Knowledge Graph for FPL (Fantasy Premier League) Data
This script builds a Neo4j knowledge graph from FPL two-season dataset
"""

import pandas as pd
import spacy
from neo4j import GraphDatabase
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
import numpy as np
from collections import Counter, defaultdict
import os

# ============================================================================
# Configuration
# ============================================================================

def load_config():
    """Load Neo4j configuration from config.txt"""
    try:
        with open('config.txt', 'r') as f:
            lines = f.readlines()
        config = {}
        for line in lines:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
        
        uri = config.get('URI')
        user = config.get('USERNAME')
        password = config.get('PASSWORD')
        
        if uri and user and password:
            return uri, user, password
        else:
            raise ValueError("Missing credentials in config file")
    except FileNotFoundError:
        raise FileNotFoundError("config.txt file not found!")

# ============================================================================
# Data Loading
# ============================================================================

def load_data(filepath='/kaggle/input/fpl-seasons-pop/fpl_two_seasons.csv'):
    """Load FPL dataset from CSV file"""
    print(f"Loading data from {filepath}...")
    df = pd.read_csv(filepath)
    print(f"Loaded {len(df)} records")
    print("\nDataset Info:")
    df.info()
    print("\nFirst few rows:")
    print(df.head())
    print("\nMissing values:")
    print(df.isnull().sum())
    return df

def load_team_data(filepath='/kaggle/input/fpl-seasons-pop/cleaned_merged_seasons.csv'):
    """Load team data for players"""
    print(f"Loading team data from {filepath}...")
    
    # Load with explicit dtype handling for mixed types
    team_df = pd.read_csv(filepath, low_memory=False)
    
    print(f"Loaded {len(team_df)} records")
    print("\nTeam dataset info:")
    team_df.info()
    
    # Clean and process team data
    # Create a mapping of player to team for each season
    # We'll need to handle the mixed types in team_x column
    team_df['team_x'] = team_df['team_x'].astype(str).replace('nan', np.nan)
    
    # Remove rows where team is null
    team_df_clean = team_df.dropna(subset=['team_x'])
    
    print(f"\nCleaned team data: {len(team_df_clean)} records")
    
    # Create a mapping of (season, name) -> team
    # Note: There might be multiple teams per season if players transferred
    # We'll take the most frequent team for each player in each season
    player_team_mapping = {}
    
    # Group by season, player name, and team
    grouped = team_df_clean.groupby(['season_x', 'name', 'team_x']).size().reset_index(name='count')
    
    # For each season and player, get the team with highest count
    for (season, player), group in grouped.groupby(['season_x', 'name']):
        most_common_team = group.loc[group['count'].idxmax(), 'team_x']
        player_team_mapping[(season, player)] = most_common_team
    
    print(f"\nCreated player-team mapping for {len(player_team_mapping)} player-season combinations")
    
    # Also create a simple name->team mapping (most recent team)
    # This will be used as fallback if season-specific mapping not found
    simple_mapping = {}
    for (season, player), team in player_team_mapping.items():
        simple_mapping[player] = team
    
    print(f"Created simple player-team mapping for {len(simple_mapping)} unique players")
    
    return player_team_mapping, simple_mapping

# ============================================================================
# Knowledge Graph Construction
# ============================================================================

def build_kg(tx, batch, player_team_mapping, simple_mapping):
    """Build Knowledge Graph from batch of records with team information"""
    tx.run("""
    UNWIND $rows AS row
    MERGE (s:Season {season_name: row.season})
    WITH row, s
    
    MERGE (gw:Gameweek {season: row.season, GW_number: row.GW})
    MERGE (s)-[:HAS_GW]->(gw)
    
    WITH row, gw
    
    MERGE (f:Fixture {season: row.season, fixture_number: row.fixture})
    SET f.kickoff_time = row.kickoff_time
    MERGE (gw)-[:HAS_FIXTURE]->(f)
    
    WITH row, f
    
    MERGE (home:Team {name: row.home_team})
    MERGE (away:Team {name: row.away_team})
    MERGE (f)-[:HAS_HOME_TEAM]->(home)
    MERGE (f)-[:HAS_AWAY_TEAM]->(away)
    
    WITH row, f, home, away
    
    // Determine which team the player is on for this fixture
    // First check if player's team matches home or away
    WITH row, f, home, away,
         CASE 
             WHEN row.player_team = row.home_team THEN home
             WHEN row.player_team = row.away_team THEN away
             ELSE null
         END AS player_team_node
    
    MERGE (p:Player {player_name: row.name, player_element: row.element})
    // Add team to player node property
    SET p.team = row.player_team
    
    WITH row, p, f, player_team_node
    
    MATCH (pos:Position {name: row.position})
    MERGE (p)-[:PLAYS_AS]->(pos)
    
    // Create relationship between player and their team
    WITH row, p, f, player_team_node
    WHERE player_team_node IS NOT NULL
    MERGE (p)-[:PLAYS_FOR]->(player_team_node)
    
    WITH row, p, f
    
    MERGE (p)-[r:PLAYED_IN]->(f)
    SET r.minutes = row.minutes,
        r.goals_scored = row.goals_scored,
        r.assists = row.assists,
        r.total_points = row.total_points,
        r.bonus = row.bonus,
        r.clean_sheets = row.clean_sheets,
        r.goals_conceded = row.goals_conceded,
        r.own_goals = row.own_goals,
        r.penalties_saved = row.penalties_saved,
        r.penalties_missed = row.penalties_missed,
        r.yellow_cards = row.yellow_cards,
        r.red_cards = row.red_cards,
        r.saves = row.saves,
        r.bps = row.bps,
        r.influence = row.influence,
        r.creativity = row.creativity,
        r.threat = row.threat,
        r.ict_index = row.ict_index,
        r.form = row.form
    """, rows=batch)

def initialize_database(driver):
    """Clear database and create initial Position nodes"""
    with driver.session() as session:
        # Clear all existing data
        session.run("MATCH (n) DETACH DELETE n")
        print("Database cleared")
        
        # Create Position nodes
        session.run("""
        UNWIND $pos AS p
        MERGE (:Position {name: p})
        """, pos=["GK", "DEF", "MID", "FWD"])
        print("Position nodes created")

def populate_knowledge_graph(driver, df, player_team_mapping, simple_mapping, batch_size=500):
    """Populate the knowledge graph with FPL data including team information"""
    print(f"\nPopulating knowledge graph with {len(df)} records...")
    
    # Add team information to each row
    print("Adding team information to dataset...")
    rows_with_teams = []
    
    for _, row in df.iterrows():
        row_dict = row.to_dict()
        
        # Try to get team from player_team_mapping using season and name
        team_key = (row_dict['season'], row_dict['name'])
        
        if team_key in player_team_mapping:
            row_dict['player_team'] = player_team_mapping[team_key]
        elif row_dict['name'] in simple_mapping:
            # Fallback to simple mapping
            row_dict['player_team'] = simple_mapping[row_dict['name']]
        else:
            # If no team found, use a default or try to infer from fixture
            # Check if player's team can be inferred from home/away team based on typical patterns
            # For now, set to unknown
            row_dict['player_team'] = "Unknown"
        
        rows_with_teams.append(row_dict)
    
    print(f"Added team information to {len(rows_with_teams)} records")
    
    total_batches = (len(rows_with_teams) + batch_size - 1) // batch_size
    
    with driver.session() as session:
        for i in range(0, len(rows_with_teams), batch_size):
            batch = rows_with_teams[i:i + batch_size]
            session.execute_write(build_kg, batch, player_team_mapping, simple_mapping)
            batch_num = i // batch_size + 1
            print(f"Inserted batch {batch_num}/{total_batches} ({len(batch)} records)")
    
    print("\n✓ Knowledge graph population complete!")
    
    # Print summary of team assignments
    team_counts = {}
    for row in rows_with_teams:
        team = row['player_team']
        team_counts[team] = team_counts.get(team, 0) + 1
    
    print("\nTeam assignment summary:")
    for team, count in sorted(team_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"  {team}: {count} records")
    
    if "Unknown" in team_counts:
        unknown_pct = (team_counts["Unknown"] / len(rows_with_teams)) * 100
        print(f"\nNote: {team_counts['Unknown']} records ({unknown_pct:.1f}%) have unknown team")

# ============================================================================
# Query Functions (for verification)
# ============================================================================

def verify_team_data(driver):
    """Verify that team data was properly added"""
    with driver.session() as session:
        # Query to check player-team relationships
        result = session.run("""
        MATCH (p:Player)-[:PLAYS_FOR]->(t:Team)
        RETURN p.player_name AS player, t.name AS team, COUNT(*) as fixture_count
        ORDER BY fixture_count DESC
        LIMIT 10
        """)
        
        print("\nTop 10 player-team relationships:")
        for record in result:
            print(f"  {record['player']} -> {record['team']} ({record['fixture_count']} fixtures)")
        
        # Count players with team information
        result = session.run("""
        MATCH (p:Player)
        OPTIONAL MATCH (p)-[:PLAYS_FOR]->(t:Team)
        RETURN 
            COUNT(DISTINCT p) as total_players,
            COUNT(DISTINCT t) as total_teams_with_relationships,
            SUM(CASE WHEN t IS NOT NULL THEN 1 ELSE 0 END) as players_with_team_relationship
        """)
        
        stats = result.single()
        print(f"\nTeam relationship statistics:")
        print(f"  Total players: {stats['total_players']}")
        print(f"  Players with team relationship: {stats['players_with_team_relationship']}")
        print(f"  Unique teams in relationships: {stats['total_teams_with_relationships']}")

# ============================================================================
# Main Execution
# ============================================================================

def main():
    """Main execution function"""
    print("=" * 70)
    print("FPL Knowledge Graph Builder with Team Information")
    print("=" * 70)
    
    # Step 1: Load configuration
    print("\n[Step 1] Loading Neo4j configuration...")
    uri, user, password = load_config()
    
    # Step 2: Connect to Neo4j
    print("\n[Step 2] Connecting to Neo4j database...")
    driver = GraphDatabase.driver(uri, auth=(user, password))
    print("Connected to Neo4j database")
    
    # Step 3: Load main data
    print("\n[Step 3] Loading FPL dataset...")
    df = load_data('/kaggle/input/fpl-seasons-pop/fpl_two_seasons.csv')
    
    # Step 4: Load team data
    print("\n[Step 4] Loading team data...")
    player_team_mapping, simple_mapping = load_team_data('/kaggle/input/fpl-seasons-pop/cleaned_merged_seasons.csv')
    
    # Step 5: Initialize database
    print("\n[Step 5] Initializing database...")
    initialize_database(driver)
    
    # Step 6: Populate knowledge graph with team data
    print("\n[Step 6] Building knowledge graph with team information...")
    populate_knowledge_graph(driver, df, player_team_mapping, simple_mapping, batch_size=500)
    
    # Step 7: Verify team data
    print("\n[Step 7] Verifying team data...")
    verify_team_data(driver)
    
    # Step 8: Close connection
    print("\n[Step 8] Closing connection...")
    driver.close()
    print(" Connection closed")
    
    print("\n" + "=" * 70)
    print("Knowledge Graph Creation Complete!")
    print("=" * 70)

if __name__ == "__main__":
    main()

FPL Knowledge Graph Builder with Team Information

[Step 1] Loading Neo4j configuration...

[Step 2] Connecting to Neo4j database...
Connected to Neo4j database

[Step 3] Loading FPL dataset...
Loading data from /kaggle/input/fpl-seasons-pop/fpl_two_seasons.csv...
Loaded 51952 records

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51952 entries, 0 to 51951
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season             51952 non-null  object 
 1   name               51952 non-null  object 
 2   position           51952 non-null  object 
 3   assists            51952 non-null  int64  
 4   bonus              51952 non-null  int64  
 5   bps                51952 non-null  int64  
 6   clean_sheets       51952 non-null  int64  
 7   creativity         51952 non-null  float64
 8   element            51952 non-null  int64  
 9   fixture            51952 non-null  int64  
 10  goals_con

In [12]:
%%writefile fpl_input_preprocessing.py


"""
Enhanced FPL Input Preprocessor with Robust LLM-based Multi-Entity Extraction
"""

import re
import json
from typing import Dict, List, Tuple, Optional
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import Counter
from huggingface_hub import InferenceClient


class FPLInputPreprocessorLLM:
    """
    Enhanced FPL preprocessor with improved LLM-based multi-entity extraction
    """
    
    def __init__(self, 
                 hf_token: str,
                 llm_model: str = "google/gemma-2-2b-it",
                 embedding_model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
                 use_llm: bool = True):
        """Initialize the preprocessor"""
        print("🚀 Initializing FPL Input Preprocessor (Enhanced)...")
        
        self.use_llm = use_llm
        
        # Initialize LLM client if enabled
        if use_llm:
            try:
                self.llm_client = InferenceClient(model=llm_model, token=hf_token)
                print(f"✅ Connected to LLM: {llm_model}")
            except Exception as e:
                print(f"⚠️ LLM connection failed: {e}")
                print("⚠️ Falling back to rule-based approach")
                self.use_llm = False
        
        # Initialize embedding model
        self.embedder = SentenceTransformer(embedding_model_name)
        print(f"✅ Loaded embedding model: {embedding_model_name}")
        
        # Define intent types
        self.intent_types = [
            'player_search',
            'performance_query', 
            'comparison',
            'recommendation',
            'team_analysis',
            'fixture_query',
            'value_analysis',
            'form_query',
            'general_query'
        ]
        
        self._init_rule_based_entities()
        self._init_player_database()
        self.error_log = []
        print("✅ System initialized\n")
    
    def _init_rule_based_entities(self):
        """Initialize rule-based entity dictionaries"""
        self.positions = {
            'GK': ['gk', 'goalkeeper', 'keeper', 'goalie', 'keepers', 'goalkeepers'],
            'DEF': ['def', 'defender', 'defence', 'defense', 'back', 'defenders', 'defensive'],
            'MID': ['mid', 'midfielder', 'midfield', 'midfielders'],
            'FWD': ['fwd', 'forward', 'striker', 'attacker', 'attack', 'forwards', 'strikers', 'attackers']
        }
        
        self.teams = {
            'Arsenal': ['arsenal', 'ars', 'gunners'],
            'Aston Villa': ['aston villa', 'villa', 'avl'],
            'Bournemouth': ['bournemouth', 'bou', 'cherries'],
            'Brentford': ['brentford', 'bre', 'bees'],
            'Brighton': ['brighton', 'bha', 'seagulls'],
            'Burnley': ['burnley', 'bur', 'clarets'],
            'Chelsea': ['chelsea', 'che', 'blues'],
            'Crystal Palace': ['crystal palace', 'palace', 'cry', 'eagles'],
            'Everton': ['everton', 'eve', 'toffees'],
            'Fulham': ['fulham', 'ful', 'cottagers'],
            'Liverpool': ['liverpool', 'liv', 'reds'],
            'Luton': ['luton', 'lut', 'hatters'],
            'Man City': ['man city', 'manchester city', 'city', 'mci', 'mcfc'],
            'Man Utd': ['man utd', 'manchester united', 'united', 'mun', 'mufc'],
            'Newcastle': ['newcastle', 'new', 'magpies', 'toon'],
            'Nottingham Forest': ['nottingham', 'forest', 'nfo', 'nottm forest'],
            'Sheffield Utd': ['sheffield', 'sheffield united', 'shu', 'blades'],
            'Tottenham': ['tottenham', 'spurs', 'tot', 'thfc'],
            'West Ham': ['west ham', 'whu', 'hammers'],
            'Wolves': ['wolves', 'wolverhampton', 'wol'],
            'Leeds': ['leeds', 'leeds united', 'lee', 'lufc'],
            'Leicester': ['leicester', 'leicester city', 'lei', 'lcfc', 'foxes'],
            'Southampton': ['southampton', 'saints', 'sou'],
        }
        
        self.metrics = {
            'total_points': ['points', 'total points', 'score', 'scoring'],
            'goals_scored': ['goals', 'goal', 'scored', 'scorer', 'scorers', 'scoring goals'],
            'assists': ['assists', 'assist', 'assisting', 'assist provider'],
            'bonus': ['bonus', 'bonus points', 'bps'],
            'minutes': ['minutes', 'playing time', 'game time'],
            'clean_sheets': ['clean sheets', 'clean sheet', 'cs', 'cleansheet', 'cleansheets'],
            'saves': ['saves', 'save', 'saved'],
            'ict_index': ['ict', 'ict index'],
            'influence': ['influence', 'influential'],
            'creativity': ['creativity', 'creative'],
            'threat': ['threat', 'threatening'],
            'form': ['form', 'recent form', 'hot', 'recent', 'lately', 'in form'],
            'cards': ['cards', 'yellow cards', 'red cards', 'yellow', 'red', 'disciplinary'],
            'goals_conceded': ['goals conceded', 'conceded', 'goals against'],
            'own_goals': ['own goals', 'own goal', 'og'],
            'penalties_saved': ['penalties saved', 'penalty saves', 'pen saves'],
            'penalties_missed': ['penalties missed', 'penalty misses', 'pen misses'],
            'yellow_cards': ['yellow cards', 'yellows', 'bookings'],
            'red_cards': ['red cards', 'reds', 'sent off', 'dismissals'],
            'bps': ['bps', 'bonus point system'],
            'value': ['value', 'price', 'cost', 'worth', 'bargain', 'expensive'],
            'ownership': ['ownership', 'owned', 'selected', '% owned']
        }
    
    def _init_player_database(self):
        """Initialize comprehensive player database with variations"""
        self.known_players = {
            # Top Forwards
            'Erling Haaland': ['haaland', 'erling', 'erling haaland'],
            'Harry Kane': ['kane', 'harry kane'],
            'Ivan Toney': ['toney', 'ivan toney'],
            'Alexander Isak': ['isak', 'alexander isak'],
            'Ollie Watkins': ['watkins', 'ollie watkins', 'oliver watkins'],
            'Darwin Nunez': ['nunez', 'darwin', 'darwin nunez'],
            'Dominic Calvert-Lewin': ['calvert-lewin', 'dcl', 'dominic calvert-lewin'],
            'Callum Wilson': ['wilson', 'callum wilson'],
            
            # Top Midfielders
            'Mohamed Salah': ['salah', 'mo salah', 'mohamed salah'],
            'Kevin De Bruyne': ['de bruyne', 'kdb', 'kevin de bruyne'],
            'Heung-Min Son': ['son', 'heung-min son', 'sonny'],
            'Bukayo Saka': ['saka', 'bukayo saka'],
            'Phil Foden': ['foden', 'phil foden'],
            'Bruno Fernandes': ['bruno', 'fernandes', 'bruno fernandes'],
            'Martin Odegaard': ['odegaard', 'martin odegaard'],
            'James Maddison': ['maddison', 'james maddison', 'madders'],
            'Marcus Rashford': ['rashford', 'marcus rashford'],
            'Jack Grealish': ['grealish', 'jack grealish'],
            
            # Top Defenders
            'Trent Alexander-Arnold': ['trent', 'taa', 'alexander-arnold', 'trent alexander-arnold'],
            'Reece James': ['reece james', 'reece'],
            'Kieran Trippier': ['trippier', 'kieran trippier'],
            'Andrew Robertson': ['robertson', 'andy robertson', 'andrew robertson', 'robbo'],
            'Virgil van Dijk': ['van dijk', 'virgil', 'virgil van dijk', 'vvd'],
            'William Saliba': ['saliba', 'william saliba'],
            'Ruben Dias': ['dias', 'ruben dias'],
            'Gabriel Magalhaes': ['gabriel', 'gabriel magalhaes'],
            
            # Top Goalkeepers
            'Alisson': ['alisson', 'alisson becker'],
            'Ederson': ['ederson', 'ederson moraes'],
            'Aaron Ramsdale': ['ramsdale', 'aaron ramsdale'],
            'Nick Pope': ['pope', 'nick pope'],
            'David Raya': ['raya', 'david raya'],
            'Robert Sanchez': ['sanchez', 'robert sanchez']
        }
        
        # Create reverse mapping for quick lookup
        self.player_variants = {}
        for player, variants in self.known_players.items():
            for variant in variants:
                self.player_variants[variant] = player
    
    def classify_intent(self, text: str) -> Tuple[str, float]:
        """Enhanced intent classification with LLM support"""
        text_lower = text.lower()
        
        # ===== EMERGENCY HOTFIX =====
        # Force performance_query for any question about specific gameweek performance
        gameweek_patterns = [
            r'performance.*gameweek.*\d+',
            r'gameweek.*\d+.*performance',
            r'how.*did.*gameweek.*\d+',
            r'what.*was.*gameweek.*\d+'
        ]
        for pattern in gameweek_patterns:
           if re.search(pattern, text_lower, re.IGNORECASE):
                return 'performance_query', 0.99  # Highest confidence
    # ===== END HOTFIX =====
        # Rule-based classification (fast path)
        
        # Top/best/leading queries
        top_keywords = ['top', 'best', 'leading', 'highest', 'most', 'who scored', 'who has']
        if any(keyword in text_lower for keyword in top_keywords):
            if any(word in text_lower for word in ['scorer', 'scorers', 'goals', 'goal']):
                return 'recommendation', 0.95
            if any(word in text_lower for word in ['assist', 'assists']):
                return 'recommendation', 0.95
            if any(word in text_lower for word in ['clean sheet', 'cleansheet']):
                return 'recommendation', 0.95
            if any(word in text_lower for word in ['bonus', 'bonus points']):
                return 'recommendation', 0.95
            return 'recommendation', 0.90
        
        # Comparison queries (check for multiple entities)
        comparison_keywords = ['compare', 'vs', 'versus', 'between', 'or', 'and', 'versus']
        if any(word in text_lower for word in comparison_keywords):
            # Check if there are at least two potential entities for comparison
            potential_players = self._extract_capitalized_names(text)
            if len(potential_players) >= 2 or len(re.findall(r'\b(vs|versus|compare)\b', text_lower)) > 0:
                return 'comparison', 0.95
            return 'comparison', 0.85
        
        # Fixture queries
        if any(word in text_lower for word in ['fixture', 'fixtures', 'schedule', 'match', 'matches', 'playing against', 'upcoming']):
            return 'fixture_query', 0.90
        
        # Team analysis queries
        if any(word in text_lower for word in ['players for', 'play for', 'roster', 'squad', 'team', 'from arsenal', 'from liverpool']):
            return 'team_analysis', 0.85
        
        # Form queries
        if any(word in text_lower for word in ['form', 'recent', 'lately', 'hot', 'in form', 'last 5', 'recently']):
            return 'form_query', 0.85
        
        # Performance queries (gameweek-specific)
        if 'gameweek' in text_lower or 'gw' in text_lower:
            return 'performance_query', 0.85
        
        # Player search (specific player names)
        if any(player.lower() in text_lower for variants in self.known_players.values() for player in variants):
            if 'gameweek' in text_lower or 'gw' in text_lower:
                return 'performance_query', 0.90
            return 'player_search', 0.80
        
        # Value analysis
        if any(word in text_lower for word in ['value', 'price', 'cost', 'worth', 'bargain', 'differential', 'cheap', 'expensive']):
            return 'value_analysis', 0.85
        
        # Use LLM for ambiguous cases
        if self.use_llm:
            try:
                return self._classify_intent_with_llm(text)
            except Exception as e:
                self.error_log.append({'type': 'llm_classification_error', 'error': str(e)})
        
        return 'general_query', 0.30
    
    def _classify_intent_with_llm(self, text: str) -> Tuple[str, float]:
        """Use LLM for intent classification"""
        prompt = f"""Analyze this Fantasy Premier League query and classify its intent:

Query: "{text}"

Available intents:
- player_search: Finding specific player information (e.g., "Haaland stats", "Salah price")
- performance_query: Player performance in specific gameweek/match (e.g., "Salah points GW5", "Haaland gameweek 3")
- comparison: Comparing multiple players or teams (e.g., "Haaland vs Salah", "compare TAA and Robertson")
- recommendation: Finding top/best players (e.g., "top scorers", "best defenders", "highest points")
- team_analysis: Team-related queries (e.g., "Arsenal players", "Liverpool squad")
- fixture_query: Match schedule and fixtures (e.g., "Man City fixtures", "upcoming matches")
- value_analysis: Player value and pricing (e.g., "best value defenders", "Haaland worth it")
- form_query: Recent player form (e.g., "players in form", "recent performance")
- general_query: General questions

Determine the SINGLE most appropriate intent. Return format: intent,confidence (0.0-1.0)

Response:"""
        
        response = self.llm_client.text_generation(prompt, max_new_tokens=50)
        parts = response.strip().split(',')
        if len(parts) == 2:
            intent = parts[0].strip().lower()
            try:
                confidence = float(parts[1].strip())
                # Validate intent
                if intent in self.intent_types:
                    return intent, confidence
            except ValueError:
                pass
        return 'general_query', 0.30
    
    def extract_entities(self, text: str) -> Dict[str, List[str]]:
        """Enhanced multi-entity extraction with improved LLM support"""
        # Always run rule-based extraction first for robustness
        rule_entities = self._extract_entities_rule_based(text)
        
        # Use LLM for enhanced extraction if available
        if self.use_llm:
            try:
                llm_entities = self._extract_entities_with_llm(text)
                return self._merge_entities(llm_entities, rule_entities)
            except Exception as e:
                self.error_log.append({'type': 'llm_extraction_error', 'error': str(e)})
        
        return rule_entities
    
    def _extract_entities_with_llm(self, text: str) -> Dict[str, List[str]]:
        """Use LLM for comprehensive entity extraction with multi-entity support"""
        prompt = f"""Extract ALL entities from this Fantasy Premier League query.

Query: "{text}"

Extract entities for ALL these categories:
1. players: ALL player names mentioned (extract ALL names)
2. teams: ALL team names mentioned
3. positions: GK, DEF, MID, FWD (only these values)
4. seasons: Year ranges like 2023-24, 2023/24, or single years
5. gameweeks: Gameweek numbers (extract ALL numbers mentioned as gameweeks)
6. metrics: Statistics mentioned (goals, assists, points, clean_sheets, bonus, form, etc.)
7. numerical_values: Any numbers that are thresholds, counts, or values
8. comparators: Comparison words (more than, less than, over, under, above, below)

CRITICAL INSTRUCTIONS:
- Extract ALL instances of each entity type. If multiple players are mentioned, extract ALL of them.
- For players: Use full canonical names when possible (e.g., "Erling Haaland" not just "Haaland")
- For teams: Use full canonical names (e.g., "Man City" not just "City")
- For metrics: Use standardized names from: total_points, goals_scored, assists, bonus, minutes, clean_sheets, saves, ict_index, influence, creativity, threat, form, cards, goals_conceded, own_goals, penalties_saved, penalties_missed, yellow_cards, red_cards, bps, value, ownership
- Return ONLY valid JSON with exactly these 8 keys.

Example response for "Compare Haaland and Salah goals vs assists in gameweek 5":
{{
  "players": ["Erling Haaland", "Mohamed Salah"],
  "teams": [],
  "positions": [],
  "seasons": [],
  "gameweeks": ["5"],
  "metrics": ["goals_scored", "assists"],
  "numerical_values": [],
  "comparators": []
}}

Now extract entities from the query above:"""

        try:
            response = self.llm_client.text_generation(prompt, max_new_tokens=500)
            
            # Extract JSON from response
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                try:
                    entities = json.loads(json_match.group())
                    
                    # Validate and clean entities
                    cleaned_entities = self._clean_llm_entities(entities, text)
                    
                    # Ensure all required keys exist
                    required_keys = ['players', 'teams', 'positions', 'seasons', 'gameweeks', 
                                   'metrics', 'numerical_values', 'comparators']
                    for key in required_keys:
                        if key not in cleaned_entities:
                            cleaned_entities[key] = []
                        elif not isinstance(cleaned_entities[key], list):
                            cleaned_entities[key] = [str(cleaned_entities[key])]
                    
                    return cleaned_entities
                except json.JSONDecodeError as e:
                    self.error_log.append({
                        'type': 'llm_json_parse_error',
                        'response': response[:200],
                        'error': str(e)
                    })
        except Exception as e:
            self.error_log.append({
                'type': 'llm_extraction_failure',
                'error': str(e)
            })
        
        return {
            'players': [], 'teams': [], 'positions': [], 'seasons': [],
            'gameweeks': [], 'metrics': [], 'numerical_values': [], 'comparators': []
        }
    
    def _clean_llm_entities(self, entities: Dict, original_text: str) -> Dict:
        """Clean and validate LLM-extracted entities"""
        cleaned = {}
        text_lower = original_text.lower()
        
        for key, values in entities.items():
            if not isinstance(values, list):
                values = [values] if values else []
            
            cleaned_values = []
            seen = set()
            
            for value in values:
                if not value:
                    continue
                
                # Convert to string and strip whitespace
                str_value = str(value).strip()
                
                # Deduplicate (case-insensitive)
                val_lower = str_value.lower()
                if val_lower in seen:
                    continue
                
                seen.add(val_lower)
                
                # Validate specific entity types
                if key == 'players':
                    # Try to map to canonical player name
                    canonical_name = self._map_to_canonical_player(str_value)
                    if canonical_name:
                        cleaned_values.append(canonical_name)
                    elif len(str_value.split()) >= 2:  # Likely a player name
                        cleaned_values.append(str_value)
                
                elif key == 'teams':
                    # Map team aliases to canonical names
                    canonical_team = self._map_to_canonical_team(str_value)
                    if canonical_team:
                        cleaned_values.append(canonical_team)
                
                elif key == 'positions':
                    # Standardize position values
                    pos = str_value.upper()
                    if pos in ['GK', 'DEF', 'MID', 'FWD']:
                        cleaned_values.append(pos)
                
                elif key == 'metrics':
                    # Standardize metric names
                    metric = self._map_to_canonical_metric(str_value)
                    if metric:
                        cleaned_values.append(metric)
                
                elif key == 'gameweeks':
                    # Extract numbers only
                    match = re.search(r'(\d+)', str_value)
                    if match:
                        cleaned_values.append(match.group(1))
                
                elif key == 'numerical_values':
                    try:
                        num = float(str_value)
                        cleaned_values.append(num)
                    except ValueError:
                        pass
                
                else:
                    # Keep other values as-is
                    cleaned_values.append(str_value)
            
            cleaned[key] = cleaned_values
        
        return cleaned
    
    def _map_to_canonical_player(self, player_name: str) -> Optional[str]:
        """Map player name or alias to canonical name"""
        name_lower = player_name.lower()
        
        # Check exact variants first
        if name_lower in self.player_variants:
            return self.player_variants[name_lower]
        
        # Check partial matches
        for canonical, variants in self.known_players.items():
            if name_lower in canonical.lower() or any(variant in name_lower for variant in variants):
                return canonical
        
        # Check if name contains known player last names
        words = name_lower.split()
        for word in words:
            for canonical, variants in self.known_players.items():
                if any(word == v for v in variants):
                    return canonical
        
        return None
    
    def _map_to_canonical_team(self, team_name: str) -> Optional[str]:
        """Map team alias to canonical name"""
        team_lower = team_name.lower()
        for canonical, variants in self.teams.items():
            if team_lower == canonical.lower() or team_lower in variants:
                return canonical
        return None
    
    def _map_to_canonical_metric(self, metric_name: str) -> Optional[str]:
        """Map metric name to canonical form"""
        metric_lower = metric_name.lower()
        for canonical, variants in self.metrics.items():
            if metric_lower == canonical or metric_lower in variants:
                return canonical
        return None
    
    def _extract_entities_rule_based(self, text: str) -> Dict[str, List[str]]:
        """Enhanced rule-based entity extraction with multi-entity support"""
        entities = {
            'players': [],
            'teams': [],
            'positions': [],
            'seasons': [],
            'gameweeks': [],
            'metrics': [],
            'numerical_values': [],
            'comparators': []
        }
        
        text_lower = text.lower()
        
        # Extract ALL metrics mentioned
        for canonical_metric, variants in self.metrics.items():
            for variant in variants:
                # Use word boundary matching for better accuracy
                pattern = r'\b' + re.escape(variant) + r'\b'
                if re.search(pattern, text_lower):
                    if canonical_metric not in entities['metrics']:
                        entities['metrics'].append(canonical_metric)
                    break
        
        # Extract ALL known players (improved detection)
        for canonical_name, variants in self.known_players.items():
            for variant in variants:
                # Use word boundary and case-insensitive matching
                pattern = r'\b' + re.escape(variant) + r'\b'
                if re.search(pattern, text_lower, re.IGNORECASE):
                    if canonical_name not in entities['players']:
                        entities['players'].append(canonical_name)
                    break
        
        # Extract capitalized names (for players not in database)
        if 'vs' in text_lower or 'versus' in text_lower or 'compare' in text_lower or 'and' in text:
            potential_players = self._extract_capitalized_names(text)
            for player in potential_players:
                if player not in entities['players']:
                    entities['players'].append(player)
        
        # Extract ALL positions
        for canonical_pos, variants in self.positions.items():
            for variant in variants:
                pattern = r'\b' + re.escape(variant) + r'\b'
                if re.search(pattern, text_lower):
                    if canonical_pos not in entities['positions']:
                        entities['positions'].append(canonical_pos)
                    break
        
        # Extract ALL teams (with word boundary matching)
        for canonical_team, variants in self.teams.items():
            for variant in variants:
                pattern = r'\b' + re.escape(variant) + r'\b'
                if re.search(pattern, text_lower):
                    if canonical_team not in entities['teams']:
                        entities['teams'].append(canonical_team)
                    break
        
        # Extract ALL seasons
        season_patterns = [
            r'20\d{2}[-/]20?\d{2}',  # 2022-23 or 2022/23
            r'20\d{2}'                # 2022
        ]
        for pattern in season_patterns:
            matches = re.findall(pattern, text)
            for match in matches:
                if match not in entities['seasons']:
                    entities['seasons'].append(match)
        
        # Extract ALL gameweeks (improved patterns)
        gw_patterns = [
            r'(?:gameweek|gw|week)\s*(\d+)',
            r'round\s*(\d+)',
            r'(\d+)\s*(?:st|nd|rd|th)\s*(?:gameweek|gw|week)'
        ]
        for pattern in gw_patterns:
            matches = re.findall(pattern, text_lower, re.IGNORECASE)
            for match in matches:
                if str(match) not in entities['gameweeks']:
                    entities['gameweeks'].append(str(match))
        
        # Extract ALL comparators
        comparators = [
            'more than', 'less than', 'greater than', 'fewer than',
            'under', 'over', 'above', 'below', 'at least', 'at most',
            'higher than', 'lower than'
        ]
        for comp in comparators:
            if comp in text_lower:
                if comp not in entities['comparators']:
                    entities['comparators'].append(comp)
        
        # Extract ALL numerical values (excluding seasons and gameweeks)
        number_pattern = r'\b(\d+(?:\.\d+)?)\b'
        numbers = re.findall(number_pattern, text)
        seen_numbers = set()
        
        for num in numbers:
            # Skip if already captured as season or gameweek
            is_season = any(season_pattern in num for season_pattern in ['202', '201', '200'])
            is_gameweek = num in entities['gameweeks']
            
            if not is_season and not is_gameweek:
                if num not in seen_numbers:
                    try:
                        entities['numerical_values'].append(float(num))
                        seen_numbers.add(num)
                    except ValueError:
                        pass
        
        return entities
    
    def _extract_capitalized_names(self, text: str) -> List[str]:
        """Extract potential player names from capitalized words with improved logic"""
        stopwords = {
            'who', 'what', 'find', 'show', 'tell', 'compare', 'the', 'vs', 'v', 'and', 'or',
            'in', 'for', 'of', 'did', 'are', 'have', 'has', 'is', 'how', 'which', 'where',
            'when', 'why', 'do', 'does', 'received', 'earned', 'had', 'players', 'playing',
            'play', 'between', 'against', 'premier', 'league', 'fantasy', 'fpl', 'scored',
            'score', 'gameweek', 'season', 'team', 'teams', 'a', 'an', 'with', 'from', 'to',
            'by', 'their', 'this', 'that', 'these', 'those', 'my', 'your', 'our', 'his', 'her'
        }
        
        words = re.split(r'(\s+|,|;|vs|versus|and|or)', text)
        potential_players = []
        current_name = []
        
        for i, word in enumerate(words):
            clean_word = word.strip().rstrip("'s,?!.:;-")
            
            # Check if word is a capitalized name component
            if (len(clean_word) >= 2 and clean_word[0].isupper() and 
                clean_word.lower() not in stopwords and not clean_word.isdigit()):
                
                # Handle hyphenated names (e.g., Calvert-Lewin)
                if '-' in clean_word:
                    parts = clean_word.split('-')
                    if all(len(p) >= 2 for p in parts):
                        potential_players.append(clean_word)
                        current_name = []
                    continue
                
                current_name.append(clean_word)
                
                # Check if next word is also capitalized (for multi-word names)
                if i + 1 < len(words):
                    next_word = words[i + 1].strip().rstrip("'s,?!.:;-")
                    if (len(next_word) >= 2 and next_word[0].isupper() and 
                        next_word.lower() not in stopwords):
                        continue
                
                # Complete the current name
                if len(current_name) >= 1:
                    player_name = ' '.join(current_name)
                    if len(player_name) >= 3:  # Minimum name length
                        potential_players.append(player_name)
                    current_name = []
        
        # Remove duplicates while preserving order
        seen = set()
        unique_players = []
        for player in potential_players:
            if player.lower() not in seen:
                unique_players.append(player)
                seen.add(player.lower())
        
        return unique_players
    
    def _merge_entities(self, llm_entities: Dict, rule_entities: Dict) -> Dict:
        """Merge LLM and rule-based entities intelligently"""
        merged = {}
        
        for key in rule_entities.keys():
            llm_vals = llm_entities.get(key, [])
            rule_vals = rule_entities.get(key, [])
            
            # Combine and deduplicate
            combined = []
            seen = set()
            
            # Prioritize LLM-extracted players (often more accurate)
            if key == 'players':
                # Add LLM players first
                for val in llm_vals:
                    if isinstance(val, str):
                        val_lower = val.lower()
                        if val_lower not in seen:
                            combined.append(val)
                            seen.add(val_lower)
                
                # Add rule-based players that weren't caught by LLM
                for val in rule_vals:
                    if isinstance(val, str):
                        val_lower = val.lower()
                        if val_lower not in seen:
                            combined.append(val)
                            seen.add(val_lower)
            else:
                # For other entities, combine all
                all_vals = llm_vals + rule_vals
                for val in all_vals:
                    if isinstance(val, (str, int, float)):
                        val_str = str(val).lower()
                        if val_str not in seen:
                            combined.append(val)
                            seen.add(val_str)
            
            merged[key] = combined
        
        return merged
    
    def generate_embedding(self, user_input: str) -> np.ndarray:
        """Generate semantic text embedding vector"""
        try:
            embedding = self.embedder.encode([user_input], convert_to_numpy=True)
            return embedding[0]
        except Exception as e:
            self.error_log.append({
                'type': 'embedding_error',
                'query': user_input,
                'error': str(e)
            })
            return np.zeros(self.embedder.get_sentence_embedding_dimension())
    
    def entities_to_criteria(self, entities: Dict, position: str = None) -> Dict[str, str]:
        """Convert extracted entities to search criteria"""
        criteria = {'total_points': 'high'}  # Default
        
        # Determine position
        positions_list = entities.get('positions', [])
        pos = position if position else (positions_list[0] if positions_list else None)
        
        # Position-specific default criteria
        if pos == 'FWD':
            criteria['goals'] = 'high'
            criteria['threat'] = 'high'
        elif pos == 'MID':
            criteria['goals'] = 'high'
            criteria['assists'] = 'high'
            criteria['creativity'] = 'high'
        elif pos == 'DEF':
            criteria['clean_sheets'] = 'high'
            criteria['bonus'] = 'high'
        elif pos == 'GK':
            criteria['clean_sheets'] = 'high'
            criteria['bonus'] = 'high'
            criteria['saves'] = 'high'
        
        # Override with explicit metrics from query
        metrics = entities.get('metrics', [])
        
        metric_to_criteria = {
            'goals_scored': 'goals',
            'total_points': 'total_points',
            'assists': 'assists',
            'clean_sheets': 'clean_sheets',
            'bonus': 'bonus',
            'minutes': 'minutes',
            'form': 'form',
            'ict_index': 'ict_index',
            'influence': 'influence',
            'creativity': 'creativity',
            'threat': 'threat',
            'cards': 'cards',
            'saves': 'saves',
            'yellow_cards': 'cards',
            'red_cards': 'cards',
            'value': 'value',
            'ownership': 'ownership'
        }
        
        for metric in metrics:
            if metric in metric_to_criteria:
                criteria[metric_to_criteria[metric]] = 'high'
        
        return criteria
    
    def generate_numeric_embedding(self, criteria: Dict) -> np.ndarray:
        """Generate a 12-dimensional numeric query embedding"""
        embedding = np.full(12, 0.5, dtype=np.float32)
        
        feature_map = {
            'goals': 0, 
            'assists': 1, 
            'total_points': 2,
            'clean_sheets': 3, 
            'minutes': 4, 
            'bonus': 5,
            'form': 6, 
            'ict_index': 7, 
            'influence': 8, 
            'creativity': 9, 
            'threat': 10, 
            'value': 11
        }
        
        for feature, value in criteria.items():
            if feature in feature_map:
                idx = feature_map[feature]
                if value == 'high':
                    embedding[idx] = 1.0
                elif value == 'low':
                    embedding[idx] = 0.0
                elif isinstance(value, (int, float)):
                    embedding[idx] = min(max(float(value), 0.0), 1.0)
        
        return embedding
    
    def preprocess(self, user_input: str, include_embedding: bool = True) -> Dict:
        """Complete preprocessing pipeline with enhanced entity extraction"""
        if not user_input or not user_input.strip():
            self.error_log.append({
                'type': 'empty_query',
                'query': user_input
            })
            return {
                'original_query': user_input,
                'intent': 'unknown',
                'intent_confidence': 0.0,
                'entities': {},
                'text_embedding': None,
                'numeric_embedding': None,
                'search_criteria': {},
                'method': 'none',
                'error': 'Empty query provided'
            }
        
        # Step 1: Classify intent
        intent, confidence = self.classify_intent(user_input)
        
        # Step 2: Extract ALL entities
        entities = self.extract_entities(user_input)
        
        # Step 3: Generate search criteria from entities
        search_criteria = self.entities_to_criteria(entities)
        
        result = {
            'original_query': user_input,
            'intent': intent,
            'intent_confidence': confidence,
            'entities': entities,
            'method': 'llm' if self.use_llm else 'rule-based',
            'search_criteria': search_criteria,
        }
        
        # Step 4: Generate embeddings if requested
        if include_embedding:
            result['text_embedding'] = self.generate_embedding(user_input)
            result['numeric_embedding'] = self.generate_numeric_embedding(search_criteria)
        else:
            result['text_embedding'] = None
            result['numeric_embedding'] = None
        
        result['embedding'] = result['text_embedding']
        
        return result
    
    def _convert_to_season_format(self, year_str: str) -> str:
        """Convert year to KG season format"""
        try:
            year = int(year_str)
            next_year = str(year + 1)[-2:]
            return f"{year}-{next_year}"
        except ValueError:
            return year_str
    
    def get_cypher_params(self, preprocessing_result: Dict) -> Dict[str, any]:
        """Convert entities to Cypher parameters with robust multi-entity support"""
        entities = preprocessing_result['entities']
        
        params = {
            'intent': preprocessing_result['intent'],
            'intent_confidence': preprocessing_result['intent_confidence']
        }
        
        # Handle multiple positions
        if entities.get('positions'):
            params['position'] = entities['positions'][0]
            if len(entities['positions']) > 1:
                params['positions'] = entities['positions']
        
        # Handle multiple teams
        if entities.get('teams'):
            params['team'] = entities['teams'][0]
            if len(entities['teams']) > 1:
                params['team2'] = entities['teams'][1]
                params['teams'] = entities['teams']
        
        # Handle multiple seasons
        if entities.get('seasons'):
            raw_season = entities['seasons'][0]
            params['season'] = self._convert_to_season_format(raw_season)
            if len(entities['seasons']) > 1:
                params['seasons'] = [self._convert_to_season_format(s) for s in entities['seasons']]
        
        # Handle multiple gameweeks
        if entities.get('gameweeks'):
            try:
                params['gameweek'] = int(entities['gameweeks'][0])
                if len(entities['gameweeks']) > 1:
                    params['gameweeks'] = [int(gw) for gw in entities['gameweeks']]
            except ValueError:
                pass
        
        # Handle multiple metrics
        if entities.get('metrics'):
            params['metric'] = entities['metrics'][0]
            if len(entities['metrics']) > 1:
                params['metric2'] = entities['metrics'][1]
                params['metrics'] = entities['metrics']
        
        # Handle multiple players (CRITICAL for comparisons)
        if entities.get('players'):
            params['player_name'] = entities['players'][0]
            if len(entities['players']) > 1:
                params['player_name2'] = entities['players'][1]
                params['player_names'] = entities['players']
        
        # Handle numerical values and comparators
        if entities.get('numerical_values'):
            params['threshold'] = entities['numerical_values'][0]
            if len(entities['numerical_values']) > 1:
                params['thresholds'] = entities['numerical_values']
        
        if entities.get('comparators'):
            params['comparator'] = entities['comparators'][0]
        
        return params
    
    def get_errors(self) -> List[Dict]:
        """Get error log for debugging"""
        return self.error_log
    
    def clear_errors(self):
        """Clear the error log"""
        self.error_log = []

Writing fpl_input_preprocessing.py


In [13]:
%%writefile fpl_Task2.py
"""
Milestone 3 - System Requirements Part 2: Graph Retrieval Layer
Implements:
  2.a - Baseline: Cypher query templates (10+ queries)
  2.b - Embeddings: Semantic similarity search (2 models for comparison)
"""

from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import Dict, List, Any
from fpl_input_preprocessing import FPLInputPreprocessorLLM as FPLInputPreprocessor


class FPLGraphRetrieval:
    """
    Graph Retrieval Layer for FPL Graph-RAG system.
    Combines baseline Cypher queries with embedding-based semantic search.
    """
    
    def __init__(self, neo4j_uri: str, neo4j_user: str, neo4j_password: str, 
                 hf_token: str = None, use_llm: bool = False):
        """
        Initialize Neo4j connection and embedding models.
        
        Args:
            neo4j_uri: Neo4j database URI
            neo4j_user: Neo4j username
            neo4j_password: Neo4j password
            hf_token: HuggingFace API token (required if use_llm=True)
            use_llm: Use LLM for intent/entity extraction (default: False for rule-based)
        """
        self.driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))
        
        self.preprocessor = FPLInputPreprocessor(
            hf_token=hf_token or "",
            use_llm=use_llm and hf_token is not None
        )
        
        # Two embedding models for comparison (requirement 2.b)
        self.embedding_models = {
            'model_1': SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2'),
            'model_2': SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
        }
        self.active_model = 'model_1'
        
        self._init_query_templates()
    
    def close(self):
        """Close Neo4j connection."""
        self.driver.close()

    # =========================================================================
    # SECTION 2.a: BASELINE - CYPHER QUERY TEMPLATES
    # =========================================================================
    
    def _init_query_templates(self):
        """Initialize 12 FIXED Cypher query templates for different intents."""
        self.query_templates = {
            
            'top_players_by_position': """
                // Top players by position for a specific season
                MATCH (gw:Gameweek {season: $season})<-[:HAS_FIXTURE]-(f:Fixture)
                MATCH (p:Player)-[:PLAYS_AS]->(pos:Position {name: $position})
                MATCH (p)-[played:PLAYED_IN]->(f)
                MATCH (p)-[:PLAYS_FOR]->(t:Team)
                RETURN p.player_name AS player,
                    t.name AS team,
                    SUM(played.total_points) AS total_points,
                    SUM(played.goals_scored) AS goals,
                    SUM(played.assists) AS assists,
                    COUNT(DISTINCT f) AS games_played
                ORDER BY total_points DESC
                LIMIT $limit
            """,
            
            'player_gameweek_performance': """
                // Player performance in specific gameweek
                MATCH (gw:Gameweek {season: $season, GW_number: $gameweek})<-[:HAS_FIXTURE]-(f:Fixture)
                MATCH (p:Player {player_name: $player_name})-[played:PLAYED_IN]->(f)
                MATCH (p)-[:PLAYS_FOR]->(t:Team)
                RETURN p.player_name AS player,
                    t.name AS team,
                    played.total_points AS points,
                    played.goals_scored AS goals,
                    played.assists AS assists,
                    played.minutes AS minutes,
                    played.bonus AS bonus,
                    played.clean_sheets AS clean_sheets,
                    played.ict_index AS ict_index
            """,
            
            'compare_players': """
                // Compare two players for a specific season - FIXED VERSION
                MATCH (p1:Player {player_name: $player_name})
                MATCH (p2:Player {player_name: $player_name2})
                
                // Get player 1 stats for the season
                OPTIONAL MATCH (p1)-[played1:PLAYED_IN]->(f1:Fixture)
                WHERE EXISTS {
                    MATCH (f1)<-[:HAS_FIXTURE]-(:Gameweek {season: $season})
                }
                
                // Get player 2 stats for the season
                OPTIONAL MATCH (p2)-[played2:PLAYED_IN]->(f2:Fixture)
                WHERE EXISTS {
                    MATCH (f2)<-[:HAS_FIXTURE]-(:Gameweek {season: $season})
                }
                
                // Get teams
                OPTIONAL MATCH (p1)-[:PLAYS_FOR]->(t1:Team)
                OPTIONAL MATCH (p2)-[:PLAYS_FOR]->(t2:Team)
                
                // Aggregate with DISTINCT to avoid duplicates
                WITH p1, t1, 
                    SUM(DISTINCT played1.total_points) AS p1_points,
                    SUM(DISTINCT played1.goals_scored) AS p1_goals,
                    SUM(DISTINCT played1.assists) AS p1_assists,
                    COUNT(DISTINCT f1) AS p1_games,
                    p2, t2,
                    SUM(DISTINCT played2.total_points) AS p2_points,
                    SUM(DISTINCT played2.goals_scored) AS p2_goals,
                    SUM(DISTINCT played2.assists) AS p2_assists,
                    COUNT(DISTINCT f2) AS p2_games
                
                RETURN p1.player_name AS player1, 
                    COALESCE(t1.name, 'Unknown') AS team1, 
                    COALESCE(p1_points, 0) AS p1_points,
                    COALESCE(p1_goals, 0) AS p1_goals,
                    COALESCE(p1_assists, 0) AS p1_assists,
                    p1_games,
                    
                    p2.player_name AS player2, 
                    COALESCE(t2.name, 'Unknown') AS team2, 
                    COALESCE(p2_points, 0) AS p2_points,
                    COALESCE(p2_goals, 0) AS p2_goals,
                    COALESCE(p2_assists, 0) AS p2_assists,
                    p2_games
            """,
            
            'team_fixtures': """
                // Team fixtures for a specific season
                MATCH (t:Team {name: $team})
                MATCH (f:Fixture)-[:HAS_HOME_TEAM|HAS_AWAY_TEAM]->(t)
                MATCH (f)<-[:HAS_FIXTURE]-(gw:Gameweek {season: $season})
                MATCH (f)-[:HAS_HOME_TEAM]->(home:Team)
                MATCH (f)-[:HAS_AWAY_TEAM]->(away:Team)
                RETURN gw.GW_number AS gameweek,
                    home.name AS home_team,
                    away.name AS away_team,
                    f.kickoff_time AS kickoff,
                    CASE WHEN home = t THEN 'home' ELSE 'away' END AS venue
                ORDER BY gw.GW_number
            """,
            
            'players_by_team': """
                // Players by team
                MATCH (t:Team {name: $team})
                MATCH (p:Player)-[:PLAYS_FOR]->(t)
                OPTIONAL MATCH (p)-[:PLAYS_AS]->(pos:Position)
                RETURN p.player_name AS player,
                    COALESCE(pos.name, 'Unknown') AS position,
                    p.player_element AS player_id
                ORDER BY position, p.player_name
            """,

            'player_team': """
                // Get team for a specific player
                MATCH (p:Player {player_name: $player_name})-[:PLAYS_FOR]->(t:Team)
                RETURN p.player_name AS player, 
                    t.name AS team,
                    p.player_element AS player_id
            """,
            
            'top_scorers': """
                // Top scorers for a specific season
                MATCH (gw:Gameweek {season: $season})<-[:HAS_FIXTURE]-(f:Fixture)
                MATCH (p:Player)-[played:PLAYED_IN]->(f)
                MATCH (p)-[:PLAYS_FOR]->(t:Team)
                RETURN p.player_name AS player,
                    t.name AS team,
                    SUM(played.goals_scored) AS goals,
                    SUM(played.total_points) AS points,
                    SUM(played.assists) AS assists,
                    COUNT(DISTINCT f) AS games_played
                ORDER BY goals DESC
                LIMIT $limit
            """,
            
            'top_assisters': """
                // Top assisters for a specific season
                MATCH (gw:Gameweek {season: $season})<-[:HAS_FIXTURE]-(f:Fixture)
                MATCH (p:Player)-[played:PLAYED_IN]->(f)
                MATCH (p)-[:PLAYS_FOR]->(t:Team)
                RETURN p.player_name AS player,
                    t.name AS team,
                    SUM(played.assists) AS assists,
                    SUM(played.total_points) AS points,
                    SUM(played.goals_scored) AS goals,
                    COUNT(DISTINCT f) AS games_played
                ORDER BY assists DESC
                LIMIT $limit
            """,
            
            'clean_sheet_leaders': """
                // Clean sheet leaders for a specific season (GK and DEF only)
                MATCH (gw:Gameweek {season: $season})<-[:HAS_FIXTURE]-(f:Fixture)
                MATCH (p:Player)-[:PLAYS_AS]->(pos:Position)
                WHERE pos.name IN ['GK', 'DEF']
                MATCH (p)-[played:PLAYED_IN]->(f)
                MATCH (p)-[:PLAYS_FOR]->(t:Team)
                RETURN p.player_name AS player,
                    pos.name AS position,
                    t.name AS team,
                    SUM(played.clean_sheets) AS clean_sheets,
                    SUM(played.total_points) AS points,
                    COUNT(DISTINCT f) AS games_played
                ORDER BY clean_sheets DESC
                LIMIT $limit
            """,
            
            'players_by_form': """
                // Players by recent form (last 5 gameweeks)
                MATCH (gw:Gameweek {season: $season})
                WHERE gw.GW_number >= $gameweek - 5 AND gw.GW_number <= $gameweek
                MATCH (gw)<-[:HAS_FIXTURE]-(f:Fixture)
                MATCH (p:Player)-[played:PLAYED_IN]->(f)
                MATCH (p)-[:PLAYS_FOR]->(t:Team)
                RETURN p.player_name AS player,
                    t.name AS team,
                    AVG(played.form) AS avg_form,
                    SUM(played.total_points) AS recent_points,
                    SUM(played.goals_scored) AS recent_goals,
                    SUM(played.assists) AS recent_assists,
                    COUNT(DISTINCT f) AS games_count
                ORDER BY avg_form DESC
                LIMIT $limit
            """,
            
            'player_season_summary': """
                // Player season summary
                MATCH (p:Player {player_name: $player_name})
                MATCH (gw:Gameweek {season: $season})<-[:HAS_FIXTURE]-(f:Fixture)
                MATCH (p)-[played:PLAYED_IN]->(f)
                MATCH (p)-[:PLAYS_AS]->(pos:Position)
                MATCH (p)-[:PLAYS_FOR]->(t:Team)
                RETURN p.player_name AS player,
                    pos.name AS position,
                    t.name AS team,
                    COUNT(DISTINCT f) AS games_played,
                    SUM(played.minutes) AS total_minutes,
                    SUM(played.total_points) AS total_points,
                    SUM(played.goals_scored) AS goals,
                    SUM(played.assists) AS assists,
                    SUM(played.clean_sheets) AS clean_sheets,
                    SUM(played.bonus) AS bonus_points,
                    AVG(played.ict_index) AS avg_ict,
                    AVG(played.form) AS avg_form
            """,
            
            'bonus_leaders': """
                // Bonus points leaders for a specific season
                MATCH (gw:Gameweek {season: $season})<-[:HAS_FIXTURE]-(f:Fixture)
                MATCH (p:Player)-[played:PLAYED_IN]->(f)
                MATCH (p)-[:PLAYS_FOR]->(t:Team)
                RETURN p.player_name AS player,
                    t.name AS team,
                    SUM(played.bonus) AS total_bonus,
                    SUM(played.total_points) AS total_points,
                    SUM(played.bps) AS bps,
                    COUNT(DISTINCT f) AS games_played
                ORDER BY total_bonus DESC
                LIMIT $limit
            """,
            
            'most_cards': """
                // Players with most cards (yellow/red) for a specific season
                MATCH (gw:Gameweek {season: $season})<-[:HAS_FIXTURE]-(f:Fixture)
                MATCH (p:Player)-[played:PLAYED_IN]->(f)
                MATCH (p)-[:PLAYS_FOR]->(t:Team)
                RETURN p.player_name AS player,
                    t.name AS team,
                    SUM(played.yellow_cards) AS yellows,
                    SUM(played.red_cards) AS reds,
                    SUM(played.yellow_cards) + SUM(played.red_cards) * 2 AS card_score,
                    COUNT(DISTINCT f) AS games_played
                ORDER BY card_score DESC
                LIMIT $limit
            """
        }
        
        # Map intents to query templates (ordered by preference)
        self.intent_to_query = {
        'recommendation': ['top_scorers', 'top_assisters', 'clean_sheet_leaders', 'bonus_leaders', 'top_players_by_position'],
        'performance_query': ['player_gameweek_performance', 'player_season_summary', 'players_by_form'],
        'comparison': ['compare_players', 'top_scorers', 'player_season_summary'],
        'player_search': ['player_season_summary', 'player_gameweek_performance', 'players_by_form'],
        'fixture_query': ['team_fixtures', 'players_by_team'],
        'form_query': ['players_by_form', 'player_season_summary', 'top_scorers'],
        'team_analysis': ['players_by_team', 'team_fixtures', 'top_scorers'],
        'value_analysis': ['top_players_by_position', 'players_by_form', 'bonus_leaders'],
        'general_query': ['top_scorers', 'top_assisters', 'bonus_leaders', 'clean_sheet_leaders'],
        'player_team_query': ['player_team', 'player_season_summary', 'players_by_team']
        }
        self.intent_primary_query = {
        'recommendation': 'top_scorers',
        'performance_query': 'player_gameweek_performance',
        'comparison': 'compare_players',  # This is the bug! Should be 'compare_players'
        'player_search': 'player_season_summary',
        'fixture_query': 'team_fixtures',
        'form_query': 'players_by_form',
        'team_analysis': 'players_by_team',
        'player_team_query': 'player_team',
        'value_analysis': 'top_players_by_position',
        'general_query': 'top_scorers'
    }
    
    def select_query(self, intent: str, entities: Dict) -> str:
        """Select appropriate query template based on intent and available entities.
        
        Priority order (highest to lowest):
        1. Intent-specific with complete entities (highest confidence)
        2. Entity combinations that clearly indicate a specific query type
        3. Intent-based fallback
        4. Generic fallback
        """
        
        metrics = entities.get('metrics', [])
        positions = entities.get('positions', [])
        teams = entities.get('teams', [])
        players = entities.get('players', [])
        gameweeks = entities.get('gameweeks', [])
        
        # =====================================================================
        # PRIORITY 1: HIGH-CONFIDENCE INTENT + ENTITY COMBINATIONS
        # =====================================================================
        
        # 1A: PERFORMANCE QUERY - Single player + specific gameweek
        if intent == 'performance_query' and len(players) >= 1 and len(gameweeks) >= 1:
            return 'player_gameweek_performance'
        
        # 1B: COMPARISON QUERY - Exactly 2 players (no gameweek needed for comparison)
        if intent == 'comparison' and len(players) == 2:
            return 'compare_players'
        
        # 1C: PLAYER SEARCH - Single player, no gameweek
        if intent == 'player_search' and len(players) >= 1 and len(gameweeks) == 0:
            return 'player_season_summary'
        
        # =====================================================================
        # PRIORITY 2: CLEAR ENTITY PATTERNS (regardless of intent)
        # =====================================================================
        
        # 2A: Single player + gameweek = ALWAYS gameweek performance
        if len(players) == 1 and len(gameweeks) >= 1:
            return 'player_gameweek_performance'
        
        # 2B: Two players = comparison (but check if it's REALLY a comparison)
        if len(players) == 2:
            # Extra validation: check if query has comparison words
            query_text = str(entities).lower()
            comparison_words = ['vs', 'versus', 'compare', 'comparison', 'versus']
            if any(word in query_text for word in comparison_words):
                return 'compare_players'
        
                # Two players without comparison words? Might be a mistake
                # Fall through to lower priority checks
        
        # 2C: Team + gameweek = team fixtures
        if len(teams) >= 1 and len(gameweeks) >= 1:
            return 'team_fixtures'
        
        # 2D: Team alone (depends on context)
        if len(teams) >= 1:
            # If asking about "players on team"
            if 'players' in str(entities).lower() or 'roster' in str(entities).lower():
                return 'players_by_team'
            return 'team_fixtures'  # Default for team queries
        
        # 2E: Position alone = top players by position
        if len(positions) >= 1 and len(players) == 0:
            return 'top_players_by_position'
        
        # =====================================================================
        # PRIORITY 3: INTENT-BASED FALLBACK (using corrected mapping)
        # =====================================================================
        
        intent_query_map = {
            'recommendation': 'top_scorers',
            'performance_query': 'player_gameweek_performance',
            'comparison': 'compare_players',
            'player_search': 'player_season_summary',
            'fixture_query': 'team_fixtures',
            'form_query': 'players_by_form',
            'team_analysis': 'players_by_team',
            'value_analysis': 'top_players_by_position',
            'general_query': 'top_scorers',
            'player_team_query': 'player_team'
        }
        
        if intent in intent_query_map:
            return intent_query_map[intent]
        
        # =====================================================================
        # PRIORITY 4: METRIC-SPECIFIC FALLBACK
        # =====================================================================
        
        metric_to_query = {
            'clean_sheets': 'clean_sheet_leaders',
            'assists': 'top_assisters',
            'goals_scored': 'top_scorers',
            'bonus': 'bonus_leaders',
            'cards': 'most_cards',
            'form': 'players_by_form',
            'value': 'top_players_by_position'
        }
        
        for metric in metrics:
            if metric in metric_to_query:
                return metric_to_query[metric]
        
        # =====================================================================
        # PRIORITY 5: SAFE GENERIC FALLBACK
        # =====================================================================
        return 'top_scorers'
    
    def execute_cypher(self, query_name: str, params: Dict) -> List[Dict]:
        """Execute a Cypher query template with parameters."""
        if query_name not in self.query_templates:
            raise ValueError(f"Unknown query template: {query_name}")
        
        query = self.query_templates[query_name]
        
        params.setdefault('limit', 10)
        params.setdefault('season', '2022-23')
        
        # Convert year to season format (e.g., '2022' -> '2022-23')
        if 'season' in params:
            season = params['season']
            if len(season) == 4 and season.isdigit():
                year = int(season)
                params['season'] = f"{year}-{str(year + 1)[-2:]}"
        
        with self.driver.session() as session:
            result = session.run(query, params)
            return [record.data() for record in result]
    
    def baseline_retrieve(self, user_input: str) -> Dict[str, Any]:
        """Baseline retrieval using Cypher queries only."""
        preprocessed = self.preprocessor.preprocess(user_input, include_embedding=False)
        params = self.preprocessor.get_cypher_params(preprocessed)
        query_name = self.select_query(preprocessed['intent'], preprocessed['entities'])
        
        try:
            results = self.execute_cypher(query_name, params)
            return {
                'method': 'baseline',
                'intent': preprocessed['intent'],
                'query_used': query_name,
                'parameters': params,
                'results': results,
                'cypher': self.query_templates[query_name]
            }
        except Exception as e:
            return {
                'method': 'baseline',
                'error': str(e),
                'intent': preprocessed['intent'],
                'query_used': query_name,
                'parameters': params
            }

    # =========================================================================
    # SECTION 2.b: NODE EMBEDDINGS - NUMERIC FEATURE VECTORS
    # =========================================================================
    #
    # APPROACH: Create numeric embeddings from player statistics
    # 
    # Why numeric embeddings for FPL?
    # - FPL data is purely numerical (goals, points, assists, etc.)
    # - No textual features to embed
    # - Direct numeric vectors preserve exact statistical relationships
    # - Faster computation than text-based embeddings
    #
    # Feature Vector Structure (12 dimensions):
    # [goals_norm, assists_norm, points_norm, clean_sheets_norm, minutes_norm,
    #  bonus_norm, form_norm, ict_norm, influence_norm, creativity_norm, 
    #  threat_norm, games_norm]
    #
    # Each feature is normalized to [0, 1] range for fair comparison
    # =========================================================================
    
    def set_embedding_model(self, model_name: str):
        """Switch between embedding models for comparison."""
        if model_name not in self.embedding_models:
            raise ValueError(f"Unknown model: {model_name}. Available: {list(self.embedding_models.keys())}")
        self.active_model = model_name
    
    def fetch_all_players_stats(self, season: str = None, use_per_game_avg: bool = True) -> List[Dict]:
        """
        Fetch ALL players with their stats and LATEST position.
        
        How it works:
        1. Match each unique player by player_element (unique ID)
        2. Get ONE position per player (collect and take first)
        3. Option A (use_per_game_avg=True): Compute PER-GAME AVERAGES
           - This ensures players with high performance per game rank higher
           - Avoids bias toward veterans who played more seasons
        4. Option B (use_per_game_avg=False): Use totals (for season-specific queries)
        5. Return one row per player (no duplicates)
        
        Args:
            season: Optional season filter (e.g., '2022-23'). If None, aggregates all.
            use_per_game_avg: If True, computes per-game averages for fair comparison.
        
        Returns:
            List of player dictionaries with stats
        """
        # Build season filter if provided
        season_filter = ""
        if season:
            season_filter = "MATCH (f)<-[:HAS_FIXTURE]-(gw:Gameweek {season: $season})"
        
        if use_per_game_avg:
            # Use per-game averages - FAIR comparison across players with different game counts
            query = f"""
                // Get all unique players
                MATCH (p:Player)
                
                // Get position (take first if multiple)
                OPTIONAL MATCH (p)-[:PLAYS_AS]->(pos:Position)
                
                // Aggregate stats from fixtures
                OPTIONAL MATCH (p)-[played:PLAYED_IN]->(f:Fixture)
                {season_filter}
                
                // Group by player_element (unique ID) to avoid duplicates
                // Use PER-GAME AVERAGES for fair comparison
                WITH p.player_element AS player_id,
                     COLLECT(DISTINCT p.player_name)[0] AS player_name,
                     COLLECT(DISTINCT pos.name)[0] AS position,
                     COUNT(played) AS games_played,
                     // Per-game averages (more fair for embedding comparison)
                     COALESCE(AVG(played.total_points), 0) AS avg_points_per_game,
                     COALESCE(SUM(played.goals_scored) * 1.0 / NULLIF(COUNT(played), 0), 0) AS goals_per_game,
                     COALESCE(SUM(played.assists) * 1.0 / NULLIF(COUNT(played), 0), 0) AS assists_per_game,
                     COALESCE(SUM(played.clean_sheets) * 1.0 / NULLIF(COUNT(played), 0), 0) AS clean_sheets_per_game,
                     COALESCE(AVG(played.minutes), 0) AS avg_minutes,
                     COALESCE(AVG(played.bonus), 0) AS avg_bonus,
                     COALESCE(AVG(played.form), 0) AS form,
                     COALESCE(AVG(played.ict_index), 0) AS ict_index,
                     COALESCE(AVG(played.influence), 0) AS influence,
                     COALESCE(AVG(played.creativity), 0) AS creativity,
                     COALESCE(AVG(played.threat), 0) AS threat,
                     // Also keep totals for reference
                     COALESCE(SUM(played.total_points), 0) AS total_points,
                     COALESCE(SUM(played.goals_scored), 0) AS goals,
                     COALESCE(SUM(played.assists), 0) AS assists,
                     COALESCE(SUM(played.clean_sheets), 0) AS clean_sheets
                
                // Filter out any null entries and return
                WHERE player_name IS NOT NULL AND position IS NOT NULL AND games_played > 0
                RETURN player_id, player_name, position, games_played,
                       avg_points_per_game, goals_per_game, assists_per_game,
                       clean_sheets_per_game, avg_minutes, avg_bonus,
                       form, ict_index, influence, creativity, threat,
                       total_points, goals, assists, clean_sheets
                ORDER BY avg_points_per_game DESC
            """
        else:
            # Use totals - for season-specific comparisons
            query = f"""
                MATCH (p:Player)
                OPTIONAL MATCH (p)-[:PLAYS_AS]->(pos:Position)
                OPTIONAL MATCH (p)-[played:PLAYED_IN]->(f:Fixture)
                {season_filter}
                WITH p.player_element AS player_id,
                     COLLECT(DISTINCT p.player_name)[0] AS player_name,
                     COLLECT(DISTINCT pos.name)[0] AS position,
                     COALESCE(SUM(played.total_points), 0) AS total_points,
                     COALESCE(SUM(played.goals_scored), 0) AS goals,
                     COALESCE(SUM(played.assists), 0) AS assists,
                     COALESCE(SUM(played.clean_sheets), 0) AS clean_sheets,
                     COALESCE(SUM(played.minutes), 0) AS minutes,
                     COALESCE(SUM(played.bonus), 0) AS bonus,
                     COALESCE(AVG(played.form), 0) AS form,
                     COALESCE(AVG(played.ict_index), 0) AS ict_index,
                     COALESCE(AVG(played.influence), 0) AS influence,
                     COALESCE(AVG(played.creativity), 0) AS creativity,
                     COALESCE(AVG(played.threat), 0) AS threat,
                     COUNT(played) AS games_played
                WHERE player_name IS NOT NULL AND position IS NOT NULL
                RETURN player_id, player_name, position,
                       total_points, goals, assists, clean_sheets,
                       minutes, bonus, form, ict_index,
                       influence, creativity, threat, games_played
                ORDER BY total_points DESC
            """
        
        params = {'season': season} if season else {}
        
        with self.driver.session() as session:
            result = session.run(query, params)
            players = [dict(record) for record in result]
            
        print(f"📊 Fetched {len(players)} unique players" + (f" (per-game averages)" if use_per_game_avg else ""))
        return players
    
    def compute_normalization_stats(self, players: List[Dict]) -> Dict[str, Dict]:
        """
        Compute min/max for each feature to normalize to [0, 1] range.
        
        Why normalize?
        - Goals per game range: 0-1, Points per game range: 0-15
        - Without normalization, points would dominate similarity
        - Normalized: each feature contributes equally
        
        Uses PER-GAME AVERAGES for fair comparison across players with different game counts.
        
        Returns:
            Dictionary with min/max for each feature
        """
        # Use per-game average features for fair comparison
        features = ['goals_per_game', 'assists_per_game', 'avg_points_per_game', 
                    'clean_sheets_per_game', 'avg_minutes', 'avg_bonus', 
                    'form', 'ict_index', 'influence', 'creativity', 'threat', 'games_played']
        
        # Fallback features if per-game stats not available
        fallback_features = ['goals', 'assists', 'total_points', 'clean_sheets', 
                             'minutes', 'bonus', 'form', 'ict_index',
                             'influence', 'creativity', 'threat', 'games_played']
        
        stats = {}
        for i, feat in enumerate(features):
            # Try per-game feature first, fallback to totals
            values = [p.get(feat, 0) or p.get(fallback_features[i], 0) or 0 for p in players]
            stats[feat] = {
                'min': min(values),
                'max': max(values),
                'range': max(values) - min(values) if max(values) != min(values) else 1
            }
        
        return stats
    
    def create_numeric_embedding(self, player: Dict, norm_stats: Dict) -> np.ndarray:
        """
        Create a normalized numeric embedding vector for a player.
        
        Uses PER-GAME AVERAGES for fair comparison:
        - A striker with 1 goal per game ranks higher than one with 0.2 goals/game
        - This prevents bias toward players who simply played more games
        
        Process:
        1. Extract per-game average values (or fallback to totals)
        2. Normalize each to [0, 1] using min-max scaling
        3. Return as numpy array
        
        Formula: normalized = (value - min) / (max - min)
        
        Args:
            player: Player dictionary with stats
            norm_stats: Normalization statistics from compute_normalization_stats
            
        Returns:
            numpy array of shape (12,) with normalized features
        """
        # Per-game average features
        features = ['goals_per_game', 'assists_per_game', 'avg_points_per_game', 
                    'clean_sheets_per_game', 'avg_minutes', 'avg_bonus', 
                    'form', 'ict_index', 'influence', 'creativity', 'threat', 'games_played']
        
        # Fallback features if per-game not available
        fallback_features = ['goals', 'assists', 'total_points', 'clean_sheets', 
                             'minutes', 'bonus', 'form', 'ict_index',
                             'influence', 'creativity', 'threat', 'games_played']
        
        embedding = []
        for i, feat in enumerate(features):
            # Try per-game feature first, fallback to totals
            raw_value = player.get(feat, 0) or player.get(fallback_features[i], 0) or 0
            stats = norm_stats[feat]
            # Min-max normalization to [0, 1]
            normalized = (raw_value - stats['min']) / stats['range']
            embedding.append(normalized)
        
        return np.array(embedding, dtype=np.float32)
    
    def create_all_embeddings(self, players: List[Dict]) -> Dict[str, np.ndarray]:
        """
        Create embeddings for all players in batch.
        
        Returns:
            Dictionary mapping player_name to embedding vector
        """
        print("🔢 Computing normalization statistics...")
        norm_stats = self.compute_normalization_stats(players)
        
        print("📐 Creating embeddings for all players...")
        embeddings = {}
        for player in players:
            name = player['player_name']
            embeddings[name] = self.create_numeric_embedding(player, norm_stats)
        
        print(f"✅ Created {len(embeddings)} embeddings of dimension {len(list(embeddings.values())[0])}")
        return embeddings, norm_stats
    
    # ═════════════════════════════════════════════════════════════════════════
    # FIXTURE EMBEDDINGS
    # ═════════════════════════════════════════════════════════════════════════
    
    def fetch_all_fixtures(self, season: str = None) -> List[Dict]:
        """Fetch all fixtures with their stats."""
        season_filter = ""
        params = {}
        
        if season:
            season_filter = "MATCH (gw:Gameweek {season: $season})<-[:HAS_FIXTURE]-(f:Fixture)"
            params['season'] = season
        else:
            season_filter = "MATCH (f:Fixture)<-[:HAS_FIXTURE]-(gw:Gameweek)"
        
        query = f"""
            {season_filter}
            OPTIONAL MATCH (f)-[:HAS_HOME_TEAM]->(home:Team)
            OPTIONAL MATCH (f)-[:HAS_AWAY_TEAM]->(away:Team)
            WITH f, gw, home.name AS home_team, away.name AS away_team
            OPTIONAL MATCH (p)-[played:PLAYED_IN]->(f)
            WITH f, gw, home_team, away_team,
                 COUNT(DISTINCT p) AS players_in_fixture,
                 AVG(played.total_points) AS avg_points,
                 MAX(played.total_points) AS max_points,
                 MIN(played.total_points) AS min_points
            RETURN DISTINCT 
                   f.fixture_number AS fixture_number,
                   gw.GW_number AS gameweek,
                   gw.season AS season,
                   home_team,
                   away_team,
                   players_in_fixture,
                   avg_points,
                   max_points,
                   min_points
        """
        
        with self.driver.session() as session:
            result = session.run(query, params)
            return [dict(record) for record in result]
    
    def create_fixture_embedding(self, fixture: Dict) -> np.ndarray:
        """Create embedding for a fixture based on team strength and match context."""
        # 8-dimensional embedding for fixtures:
        # [gameweek_norm, home_strength, away_strength, avg_points, max_points, min_points, num_players, fixture_quality]
        
        embedding = np.zeros(8, dtype=np.float32)
        
        # Normalize gameweek (assuming max 38 gameweeks)
        gameweek = fixture.get('gameweek', 1)
        embedding[0] = min(gameweek / 38.0, 1.0)
        
        # Team strength (proxy: use team name hash for consistency)
        home_team = fixture.get('home_team', '')
        away_team = fixture.get('away_team', '')
        
        # Hash teams to get pseudo-strength (0-1)
        home_strength = (hash(home_team) % 100) / 100.0 if home_team else 0.5
        away_strength = (hash(away_team) % 100) / 100.0 if away_team else 0.5
        
        embedding[1] = home_strength
        embedding[2] = away_strength
        
        # Points statistics (normalized to 0-1 range, assuming max ~100 points)
        embedding[3] = min((fixture.get('avg_points', 0) or 0) / 100.0, 1.0)
        embedding[4] = min((fixture.get('max_points', 0) or 0) / 150.0, 1.0)
        embedding[5] = min((fixture.get('min_points', 0) or 0) / 50.0, 1.0)
        
        # Number of players in fixture (normalized, assuming max ~1000)
        num_players = fixture.get('players_in_fixture', 0) or 0
        embedding[6] = min(num_players / 1000.0, 1.0)
        
        # Fixture "quality" - higher when there's good scoring potential
        quality = ((embedding[3] + embedding[4]) / 2.0) * (1.0 + embedding[6] / 2.0)
        embedding[7] = min(quality, 1.0)
        
        return embedding
    
    def store_fixture_embeddings_in_neo4j(self, season: str = None) -> Dict:
        """Store fixture embeddings in Neo4j."""
        fixtures = self.fetch_all_fixtures(season)
        
        if not fixtures:
            return {'error': 'No fixtures found', 'count': 0}
        
        # Create embeddings
        embeddings = {}
        for fixture in fixtures:
            fixture_key = (fixture['fixture_number'], fixture['season'])
            embeddings[fixture_key] = self.create_fixture_embedding(fixture)
        
        # Store in Neo4j
        update_query = """
            MATCH (f:Fixture {fixture_number: $fixture_number})
            WHERE (f)<-[:HAS_FIXTURE]-(gw:Gameweek {season: $season})
            SET f.embedding = $embedding,
                f.embedding_type = 'fixture',
                f.embedding_dim = 8
        """
        
        stored = 0
        with self.driver.session() as session:
            for fixture in fixtures:
                fixture_key = (fixture['fixture_number'], fixture['season'])
                if fixture_key in embeddings:
                    try:
                        session.run(update_query, {
                            'fixture_number': fixture['fixture_number'],
                            'season': fixture['season'],
                            'embedding': embeddings[fixture_key].tolist()
                        })
                        stored += 1
                    except Exception as e:
                        print(f"⚠️ Error storing fixture {fixture['fixture_number']}: {e}")
        
        return {
            'fixtures_processed': len(fixtures),
            'embeddings_stored': stored,
            'embedding_dimensions': 8
        }
    
    # ═════════════════════════════════════════════════════════════════════════
    # GAMEWEEK EMBEDDINGS
    # ═════════════════════════════════════════════════════════════════════════
    
    def fetch_all_gameweeks(self, season: str = None) -> List[Dict]:
        """Fetch all gameweeks with their stats."""
        season_filter = ""
        params = {}
        
        if season:
            season_filter = "WHERE gw.season = $season"
            params['season'] = season
        
        query = f"""
            MATCH (gw:Gameweek)
            {season_filter}
            OPTIONAL MATCH (gw)<-[:HAS_FIXTURE]-(f:Fixture)
            OPTIONAL MATCH (p)-[played:PLAYED_IN]->(f)
            WITH gw,
                 COUNT(DISTINCT f) AS num_fixtures,
                 COUNT(DISTINCT p) AS num_players,
                 AVG(played.total_points) AS avg_points,
                 MAX(played.total_points) AS max_points,
                 MIN(played.total_points) AS min_points,
                 STDEV(played.total_points) AS points_variance
            RETURN gw.GW_number AS gameweek,
                   gw.season AS season,
                   num_fixtures,
                   num_players,
                   avg_points,
                   max_points,
                   min_points,
                   COALESCE(points_variance, 0) AS points_variance
        """
        
        with self.driver.session() as session:
            result = session.run(query, params)
            return [dict(record) for record in result]
    
    def create_gameweek_embedding(self, gameweek: Dict) -> np.ndarray:
        """Create embedding for a gameweek based on match activity and scoring stats."""
        # 8-dimensional embedding for gameweeks:
        # [gameweek_norm, fixture_density, player_coverage, avg_points, max_points, min_points, variance, excitement]
        
        embedding = np.zeros(8, dtype=np.float32)
        
        # Gameweek number (normalized to 0-1, assuming max 38)
        gw_num = gameweek.get('gameweek', 1)
        embedding[0] = min(gw_num / 38.0, 1.0)
        
        # Fixture density (max 10 fixtures per gameweek)
        num_fixtures = gameweek.get('num_fixtures', 0) or 0
        embedding[1] = min(num_fixtures / 10.0, 1.0)
        
        # Player coverage (normalized, assuming ~1000 players across all fixtures)
        num_players = gameweek.get('num_players', 0) or 0
        embedding[2] = min(num_players / 1000.0, 1.0)
        
        # Points statistics (normalized)
        embedding[3] = min((gameweek.get('avg_points', 0) or 0) / 100.0, 1.0)
        embedding[4] = min((gameweek.get('max_points', 0) or 0) / 150.0, 1.0)
        embedding[5] = min((gameweek.get('min_points', 0) or 0) / 50.0, 1.0)
        
        # Points variance (normalized, high variance = more variability)
        variance = (gameweek.get('points_variance', 0) or 0)
        embedding[6] = min(variance / 100.0, 1.0)
        
        # "Excitement" score - combination of activity and variance
        excitement = (embedding[1] * 0.3 + embedding[2] * 0.3 + embedding[6] * 0.4)
        embedding[7] = excitement
        
        return embedding
    
    def store_gameweek_embeddings_in_neo4j(self, season: str = None) -> Dict:
        """Store gameweek embeddings in Neo4j."""
        gameweeks = self.fetch_all_gameweeks(season)
        
        if not gameweeks:
            return {'error': 'No gameweeks found', 'count': 0}
        
        # Create embeddings
        embeddings = {}
        for gw in gameweeks:
            gw_key = (gw['gameweek'], gw['season'])
            embeddings[gw_key] = self.create_gameweek_embedding(gw)
        
        # Store in Neo4j
        update_query = """
            MATCH (gw:Gameweek {GW_number: $gameweek, season: $season})
            SET gw.embedding = $embedding,
                gw.embedding_type = 'gameweek',
                gw.embedding_dim = 8
        """
        
        stored = 0
        with self.driver.session() as session:
            for gw in gameweeks:
                gw_key = (gw['gameweek'], gw['season'])
                if gw_key in embeddings:
                    try:
                        session.run(update_query, {
                            'gameweek': gw['gameweek'],
                            'season': gw['season'],
                            'embedding': embeddings[gw_key].tolist()
                        })
                        stored += 1
                    except Exception as e:
                        print(f"⚠️ Error storing gameweek {gw['gameweek']}: {e}")
        
        return {
            'gameweeks_processed': len(gameweeks),
            'embeddings_stored': stored,
            'embedding_dimensions': 8
        }
    
    def create_vector_index(self, embedding_type: str = 'numeric'):
        """
        Create a vector index in Neo4j for fast similarity search.
        
        Args:
            embedding_type: 'numeric' (12 dims), 'minilm' (384 dims), or 'mpnet' (768 dims)
        """
        dims_map = {'numeric': 12, 'minilm': 384, 'mpnet': 768}
        property_map = {'numeric': 'embedding', 'minilm': 'embedding_minilm', 'mpnet': 'embedding_mpnet'}
        
        dims = dims_map.get(embedding_type, 12)
        prop = property_map.get(embedding_type, 'embedding')
        index_name = f"player_{embedding_type}_embeddings"
        
        print(f"📁 Creating vector index '{index_name}' in Neo4j...")
        
        drop_query = f"DROP INDEX {index_name} IF EXISTS"
        create_query = f"""
            CREATE VECTOR INDEX {index_name} IF NOT EXISTS
            FOR (p:Player) ON (p.{prop})
            OPTIONS {{
                indexConfig: {{
                    `vector.dimensions`: {dims},
                    `vector.similarity_function`: 'cosine'
                }}
            }}
        """
        
        with self.driver.session() as session:
            try:
                session.run(drop_query)
            except:
                pass
            session.run(create_query)
        
        print(f"✅ Vector index created ({dims} dimensions, cosine similarity)")
    
    def create_text_representation(self, player: Dict) -> str:
        """Create text description from player stats for text embedding."""
        return (
            f"Football player {player.get('player_name', 'Unknown')} "
            f"plays as {player.get('position', 'Unknown')} position. "
            f"Season statistics: {player.get('total_points', 0)} total FPL points, "
            f"{player.get('goals', 0)} goals scored, "
            f"{player.get('assists', 0)} assists provided, "
            f"{player.get('clean_sheets', 0)} clean sheets. "
            f"Playing time: {player.get('minutes', 0)} minutes in {player.get('games_played', 0)} games. "
            f"Bonus points earned: {player.get('bonus', 0)}. "
            f"Performance metrics: form {player.get('form', 0):.2f}, "
            f"ICT index {player.get('ict_index', 0):.2f}."
        )
    
    def store_text_embeddings_in_neo4j(self, model_name: str = 'model_1', batch_size: int = 32) -> Dict:
        """
        Store TEXT-BASED embeddings using sentence transformers.
        
        This satisfies the requirement to "experiment with at least TWO different
        embedding models for comparison" - storing both MiniLM and MPNet embeddings.
        
        Args:
            model_name: 'model_1' (MiniLM, 384 dims) or 'model_2' (MPNet, 768 dims)
            batch_size: Batch size for encoding
            
        Returns:
            Dictionary with statistics
        """
        model_info = {
            'model_1': {'name': 'MiniLM', 'dims': 384, 'property': 'embedding_minilm'},
            'model_2': {'name': 'MPNet', 'dims': 768, 'property': 'embedding_mpnet'}
        }
        
        if model_name not in model_info:
            raise ValueError(f"Unknown model: {model_name}")
        
        info = model_info[model_name]
        
        print("=" * 70)
        print(f"STORING TEXT EMBEDDINGS ({info['name']}) IN NEO4J")
        print("=" * 70)
        
        # Fetch players
        print("📊 Fetching players...")
        players = self.fetch_all_players_stats()
        
        if not players:
            return {'error': 'No players found'}
        
        # Create text representations
        print("📝 Creating text representations...")
        texts = [self.create_text_representation(p) for p in players]
        
        # Generate embeddings in batches
        print(f"🔢 Generating embeddings with {info['name']} (batch_size={batch_size})...")
        model = self.embedding_models[model_name]
        
        all_embeddings = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            batch_emb = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
            all_embeddings.append(batch_emb)
            if (i + batch_size) % 200 == 0:
                print(f"   Encoded {min(i + batch_size, len(texts))}/{len(texts)} texts...")
        
        embeddings = np.vstack(all_embeddings)
        print(f"✅ Generated {len(embeddings)} embeddings of dimension {embeddings.shape[1]}")
        
        # Store in Neo4j
        print(f"💾 Storing embeddings as '{info['property']}' in Neo4j...")
        
        update_query = f"""
            MATCH (p:Player {{player_name: $player_name}})
            SET p.{info['property']} = $embedding,
                p.{info['property']}_model = $model_name
        """
        
        stored = 0
        with self.driver.session() as session:
            for i, player in enumerate(players):
                if i < len(embeddings):
                    session.run(update_query, {
                        'player_name': player['player_name'],
                        'embedding': embeddings[i].tolist(),
                        'model_name': info['name']
                    })
                    stored += 1
                    
                    if stored % 200 == 0:
                        print(f"   Stored {stored}/{len(players)} embeddings...")
        
        print(f"✅ Stored {stored} embeddings")
        
        # Create index
        idx_type = 'minilm' if model_name == 'model_1' else 'mpnet'
        self.create_vector_index(idx_type)
        
        print("=" * 70)
        
        return {
            'model': info['name'],
            'players_processed': len(players),
            'embeddings_stored': stored,
            'dimensions': info['dims']
        }
    
    def store_all_embeddings(self) -> Dict:
        """
        Store ALL THREE types of embeddings for complete comparison:
        1. Numeric (12 dims)
        2. MiniLM text (384 dims)
        3. MPNet text (768 dims)
        
        This fully satisfies the project requirement to experiment with
        at least TWO different embedding models.
        """
        print("=" * 80)
        print("STORING ALL EMBEDDING TYPES FOR COMPARISON")
        print("=" * 80)
        
        results = {}
        
        # 1. Numeric embeddings
        print("\n[1/3] NUMERIC EMBEDDINGS")
        results['numeric'] = self.store_embeddings_in_neo4j()
        
        # 2. MiniLM text embeddings
        print("\n[2/3] TEXT EMBEDDINGS (MiniLM)")
        results['minilm'] = self.store_text_embeddings_in_neo4j('model_1')
        
        # 3. MPNet text embeddings
        print("\n[3/3] TEXT EMBEDDINGS (MPNet)")
        results['mpnet'] = self.store_text_embeddings_in_neo4j('model_2')
        
        print("\n" + "=" * 80)
        print("ALL EMBEDDINGS STORED SUCCESSFULLY")
        print("=" * 80)
        
        print("\nSummary:")
        print(f"  • Numeric: {results['numeric'].get('embeddings_stored', 0)} players, 12 dims")
        print(f"  • MiniLM:  {results['minilm'].get('embeddings_stored', 0)} players, 384 dims")
        print(f"  • MPNet:   {results['mpnet'].get('embeddings_stored', 0)} players, 768 dims")
        
        return results
    
    def store_embeddings_in_neo4j(self) -> Dict:
        """
        Main function to create and store embeddings for ALL players.
        
        Process:
        1. Fetch all players with latest position (no duplicates)
        2. Compute normalization statistics
        3. Create numeric embeddings for each player
        4. Store embeddings in Neo4j Player nodes
        5. Create vector index for fast search
        
        Returns:
            Dictionary with statistics about the operation
        """
        print("=" * 70)
        print("STORING NUMERIC NODE EMBEDDINGS IN NEO4J")
        print("=" * 70)
        
        # Step 1: Fetch all players
        players = self.fetch_all_players_stats()
        
        if not players:
            return {'error': 'No players found', 'count': 0}
        
        # Step 2 & 3: Create embeddings
        embeddings, norm_stats = self.create_all_embeddings(players)
        
        # Step 4: Store in Neo4j
        print("💾 Storing embeddings in Neo4j...")
        
        update_query = """
            MATCH (p:Player {player_name: $player_name})
            SET p.embedding = $embedding,
                p.embedding_type = 'numeric',
                p.embedding_dim = 12
        """
        
        stored = 0
        with self.driver.session() as session:
            for player in players:
                name = player['player_name']
                if name in embeddings:
                    session.run(update_query, {
                        'player_name': name,
                        'embedding': embeddings[name].tolist()
                    })
                    stored += 1
                    
                    if stored % 200 == 0:
                        print(f"   Stored {stored}/{len(players)} embeddings...")
        
        print(f"✅ Stored {stored} embeddings in Neo4j")
        
        # Step 5: Create vector index
        self.create_vector_index()
        
        # Store normalization stats for later use
        self._norm_stats = norm_stats
        
        print("=" * 70)
        print("✅ EMBEDDING STORAGE COMPLETE")
        print("=" * 70)
        
        return {
            'players_processed': len(players),
            'embeddings_stored': stored,
            'embedding_dimensions': 12,
            'features': list(norm_stats.keys()),
            'normalization_stats': {k: {'min': v['min'], 'max': v['max']} 
                                    for k, v in norm_stats.items()}
        }
    
    def create_query_embedding(self, criteria: Dict) -> np.ndarray:
        """
        Create an embedding vector from search criteria.
        
        Example criteria:
        {'goals_per_game': 'high', 'assists_per_game': 'high', 'position': 'FWD'}
        
        Uses per-game average features to match the stored embeddings.
        Converts to normalized vector for similarity search.
        """
        # Default: mid-range values (0.5)
        embedding = np.full(12, 0.5, dtype=np.float32)
        
        # Feature indices (per-game averages)
        feature_map = {
            # Per-game features (primary)
            'goals_per_game': 0, 'assists_per_game': 1, 'avg_points_per_game': 2, 
            'clean_sheets_per_game': 3, 'avg_minutes': 4, 'avg_bonus': 5,
            'form': 6, 'ict_index': 7, 'influence': 8, 'creativity': 9, 
            'threat': 10, 'games_played': 11,
            # Legacy aliases (for backward compatibility)
            'goals': 0, 'assists': 1, 'total_points': 2, 'points': 2,
            'clean_sheets': 3, 'minutes': 4, 'bonus': 5
        }
        
        for feature, value in criteria.items():
            if feature in feature_map:
                idx = feature_map[feature]
                if value == 'high':
                    embedding[idx] = 1.0
                elif value == 'low':
                    embedding[idx] = 0.0
                elif isinstance(value, (int, float)):
                    embedding[idx] = min(max(value, 0), 1)
        
        return embedding
    
    def semantic_search(self, query_embedding: np.ndarray, top_k: int = 10, 
                    position: str = None, embedding_type: str = 'numeric') -> List[Dict]:
        """
        Find similar players using cosine similarity with any embedding type.
        
        Args:
            query_embedding: Query vector (dims depend on embedding_type)
            top_k: Number of results to return
            position: Optional position filter (GK, DEF, MID, FWD)
            embedding_type: 'numeric' (12d), 'minilm' (384d), or 'mpnet' (768d)
        """
        # Map embedding type to Neo4j property
        property_map = {
            'numeric': 'embedding',
            'minilm': 'embedding_minilm',
            'mpnet': 'embedding_mpnet'
        }
        
        if embedding_type not in property_map:
            raise ValueError(f"Unsupported embedding_type: {embedding_type}")
        
        emb_prop = property_map[embedding_type]
        
        # Build query
        if position:
            query = f"""
                MATCH (p:Player)
                WHERE p.{emb_prop} IS NOT NULL
                OPTIONAL MATCH (p)-[:PLAYS_AS]->(pos:Position {{name: $position}})
                WITH p.player_element AS player_id,
                    COLLECT(DISTINCT p.player_name)[0] AS player,
                    COLLECT(DISTINCT pos.name)[0] AS position,
                    p.{emb_prop} AS embedding
                WHERE player IS NOT NULL AND position = $position
                RETURN player, position, embedding
            """
            params = {'position': position}
        else:
            query = f"""
                MATCH (p:Player)
                WHERE p.{emb_prop} IS NOT NULL
                OPTIONAL MATCH (p)-[:PLAYS_AS]->(pos:Position)
                WITH p.player_element AS player_id,
                    COLLECT(DISTINCT p.player_name)[0] AS player,
                    COLLECT(DISTINCT pos.name)[0] AS position,
                    p.{emb_prop} AS embedding
                WHERE player IS NOT NULL
                RETURN player, position, embedding
            """
            params = {}
        
        with self.driver.session() as session:
            result = session.run(query, params)
            players = [dict(record) for record in result]
        
        if not players:
            return []
        
        # Compute cosine similarity
        query_norm = query_embedding / (np.linalg.norm(query_embedding) + 1e-10)
        
        similarities = []
        seen_players = set()
        
        for player in players:
            player_name = player['player']
            if player_name in seen_players:
                continue
            seen_players.add(player_name)
            
            player_emb = np.array(player['embedding'], dtype=np.float32)
            player_norm = player_emb / (np.linalg.norm(player_emb) + 1e-10)
            similarity = float(np.dot(query_norm, player_norm))
            
            similarities.append({
                'player': player_name,
                'position': player['position'],
                'similarity_score': round(similarity, 4),
                'embedding_type': embedding_type
            })
        
        similarities.sort(key=lambda x: x['similarity_score'], reverse=True)
        return similarities[:top_k]
    
    def search_with_text_embedding(self, query: str, model_name: str = 'model_1',
                                   top_k: int = 10, position: str = None) -> Dict:
        """
        Search using text embeddings (MiniLM or MPNet).
        
        Args:
            query: Natural language query
            model_name: 'model_1' (MiniLM) or 'model_2' (MPNet)
            top_k: Number of results
            position: Optional position filter
        """
        model = self.embedding_models[model_name]
        query_emb = model.encode([query], convert_to_numpy=True)[0]
        
        emb_type = 'minilm' if model_name == 'model_1' else 'mpnet'
        results = self.semantic_search(query_emb, top_k, position, emb_type)
        
        return {
            'method': f'text_{emb_type}',
            'model': model_name,
            'query': query,
            'position_filter': position,
            'results': results
        }
    
    def compare_all_embedding_models(self, position: str = 'FWD', top_k: int = 10) -> Dict:
        """
        Compare search results across all THREE embedding approaches.
        
        This is the main comparison function for the project requirement:
        "experiment with at least TWO different embedding models for comparison"
        
        Args:
            position: Position to filter (FWD, MID, DEF, GK)
            top_k: Number of results per approach
        """
        print("=" * 80)
        print(f"COMPARING ALL EMBEDDING MODELS (Position: {position})")
        print("=" * 80)
        
        results = {}
        
        # 1. Numeric search (criteria-based)
        print("\n📊 [1/3] Numeric Embeddings (12 dims)")
        criteria = {'goals': 'high', 'total_points': 'high', 'assists': 'high'}
        query_numeric = self.create_query_embedding(criteria)
        results['numeric'] = {
            'dims': 12,
            'criteria': criteria,
            'results': self.semantic_search(query_numeric, top_k, position, 'numeric')
        }
        
        # 2. MiniLM search (semantic)
        print("📊 [2/3] MiniLM Text Embeddings (384 dims)")
        query_text = f"Best {position} player with high goals and assists"
        results['minilm'] = self.search_with_text_embedding(query_text, 'model_1', top_k, position)
        results['minilm']['dims'] = 384
        
        # 3. MPNet search (semantic)
        print("📊 [3/3] MPNet Text Embeddings (768 dims)")
        results['mpnet'] = self.search_with_text_embedding(query_text, 'model_2', top_k, position)
        results['mpnet']['dims'] = 768
        
        # Display results
        print("\n" + "─" * 80)
        print("SEARCH RESULTS COMPARISON")
        print("─" * 80)
        
        print(f"\n{'Rank':<6} {'Numeric (12d)':<25} {'MiniLM (384d)':<25} {'MPNet (768d)':<25}")
        print("─" * 85)
        
        for i in range(min(top_k, 10)):
            num_res = results['numeric']['results'][i] if i < len(results['numeric']['results']) else {}
            mini_res = results['minilm']['results'][i] if i < len(results['minilm']['results']) else {}
            mpn_res = results['mpnet']['results'][i] if i < len(results['mpnet']['results']) else {}
            
            num_name = num_res.get('player', '-')[:22] if num_res else '-'
            mini_name = mini_res.get('player', '-')[:22] if mini_res else '-'
            mpn_name = mpn_res.get('player', '-')[:22] if mpn_res else '-'
            
            print(f"{i+1:<6} {num_name:<25} {mini_name:<25} {mpn_name:<25}")
        
        # Calculate agreement
        print("\n" + "─" * 80)
        print("MODEL AGREEMENT ANALYSIS")
        print("─" * 80)
        
        top10_numeric = set(r['player'] for r in results['numeric']['results'][:10])
        top10_minilm = set(r['player'] for r in results['minilm']['results'][:10])
        top10_mpnet = set(r['player'] for r in results['mpnet']['results'][:10])
        
        print(f"\n   Numeric ∩ MiniLM: {len(top10_numeric & top10_minilm)}/10 players agree")
        print(f"   Numeric ∩ MPNet:  {len(top10_numeric & top10_mpnet)}/10 players agree")
        print(f"   MiniLM ∩ MPNet:   {len(top10_minilm & top10_mpnet)}/10 players agree")
        print(f"   All three agree:  {len(top10_numeric & top10_minilm & top10_mpnet)}/10 players")
        
        print("\n" + "=" * 80)
        
        return results
    
    def embedding_retrieve(self, criteria: Dict = None, position: str = None, 
                        top_k: int = 10, embedding_type: str = 'numeric') -> Dict[str, Any]:
        """
        Retrieve similar players based on criteria using specified embedding type.
        
        Args:
            criteria: Dictionary of desired attributes
            position: Filter by position (GK, DEF, MID, FWD)
            top_k: Number of results
            embedding_type: 'numeric', 'minilm', or 'mpnet'
        """
        if criteria is None:
            criteria = {'total_points': 'high'}
        
        if embedding_type == 'numeric':
            # For numeric embeddings, create query vector from criteria
            query_embedding = self.create_query_embedding(criteria)
        else:
            # For text embeddings, convert criteria to text query
            query_text = self._criteria_to_text_query(criteria, position)
            model_name = 'model_1' if embedding_type == 'minilm' else 'model_2'
            model = self.embedding_models[model_name]
            query_embedding = model.encode([query_text], convert_to_numpy=True)[0]
        
        results = self.semantic_search(query_embedding, top_k, position, embedding_type)
        
        return {
            'method': f'{embedding_type}_embedding',
            'criteria': criteria,
            'position_filter': position,
            'embedding_type': embedding_type,
            'embedding_dimensions': query_embedding.shape[0],
            'results': results
        }
    
    def find_similar_players(self, player_name: str, top_k: int = 10, 
                             same_position: bool = True) -> List[Dict]:
        """
        Find players similar to a given player.
        
        How it works:
        1. Get the target player's embedding from Neo4j
        2. Search for players with similar embeddings
        3. Optionally filter to same position
        
        Args:
            player_name: Name of the player to find similar to
            top_k: Number of similar players to return
            same_position: If True, only return players of same position
        """
        # Get target player's embedding and position (handle multiple matches)
        query = """
            MATCH (p:Player {player_name: $player_name})
            OPTIONAL MATCH (p)-[:PLAYS_AS]->(pos:Position)
            WHERE p.embedding IS NOT NULL
            WITH p.embedding AS embedding, 
                 COLLECT(DISTINCT pos.name)[0] AS position
            RETURN embedding, position
            LIMIT 1
        """
        
        with self.driver.session() as session:
            result = session.run(query, {'player_name': player_name})
            record = result.single()
            
            if not record or record['embedding'] is None:
                return [{'error': f'Player "{player_name}" not found or has no embedding'}]
            
            target_embedding = np.array(record['embedding'], dtype=np.float32)
            target_position = record['position'] if same_position else None
        
        # Find similar players (excluding the target player)
        results = self.semantic_search(target_embedding, top_k + 1, target_position)
        
        # Remove the target player from results
        results = [r for r in results if r['player'] != player_name][:top_k]
        
        return results

    # =========================================================================
    # COMBINED RETRIEVAL
    # =========================================================================
    
    def retrieve(self, user_input: str, method: str = 'both', 
                embedding_type: str = 'numeric') -> Dict[str, Any]:
        """
        Main retrieval method combining baseline and embeddings.
        
        Args:
            user_input: Raw user query
            method: 'baseline', 'embedding', or 'both'
            embedding_type: 'numeric', 'minilm', or 'mpnet' - which embedding to use
        """
        results = {'user_input': user_input}
        
        # Always run baseline for intent/entity extraction
        baseline_result = self.baseline_retrieve(user_input)
        results['baseline'] = baseline_result
        
        preprocessed = self.preprocessor.preprocess(user_input, include_embedding=False)
        query_used = baseline_result.get('query_used', 'top_scorers')
        entities = preprocessed.get('entities', {})
        
        if method in ['embedding', 'both']:
            # Determine if this query type can benefit from embeddings
            # ALL player-centric queries should use embeddings
            player_centric_queries = {
                'top_players_by_position',      # ✓ Player similarity by position
                'top_scorers',                   # ✓ Player similarity by goals
                'top_assisters',                 # ✓ Player similarity by assists
                'clean_sheet_leaders',           # ✓ Defender/GK similarity
                'players_by_form',               # ✓ Player similarity by form
                'bonus_leaders',                 # ✓ Player similarity by bonus points
                'most_cards',                    # ✓ Player similarity by cards
                'player_season_summary',         # ✓ Find similar players to reference player
                'compare_players',               # ✓ Find similar players to both
                'players_by_team',               # ✓ Find similar players on other teams
                'player_gameweek_performance',   # ✓ Find similar players with similar performances
            }
            
            if query_used in player_centric_queries:
                # Extract position from query if available
                positions = entities.get('positions', [])
                position = positions[0] if positions else None
                
                # Get search criteria from preprocessed query
                search_criteria = preprocessed.get('search_criteria', {})
                
                if embedding_type == 'numeric':
                    # Use numeric embeddings
                    results['embedding'] = self.embedding_retrieve(
                        criteria=search_criteria, 
                        position=position, 
                        top_k=10
                    )
                elif embedding_type == 'minilm':
                    # Use MiniLM text embeddings
                    # Convert search criteria to natural language query
                    query_text = self._criteria_to_text_query(search_criteria, position)
                    results['embedding'] = self.search_with_text_embedding(
                        query=query_text,
                        model_name='model_1',
                        top_k=10,
                        position=position
                    )
                elif embedding_type == 'mpnet':
                    # Use MPNet text embeddings
                    query_text = self._criteria_to_text_query(search_criteria, position)
                    results['embedding'] = self.search_with_text_embedding(
                        query=query_text,
                        model_name='model_2',
                        top_k=10,
                        position=position
                    )
                
                results['embedding']['embedding_type'] = embedding_type
            else:
                results['embedding'] = {
                    'method': 'skipped',
                    'reason': f'Query "{query_used}" may not benefit from player embeddings',
                    'embedding_type': embedding_type
                }
        
        if method == 'both' and 'embedding' in results:
            # Merge results if embeddings were actually used
            embedding_results = results.get('embedding', {}).get('results', [])
            baseline_results = results.get('baseline', {}).get('results', [])
            
            if embedding_results:  # Only merge if embeddings returned results
                results['combined'] = self._merge_results(baseline_results, embedding_results)
            else:
                # Use only baseline results if embeddings skipped or empty
                results['combined'] = baseline_results
        
        return results

    def _criteria_to_text_query(self, criteria: Dict, position: str = None) -> str:
        """
        Convert search criteria to natural language query for text embeddings.
        
        Example: {'goals': 'high', 'assists': 'high'} -> 
                "Players with high goals and assists"
        """
        parts = []
        
        # Map criteria values to natural language
        value_map = {
            'high': 'high',
            'low': 'low',
            'good': 'good',
            'bad': 'poor'
        }
        
        # Map criteria keys to natural language
        criteria_map = {
            'goals': 'goals',
            'assists': 'assists', 
            'total_points': 'FPL points',
            'clean_sheets': 'clean sheets',
            'bonus': 'bonus points',
            'minutes': 'minutes played',
            'form': 'form',
            'ict_index': 'ICT index',
            'influence': 'influence',
            'creativity': 'creativity',
            'threat': 'threat',
            'cards': 'cards',
            'saves': 'saves',
            'value': 'value',
            'ownership': 'ownership'
        }
        
        for key, value in criteria.items():
            if key in criteria_map and value in value_map:
                parts.append(f"{value_map[value]} {criteria_map[key]}")
        
        # Build the query
        if position:
            position_map = {
                'FWD': 'forwards',
                'MID': 'midfielders', 
                'DEF': 'defenders',
                'GK': 'goalkeepers'
            }
            position_text = position_map.get(position, 'players')
            if parts:
                query = f"{position_text} with " + " and ".join(parts)
            else:
                query = f"best {position_text}"
        else:
            if parts:
                query = "Players with " + " and ".join(parts)
            else:
                query = "best players"
        
        return query    
    def _merge_results(self, baseline_results: List, embedding_results: List) -> List:
        """Merge results from both methods, removing duplicates."""
        combined = baseline_results.copy() if baseline_results else []
        seen_players = {r.get('player') for r in combined if r.get('player')}
        
        for result in (embedding_results or []):
            if result.get('player') not in seen_players:
                combined.append(result)
                seen_players.add(result.get('player'))
        
        return combined
    
    def compare_embedding_approaches(self, player_name: str = "Erling Haaland") -> Dict:
        """
        Compare numeric embeddings vs text-based embeddings.
        
        This demonstrates the two approaches:
        1. Numeric: Direct feature vectors [goals, assists, ...]
        2. Text-based: Convert stats to text, then embed with transformer
        """
        print("=" * 70)
        print("COMPARING EMBEDDING APPROACHES")
        print("=" * 70)
        
        # Get player stats
        query = """
            MATCH (p:Player {player_name: $name})-[:PLAYS_AS]->(pos:Position)
            OPTIONAL MATCH (p)-[played:PLAYED_IN]->(f:Fixture)
            RETURN p.player_name AS player_name,
                   pos.name AS position,
                   COALESCE(SUM(played.total_points), 0) AS total_points,
                   COALESCE(SUM(played.goals_scored), 0) AS goals,
                   COALESCE(SUM(played.assists), 0) AS assists,
                   p.embedding AS numeric_embedding
        """
        
        with self.driver.session() as session:
            result = session.run(query, {'name': player_name})
            record = result.single()
        
        if not record:
            return {'error': f'Player {player_name} not found'}
        
        player_data = dict(record)
        
        # Approach 1: Numeric embedding (already stored)
        numeric_emb = player_data.get('numeric_embedding')
        
        # Approach 2: Text-based embedding
        text_repr = (
            f"Player {player_data['player_name']} is a {player_data['position']} "
            f"with {player_data['goals']} goals, {player_data['assists']} assists, "
            f"and {player_data['total_points']} total points."
        )
        
        text_embeddings = {}
        for model_name, model in self.embedding_models.items():
            text_emb = model.encode([text_repr], convert_to_numpy=True)[0]
            text_embeddings[model_name] = {
                'dimensions': len(text_emb),
                'sample': text_emb[:5].tolist()
            }
        
        return {
            'player': player_name,
            'numeric_embedding': {
                'approach': 'Direct numeric features',
                'dimensions': 12,
                'features': ['goals', 'assists', 'points', 'clean_sheets', 
                            'minutes', 'bonus', 'form', 'ict', 'influence', 
                            'creativity', 'threat', 'games'],
                'values': numeric_emb[:5] if numeric_emb else None,
                'pros': ['Fast computation', 'Preserves exact relationships', 'Small storage'],
                'cons': ['Cannot handle semantic queries', 'Fixed features only']
            },
            'text_embeddings': {
                'approach': 'Text-based (sentence transformers)',
                'models': text_embeddings,
                'pros': ['Handles semantic queries', 'Flexible input'],
                'cons': ['Slower', 'Larger storage', 'Indirect representation']
            }
        }


# =============================================================================
# TESTING FUNCTIONS
# =============================================================================

def test_numeric_embeddings():
    """Test the numeric embedding functionality."""
    print("=" * 70)
    print("NUMERIC NODE EMBEDDINGS - TEST SUITE")
    print("=" * 70)
    
    # Initialize
    retriever = FPLGraphRetrieval(
        neo4j_uri=NEO4J_URI,
        neo4j_user=NEO4J_USER,
        neo4j_password=NEO4J_PASSWORD
    )
    
    # Test 1: Fetch all players
    print("\n" + "─" * 70)
    print("TEST 1: Fetch All Players (No Duplicates)")
    print("─" * 70)
    players = retriever.fetch_all_players_stats()
    print(f"✅ Total unique players: {len(players)}")
    
    # Check for duplicates
    names = [p['player_name'] for p in players]
    unique_names = set(names)
    print(f"✅ Unique names: {len(unique_names)} (duplicates: {len(names) - len(unique_names)})")
    
    # Position distribution
    positions = {}
    for p in players:
        pos = p['position']
        positions[pos] = positions.get(pos, 0) + 1
    print(f"✅ Position distribution: {positions}")
    
    # Test 2: Store embeddings
    print("\n" + "─" * 70)
    print("TEST 2: Store Embeddings in Neo4j")
    print("─" * 70)
    result = retriever.store_embeddings_in_neo4j()
    print(f"✅ Result: {result}")
    
    # Test 3: Search by criteria
    print("\n" + "─" * 70)
    print("TEST 3: Search by Criteria")
    print("─" * 70)
    
    test_searches = [
        {'criteria': {'goals': 'high', 'assists': 'high'}, 'position': 'FWD', 'desc': 'High-scoring forwards'},
        {'criteria': {'clean_sheets': 'high'}, 'position': 'GK', 'desc': 'Goalkeepers with clean sheets'},
        {'criteria': {'creativity': 'high', 'assists': 'high'}, 'position': 'MID', 'desc': 'Creative midfielders'},
        {'criteria': {'clean_sheets': 'high', 'bonus': 'high'}, 'position': 'DEF', 'desc': 'High-performing defenders'},
    ]
    
    for search in test_searches:
        print(f"\n🔍 {search['desc']} ({search['position']}):")
        results = retriever.embedding_retrieve(
            criteria=search['criteria'], 
            position=search['position'],
            top_k=5
        )
        for i, r in enumerate(results['results'][:5], 1):
            print(f"   {i}. {r['player']} ({r['position']}) - Similarity: {r['similarity_score']:.3f}")
    
    # Test 4: Find similar players
    print("\n" + "─" * 70)
    print("TEST 4: Find Similar Players")
    print("─" * 70)
    
    test_players = ["Erling Haaland", "Kevin De Bruyne", "Virgil van Dijk"]
    
    for player in test_players:
        print(f"\n🔍 Players similar to {player}:")
        similar = retriever.find_similar_players(player, top_k=5, same_position=True)
        if 'error' in similar[0] if similar else True:
            print(f"   ⚠️ {similar[0].get('error', 'Not found')}")
        else:
            for i, r in enumerate(similar[:5], 1):
                print(f"   {i}. {r['player']} ({r['position']}) - Similarity: {r['similarity_score']:.3f}")
    
    # Close connection
    retriever.close()
    
    print("\n" + "=" * 70)
    print("✅ ALL TESTS COMPLETE")
    print("=" * 70)


def test_plays_for_relationships():
    """Verify `PLAYS_FOR` relationships exist and are consistent with embeddings.

    - Runs a simple Cypher check: `MATCH p=()-[:PLAYS_FOR]->() RETURN p;`
    - Ensures that players which have embeddings in Neo4j also have a `PLAYS_FOR` relationship
    - Prints summary statistics and a small sample for manual inspection
    """
    print("\n" + "=" * 70)
    print("PLAYS_FOR RELATIONSHIP & EMBEDDING CONSISTENCY CHECK")
    print("=" * 70)

    retriever = FPLGraphRetrieval(
        neo4j_uri=NEO4J_URI,
        neo4j_user=NEO4J_USER,
        neo4j_password=NEO4J_PASSWORD
    )

    # Ensure embeddings exist (store numeric embeddings if missing)
    print("\n🔁 Ensuring numeric embeddings are stored (this may take a while)...")
    store_res = retriever.store_embeddings_in_neo4j()
    print(f"   Store result: {store_res}")

    # 1) Count PLAYS_FOR relationships and show a small sample
    with retriever.driver.session() as session:
        count_res = session.run("MATCH ()-[:PLAYS_FOR]->() RETURN count(*) AS rel_count")
        rel_count = count_res.single()['rel_count']

        sample_res = session.run(
            "MATCH (p:Player)-[r:PLAYS_FOR]->(t:Team) RETURN p.player_name AS player, t.name AS team LIMIT 10"
        )
        sample = [dict(rec) for rec in sample_res]

        # 2) Find how many players with embeddings are missing PLAYS_FOR
        missing_res = session.run(
            "MATCH (p:Player) WHERE p.embedding IS NOT NULL AND NOT (p)-[:PLAYS_FOR]->() RETURN count(p) AS missing"
        )
        missing = missing_res.single()['missing']

    print(f"\n✅ Total PLAYS_FOR relationships: {rel_count}")
    print(f"✅ Players with embeddings but missing PLAYS_FOR: {missing}")
    print("\nSample PLAYS_FOR rows:")
    for row in sample:
        print(f"   - {row['player']} -> {row['team']}")

    retriever.close()

    print("\n" + "=" * 70)
    print("PLAYS_FOR CHECK COMPLETE")
    print("=" * 70)


def test_all_embedding_approaches():
    """
    Compare ALL THREE embedding approaches:
    1. Numeric embeddings (direct feature vectors)
    2. Text embedding with MiniLM (model_1)
    3. Text embedding with MPNet (model_2)
    """
    import time
    
    print("=" * 80)
    print("COMPLETE EMBEDDING APPROACH COMPARISON")
    print("Numeric Features vs Text-Based (MiniLM) vs Text-Based (MPNet)")
    print("=" * 80)
    
    retriever = FPLGraphRetrieval(
        neo4j_uri=NEO4J_URI,
        neo4j_user=NEO4J_USER,
        neo4j_password=NEO4J_PASSWORD
    )
    
    print("\n📊 Fetching player data...")
    players = retriever.fetch_all_players_stats()[:200]
    print(f"   Loaded {len(players)} players for testing")
    
    # ─────────────────────────────────────────────────────────────
    print("\n" + "─" * 80)
    print("APPROACH SPECIFICATIONS")
    print("─" * 80)
    
    print("""
    ┌─────────────────────────────────────────────────────────────────────────┐
    │ NUMERIC FEATURES (12 dimensions)                                        │
    │ • Direct normalized stats: [goals, assists, points, clean_sheets, ...]  │
    │ • No text conversion needed                                             │
    │ • Fast and compact                                                      │
    ├─────────────────────────────────────────────────────────────────────────┤
    │ TEXT + MiniLM (384 dimensions)                                          │
    │ • Convert stats to text description                                     │
    │ • Embed with all-MiniLM-L6-v2 (22M params)                              │
    │ • Good speed/accuracy tradeoff                                          │
    ├─────────────────────────────────────────────────────────────────────────┤
    │ TEXT + MPNet (768 dimensions)                                           │
    │ • Convert stats to text description                                     │
    │ • Embed with all-mpnet-base-v2 (109M params)                            │
    │ • Best semantic understanding                                           │
    └─────────────────────────────────────────────────────────────────────────┘
    """)
    
    # ─────────────────────────────────────────────────────────────
    print("─" * 80)
    print("EMBEDDING GENERATION SPEED")
    print("─" * 80)
    
    embeddings = {}
    times = {}
    
    # 1. Numeric embeddings
    start = time.time()
    norm_stats = retriever.compute_normalization_stats(players)
    numeric_embs = [retriever.create_numeric_embedding(p, norm_stats) for p in players]
    times['numeric'] = time.time() - start
    embeddings['numeric'] = numeric_embs
    
    # 2. Create text representations
    texts = []
    for p in players:
        text = (f"Football player {p['player_name']} plays as {p['position']}. "
                f"Stats: {p['total_points']} points, {p['goals']} goals, "
                f"{p['assists']} assists, {p['clean_sheets']} clean sheets.")
        texts.append(text)
    
    # 3. MiniLM embeddings
    start = time.time()
    text_embs_minilm = retriever.embedding_models['model_1'].encode(texts, convert_to_numpy=True)
    times['minilm'] = time.time() - start
    embeddings['minilm'] = text_embs_minilm
    
    # 4. MPNet embeddings
    start = time.time()
    text_embs_mpnet = retriever.embedding_models['model_2'].encode(texts, convert_to_numpy=True)
    times['mpnet'] = time.time() - start
    embeddings['mpnet'] = text_embs_mpnet
    
    print(f"\n{'Approach':<20} {'Dims':<8} {'Time':<10} {'Speed':<15} {'Storage':<12}")
    print("─" * 70)
    specs = [
        ('Numeric', 12, times['numeric']),
        ('Text+MiniLM', 384, times['minilm']),
        ('Text+MPNet', 768, times['mpnet'])
    ]
    for name, dims, t in specs:
        storage = dims * 4  # bytes per float32
        print(f"{name:<20} {dims:<8} {t:<10.3f}s {len(players)/t:<15.0f}/sec {storage:<12} bytes")
    
    print(f"\n🏆 Numeric is {times['mpnet']/times['numeric']:.0f}x faster than MPNet")
    
    # ─────────────────────────────────────────────────────────────
    print("\n" + "─" * 80)
    print("SEARCH RESULTS COMPARISON")
    print("─" * 80)
    
    # Filter to forwards
    fwd_idx = [i for i, p in enumerate(players) if p['position'] == 'FWD']
    print(f"\nSearching among {len(fwd_idx)} forwards for 'top goal scorer'")
    
    # Numeric search
    print("\n📊 NUMERIC APPROACH (criteria: goals=high, points=high):")
    query_num = retriever.create_query_embedding({'goals': 'high', 'total_points': 'high'})
    q_norm = query_num / (np.linalg.norm(query_num) + 1e-10)
    
    num_results = []
    for i in fwd_idx:
        e = embeddings['numeric'][i]
        e_norm = e / (np.linalg.norm(e) + 1e-10)
        sim = float(np.dot(q_norm, e_norm))
        num_results.append((players[i]['player_name'], sim, players[i]['goals']))
    num_results.sort(key=lambda x: x[1], reverse=True)
    
    for rank, (name, sim, goals) in enumerate(num_results[:5], 1):
        print(f"   {rank}. {name:<25} Sim: {sim:.4f}  Goals: {goals}")
    
    # Text searches
    query_text = "Top scoring forward striker with many goals"
    
    for label, key, model_key in [('MiniLM', 'minilm', 'model_1'), ('MPNet', 'mpnet', 'model_2')]:
        print(f"\n📊 TEXT + {label} (query: \"{query_text}\"):")
        
        q_emb = retriever.embedding_models[model_key].encode([query_text], convert_to_numpy=True)[0]
        q_norm = q_emb / (np.linalg.norm(q_emb) + 1e-10)
        
        txt_results = []
        for i in fwd_idx:
            e = embeddings[key][i]
            e_norm = e / (np.linalg.norm(e) + 1e-10)
            sim = float(np.dot(q_norm, e_norm))
            txt_results.append((players[i]['player_name'], sim, players[i]['goals']))
        txt_results.sort(key=lambda x: x[1], reverse=True)
        
        for rank, (name, sim, goals) in enumerate(txt_results[:5], 1):
            print(f"   {rank}. {name:<25} Sim: {sim:.4f}  Goals: {goals}")
    
    # ─────────────────────────────────────────────────────────────
    print("\n" + "─" * 80)
    print("TOP-10 RANKING AGREEMENT")
    print("─" * 80)
    
    # Get top 10 from each for all players
    all_top10 = {}
    
    # Numeric
    q = retriever.create_query_embedding({'goals': 'high', 'total_points': 'high', 'assists': 'high'})
    q_n = q / (np.linalg.norm(q) + 1e-10)
    res = [(players[i]['player_name'], float(np.dot(q_n, embeddings['numeric'][i]/np.linalg.norm(embeddings['numeric'][i]+1e-10)))) for i in range(len(players))]
    res.sort(key=lambda x: x[1], reverse=True)
    all_top10['Numeric'] = [x[0] for x in res[:10]]
    
    # Text models
    q_txt = "Best player with high goals assists and points"
    for label, key, mkey in [('MiniLM', 'minilm', 'model_1'), ('MPNet', 'mpnet', 'model_2')]:
        q_emb = retriever.embedding_models[mkey].encode([q_txt], convert_to_numpy=True)[0]
        q_n = q_emb / (np.linalg.norm(q_emb) + 1e-10)
        res = []
        for i in range(len(players)):
            e = embeddings[key][i]
            e_n = e / (np.linalg.norm(e) + 1e-10)
            res.append((players[i]['player_name'], float(np.dot(q_n, e_n))))
        res.sort(key=lambda x: x[1], reverse=True)
        all_top10[label] = [x[0] for x in res[:10]]
    
    print(f"\n{'Rank':<6} {'Numeric':<22} {'MiniLM':<22} {'MPNet':<22}")
    print("─" * 75)
    for i in range(10):
        print(f"{i+1:<6} {all_top10['Numeric'][i]:<22} {all_top10['MiniLM'][i]:<22} {all_top10['MPNet'][i]:<22}")
    
    # Overlaps
    s1, s2, s3 = set(all_top10['Numeric']), set(all_top10['MiniLM']), set(all_top10['MPNet'])
    print(f"\n📊 Agreement:")
    print(f"   Numeric ∩ MiniLM: {len(s1&s2)}/10")
    print(f"   Numeric ∩ MPNet:  {len(s1&s3)}/10")
    print(f"   MiniLM ∩ MPNet:   {len(s2&s3)}/10")
    print(f"   All three:        {len(s1&s2&s3)}/10")
    
    retriever.close()
    
    print("\n" + "=" * 80)
    print("SUMMARY")
    print("=" * 80)
    print("""
    ╔═══════════════════════════════════════════════════════════════════════════╗
    ║ WHEN TO USE EACH APPROACH                                                 ║
    ╠═══════════════════════════════════════════════════════════════════════════╣
    ║                                                                           ║
    ║ NUMERIC EMBEDDINGS:                                                       ║
    ║   → "Find players with high goals and low price"                          ║
    ║   → "Who has the best form among midfielders?"                            ║
    ║   → Stats-based filtering and ranking                                     ║
    ║                                                                           ║
    ║ TEXT + MiniLM:                                                            ║
    ║   → "Find a creative playmaker"                                           ║
    ║   → "Who is similar to De Bruyne?"                                        ║
    ║   → Real-time semantic search with good accuracy                          ║
    ║                                                                           ║
    ║ TEXT + MPNet:                                                             ║
    ║   → "Find a clinical finisher who performs in big games"                  ║
    ║   → Complex semantic understanding                                        ║
    ║   → When accuracy matters more than speed                                 ║
    ║                                                                           ║
    ╚═══════════════════════════════════════════════════════════════════════════╝
    """)


def test_two_experiments():
    """
    Test the TWO EXPERIMENTS required by the project:
    
    Experiment 1: Baseline only (Cypher queries)
    Experiment 2: Baseline + Embeddings (combined results)
    """
    print("=" * 80)
    print("TWO EXPERIMENTS - BASELINE vs BASELINE + EMBEDDINGS")
    print("=" * 80)
    
    retriever = FPLGraphRetrieval(
        neo4j_uri=NEO4J_URI,
        neo4j_user=NEO4J_USER,
        neo4j_password=NEO4J_PASSWORD
    )

    # Ensure numeric embeddings are stored before running embedding experiments
    print("\n🔁 Ensuring numeric embeddings are stored (required for Experiment 2)...")
    store_info = retriever.store_embeddings_in_neo4j()
    print(f"   Embedding storage summary: {store_info}")
    
    test_queries = [
        "Who are the top forwards in 2022?",
        "Best midfielders with assists",
        "Top defenders this season",
        "Goalkeepers with clean sheets"
    ]
    
    for query in test_queries:
        print("\n" + "=" * 80)
        print(f"QUERY: \"{query}\"")
        print("=" * 80)
        
        # ─────────────────────────────────────────────────────────────
        # EXPERIMENT 1: BASELINE ONLY (Cypher Queries)
        # ─────────────────────────────────────────────────────────────
        print("\n📊 EXPERIMENT 1: BASELINE ONLY (Cypher Queries)")
        print("─" * 60)
        
        baseline_result = retriever.baseline_retrieve(query)
        
        print(f"   Intent: {baseline_result.get('intent')}")
        print(f"   Query Used: {baseline_result.get('query_used')}")
        print(f"   Parameters: {baseline_result.get('parameters')}")
        
        if baseline_result.get('error'):
            print(f"   ❌ Error: {baseline_result.get('error')}")
        else:
            print(f"   Results ({len(baseline_result.get('results', []))} found):")
            for i, r in enumerate(baseline_result.get('results', [])[:5], 1):
                player = r.get('player', r.get('player_name', 'Unknown'))
                points = r.get('total_points', r.get('points', '-'))
                print(f"      {i}. {player} - Points: {points}")

        # Post-check: verify that baseline result players have embeddings and PLAYS_FOR
        players_checked = baseline_result.get('results', [])[:10]
        with retriever.driver.session() as session:
            emb_missing = 0
            plays_for_missing = 0
            for p in players_checked:
                name = p.get('player') or p.get('player_name')
                rec = session.run(
                    "MATCH (p:Player {player_name: $name}) OPTIONAL MATCH (p)-[:PLAYS_FOR]->(t:Team) RETURN p.embedding IS NOT NULL AS has_emb, t.name AS team",
                    {'name': name}
                ).single()
                if not rec:
                    emb_missing += 1
                    plays_for_missing += 1
                else:
                    if not rec['has_emb']:
                        emb_missing += 1
                    if not rec['team']:
                        plays_for_missing += 1

        print(f"\n   Post-check: of top {len(players_checked)} baseline players -> missing embeddings: {emb_missing}, missing PLAYS_FOR: {plays_for_missing}")
        
        # ─────────────────────────────────────────────────────────────
        # EXPERIMENT 2: BASELINE + EMBEDDINGS (Combined)
        # ─────────────────────────────────────────────────────────────
        print("\n📊 EXPERIMENT 2: BASELINE + NUMERIC EMBEDDINGS")
        print("─" * 60)
        
        # Get embedding results
        preprocessed = retriever.preprocessor.preprocess(query, include_embedding=False)
        entities = preprocessed['entities']
        
        # Get position filter if available
        position = entities.get('positions', [None])[0]
        metrics = entities.get('metrics', [])
        
        # Create embedding criteria based on position and query
        # Use per-game average features (goals_per_game, etc.)
        criteria = {'avg_points_per_game': 'high'}
        
        # Position-specific criteria
        if position == 'FWD':
            # Forwards: prioritize goals
            criteria['goals_per_game'] = 'high'
            criteria['threat'] = 'high'
        elif position == 'MID':
            # Midfielders: balance goals and assists
            criteria['goals_per_game'] = 'high'
            criteria['assists_per_game'] = 'high'
            criteria['creativity'] = 'high'
        elif position == 'DEF':
            # Defenders: clean sheets and bonus
            criteria['clean_sheets_per_game'] = 'high'
            criteria['avg_bonus'] = 'high'
        elif position == 'GK':
            # Goalkeepers: clean sheets
            criteria['clean_sheets_per_game'] = 'high'
            criteria['avg_bonus'] = 'high'
        
        # Add explicit metric criteria from query
        if 'goals_scored' in metrics or 'goals' in str(query).lower():
            criteria['goals_per_game'] = 'high'
        if 'assists' in metrics or 'assists' in str(query).lower():
            criteria['assists_per_game'] = 'high'
        if 'clean_sheets' in metrics or 'clean' in str(query).lower():
            criteria['clean_sheets_per_game'] = 'high'
        
        embedding_result = retriever.embedding_retrieve(criteria=criteria, position=position, top_k=5)
        
        print(f"   Criteria: {embedding_result.get('criteria')}")
        print(f"   Position Filter: {embedding_result.get('position_filter')}")
        print(f"   Embedding Dims: {embedding_result.get('embedding_dimensions')}")
        print(f"   Results ({len(embedding_result.get('results', []))} found):")
        for i, r in enumerate(embedding_result.get('results', [])[:5], 1):
            print(f"      {i}. {r.get('player')} ({r.get('position')}) - Sim: {r.get('similarity_score', 0):.3f}")
        
        # ─────────────────────────────────────────────────────────────
        # COMBINED RESULTS
        # ─────────────────────────────────────────────────────────────
        print("\n📊 COMBINED RESULTS (Baseline + Embeddings merged)")
        print("─" * 60)
        
        baseline_players = baseline_result.get('results', [])
        embedding_players = embedding_result.get('results', [])
        
        combined = retriever._merge_results(baseline_players, embedding_players)
        
        print(f"   Baseline found: {len(baseline_players)} players")
        print(f"   Embeddings found: {len(embedding_players)} players")
        print(f"   Combined (deduplicated): {len(combined)} players")
        
        # Find overlap
        baseline_names = set(r.get('player', r.get('player_name', '')) for r in baseline_players[:10])
        embedding_names = set(r.get('player', '') for r in embedding_players[:10])
        overlap = baseline_names & embedding_names
        
        print(f"   Overlap (in top 10): {len(overlap)} players")
        if overlap:
            print(f"   Common players: {list(overlap)[:3]}")
    
    retriever.close()
    
    print("\n" + "=" * 80)
    print("✅ TWO EXPERIMENTS COMPLETE")
    print("=" * 80)
    print("""
    EXPERIMENT SUMMARY:
    
    ┌──────────────────────────────────────────────────────────────────────────┐
    │ EXPERIMENT 1: BASELINE ONLY                                              │
    │ • Uses Cypher queries based on intent classification                     │
    │ • Exact match filtering (position, season, etc.)                         │
    │ • Returns structured data from Knowledge Graph                           │
    ├──────────────────────────────────────────────────────────────────────────┤
    │ EXPERIMENT 2: BASELINE + EMBEDDINGS                                      │
    │ • Combines Cypher results with embedding similarity search               │
    │ • Numeric embeddings find statistically similar players                  │
    │ • Results are merged and deduplicated                                    │
    │ • Provides both exact matches AND similar alternatives                   │
    └──────────────────────────────────────────────────────────────────────────┘
    """)


def test_full_embedding_comparison():
    """
    Full test that stores ALL embedding types and compares them.
    This satisfies the project requirement for TWO embedding models.
    """
    print("=" * 80)
    print("FULL EMBEDDING MODEL COMPARISON TEST")
    print("Requirement: Experiment with at least TWO different embedding models")
    print("=" * 80)
    
    retriever = FPLGraphRetrieval(
        neo4j_uri=NEO4J_URI,
        neo4j_user=NEO4J_USER,
        neo4j_password=NEO4J_PASSWORD
    )
    
    # Step 1: Store ALL embedding types
    print("\n" + "─" * 80)
    print("STEP 1: STORE ALL EMBEDDING TYPES IN NEO4J")
    print("─" * 80)
    
    storage_results = retriever.store_all_embeddings()
    
    # Step 2: Compare search results across all models
    print("\n" + "─" * 80)
    print("STEP 2: COMPARE SEARCH RESULTS")
    print("─" * 80)
    
    for position in ['FWD', 'MID', 'DEF', 'GK']:
        print(f"\n{'='*80}")
        print(f"TESTING POSITION: {position}")
        print('='*80)
        retriever.compare_all_embedding_models(position=position, top_k=5)
    
    # Step 3: Find similar players comparison
    print("\n" + "─" * 80)
    print("STEP 3: FIND SIMILAR PLAYERS (ALL MODELS)")
    print("─" * 80)
    
    test_players = ["Erling Haaland", "Kevin De Bruyne", "Virgil van Dijk"]
    
    for player in test_players:
        print(f"\n🔍 Players similar to {player}:")
        
        # Numeric
        similar = retriever.find_similar_players(player, top_k=3, same_position=True)
        print(f"   Numeric: {[r['player'] for r in similar if 'error' not in r][:3]}")
    
    retriever.close()
    
    print("\n" + "=" * 80)
    print("✅ FULL COMPARISON TEST COMPLETE")
    print("=" * 80)
    print("""
    SUMMARY OF EMBEDDING MODELS COMPARED:
    
    ┌────────────────┬──────────┬─────────────────────────────────────────┐
    │ Model          │ Dims     │ Description                             │
    ├────────────────┼──────────┼─────────────────────────────────────────┤
    │ Numeric        │ 12       │ Direct normalized stats                 │
    │ MiniLM         │ 384      │ Text → sentence-transformers/all-MiniLM │
    │ MPNet          │ 768      │ Text → sentence-transformers/all-mpnet  │
    └────────────────┴──────────┴─────────────────────────────────────────┘
    
    This satisfies the requirement: "experiment with at least TWO different 
    embedding models for comparison" by comparing:
    1. MiniLM (all-MiniLM-L6-v2) - 384 dimensions
    2. MPNet (all-mpnet-base-v2) - 768 dimensions
    
    Additionally, we compare these TEXT embeddings with NUMERIC embeddings
    to show the tradeoffs between direct features vs semantic understanding.
    """)


def test_baseline_queries_run():
    """Run all 12 baseline Cypher queries with example parameters, verify
    they return results, and check that returned players have embeddings
    and a `PLAYS_FOR` relationship. Prints a concise report.
    """
    print("\n" + "=" * 80)
    print("RUNNING BASELINE QUERY TESTS (with embedding & PLAYS_FOR checks)")
    print("=" * 80)

    retriever = FPLGraphRetrieval(
        neo4j_uri=NEO4J_URI,
        neo4j_user=NEO4J_USER,
        neo4j_password=NEO4J_PASSWORD
    )

    # Ensure numeric embeddings exist for semantic checks
    print("\n🔁 Ensuring numeric embeddings are stored (required for embedding checks)...")
    try:
        store_info = retriever.store_embeddings_in_neo4j()
        print(f"   Embedding storage: {store_info}")
    except Exception as e:
        print(f"   ⚠️ Failed to store embeddings: {e}")

    # Define example parameters for each query template
    tests = [
        ('top_players_by_position', {'position': 'FWD', 'season': '2022-23', 'limit': 10}),
        ('player_gameweek_performance', {'player_name': 'Erling Haaland', 'gameweek': 20, 'season': '2022-23'}),
        ('compare_players', {'player_name': 'Erling Haaland', 'player_name2': 'Harry Kane', 'season': '2022-23'}),
        ('team_fixtures', {'team': 'Manchester City', 'season': '2022-23'}),
        ('players_by_team', {'team': 'Manchester City'}),
        ('top_scorers', {'season': '2022-23', 'limit': 10}),
        ('top_assisters', {'season': '2022-23', 'limit': 10}),
        ('clean_sheet_leaders', {'season': '2022-23', 'limit': 10}),
        ('players_by_form', {'season': '2022-23', 'gameweek': 20, 'limit': 10}),
        ('player_season_summary', {'player_name': 'Erling Haaland', 'season': '2022-23'}),
        ('bonus_leaders', {'season': '2022-23', 'limit': 10}),
        ('most_cards', {'season': '2022-23', 'limit': 10})
    ]

    summary = {}

    with retriever.driver.session() as session:
        for name, params in tests:
            print("\n" + "-" * 70)
            print(f"Running template: {name}  with params: {params}")
            try:
                results = retriever.execute_cypher(name, params)
            except Exception as e:
                print(f"   ❌ Query execution failed: {e}")
                summary[name] = {'status': 'error', 'error': str(e)}
                continue

            ok = bool(results)
            print(f"   Results returned: {len(results)}")

            # Extract up to 5 player names from results for per-player checks
            sample_players = []
            for r in results[:5]:
                # Common keys to look for
                if isinstance(r, dict):
                    if 'player' in r and r['player']:
                        sample_players.append(r['player'])
                    elif 'player_name' in r and r['player_name']:
                        sample_players.append(r['player_name'])
                    elif 'player1' in r and r['player1']:
                        sample_players.append(r['player1'])
                    elif 'player2' in r and r['player2']:
                        sample_players.append(r['player2'])
                    else:
                        # try any string-looking value
                        for v in r.values():
                            if isinstance(v, str) and len(v) > 1:
                                sample_players.append(v)
                                break

            sample_players = list(dict.fromkeys(sample_players))  # unique preserve order

            emb_missing = 0
            plays_for_missing = 0
            for pname in sample_players:
                rec = session.run(
                    "MATCH (p:Player {player_name: $name}) OPTIONAL MATCH (p)-[:PLAYS_FOR]->(t:Team) RETURN p.embedding IS NOT NULL AS has_emb, t.name AS team",
                    {'name': pname}
                ).single()
                if not rec:
                    emb_missing += 1
                    plays_for_missing += 1
                else:
                    if not rec['has_emb']:
                        emb_missing += 1
                    if not rec['team']:
                        plays_for_missing += 1

            summary[name] = {
                'results_count': len(results),
                'sample_players_checked': sample_players,
                'emb_missing_in_sample': emb_missing,
                'plays_for_missing_in_sample': plays_for_missing,
                'status': 'ok' if ok and emb_missing == 0 and plays_for_missing == 0 else 'warning' if ok else 'fail'
            }

            # Print quick per-query findings
            print(f"   Sample players checked: {sample_players}")
            print(f"   Embeddings missing in sample: {emb_missing}")
            print(f"   PLAYS_FOR missing in sample: {plays_for_missing}")

    retriever.close()

    print("\n" + "=" * 80)
    print("BASELINE QUERIES TEST SUMMARY")
    print("=" * 80)
    for k, v in summary.items():
        print(f"- {k}: status={v['status']}, results={v.get('results_count', 0)}, emb_missing={v.get('emb_missing_in_sample', 0)}, plays_for_missing={v.get('plays_for_missing_in_sample', 0)}")

    print("\nTests complete. Review warnings and errors above.")


def test_each_template_with_example_query():
    """Test each of the 12 baseline query templates with one hardcoded example query.
    
    For each template, this function:
    - Uses one realistic example query that will trigger that template
    - Calls retrieve(user_input, method='both')
    - Validates baseline/embedding/combined results exist and make sense
    - Checks up to 3 sample baseline players for p.embedding and PLAYS_FOR
    - Prints detailed per-template findings
    """
    print("\n" + "=" * 80)
    print("TESTING EACH TEMPLATE WITH HARDCODED EXAMPLE QUERIES")
    print("=" * 80)

    retriever = FPLGraphRetrieval(
        neo4j_uri=NEO4J_URI,
        neo4j_user=NEO4J_USER,
        neo4j_password=NEO4J_PASSWORD
    )

    # Ensure embeddings are stored before running tests
    print("\n🔁 Ensuring numeric embeddings are stored...")
    try:
        store_res = retriever.store_embeddings_in_neo4j()
        print(f"   Stored {store_res['embeddings_stored']} embeddings")
    except Exception as e:
        print(f"   ⚠️ Embedding storage failed: {e}")

    # Define one example query per template
    template_queries = {
        'top_players_by_position': "Who are the top forwards in 2022?",
        'player_gameweek_performance': "How did Erling Haaland perform in gameweek 20 of 2022-23?",
        'compare_players': "Compare Erling Haaland vs Harry Kane in 2022-23",
        'team_fixtures': "Show me Manchester City's fixtures in season 2022-23",
        'players_by_team': "Which players play for Manchester City?",
        'top_scorers': "Who are the top scorers in the 2022-23 season?",
        'top_assisters': "Who has the most assists in 2022-23?",
        'clean_sheet_leaders': "Which goalkeepers have the most clean sheets in 2022-23?",
        'players_by_form': "Who are the players with the best form around gameweek 20 in 2022-23?",
        'player_season_summary': "What is Erling Haaland's full season summary for 2022-23?",
        'bonus_leaders': "Who earned the most bonus points in 2022-23?",
        'most_cards': "Which players received the most cards (yellow/red) in 2022-23?"
    }

    results_summary = {}

    for template_name, example_query in template_queries.items():
        print("\n" + "=" * 80)
        print(f"TEMPLATE: {template_name}")
        print(f"QUERY: \"{example_query}\"")
        print("=" * 80)

        try:
            # Call retrieve with method='both' (baseline + embeddings)
            output = retriever.retrieve(example_query, method='both')
        except Exception as e:
            print(f"❌ ERROR: retrieve() raised exception: {e}")
            results_summary[template_name] = {
                'status': 'error',
                'error': str(e),
                'baseline_count': 0,
                'embedding_count': 0,
                'combined_count': 0,
                'notes': [f"exception: {str(e)}"]
            }
            continue

        baseline = output.get('baseline', {})
        embedding = output.get('embedding', {})
        combined = output.get('combined', [])

        # Extract counts
        baseline_results = baseline.get('results', []) if isinstance(baseline, dict) else []
        embedding_results = embedding.get('results', []) if isinstance(embedding, dict) else []

        baseline_count = len(baseline_results)
        embedding_count = len(embedding_results)
        combined_count = len(combined)

        print(f"\n📊 BASELINE RETRIEVAL:")
        print(f"   Intent: {baseline.get('intent', 'unknown')}")
        print(f"   Query Used: {baseline.get('query_used', 'unknown')}")
        print(f"   Results Count: {baseline_count}")
        if baseline_results:
            for i, r in enumerate(baseline_results[:3], 1):
                name = r.get('player') or r.get('player_name', 'Unknown')
                points = r.get('total_points') or r.get('points', '-')
                print(f"      {i}. {name} - Points: {points}")

        print(f"\n📊 EMBEDDING RETRIEVAL:")
        print(f"   Criteria: {embedding.get('criteria', {})}")
        print(f"   Position Filter: {embedding.get('position_filter')}")
        print(f"   Results Count: {embedding_count}")
        if embedding_results:
            for i, r in enumerate(embedding_results[:3], 1):
                name = r.get('player', 'Unknown')
                sim = r.get('similarity_score', 0)
                print(f"      {i}. {name} - Similarity: {sim:.3f}")

        print(f"\n📊 COMBINED RESULTS:")
        print(f"   Total (deduplicated): {combined_count}")

        # Check sample baseline players for embeddings and PLAYS_FOR
        sample_players = []
        for r in baseline_results[:3]:
            if isinstance(r, dict):
                name = r.get('player') or r.get('player_name') or r.get('player1') or r.get('player2')
                if name:
                    sample_players.append(name)

        emb_missing = 0
        plays_for_missing = 0
        player_status = []

        with retriever.driver.session() as session:
            for name in sample_players:
                rec = session.run(
                    "MATCH (p:Player {player_name: $name}) OPTIONAL MATCH (p)-[:PLAYS_FOR]->(t:Team) RETURN p.embedding IS NOT NULL AS has_emb, t.name AS team",
                    {'name': name}
                ).single()
                if not rec:
                    emb_missing += 1
                    plays_for_missing += 1
                    player_status.append({'player': name, 'has_embedding': False, 'has_plays_for': False})
                else:
                    has_emb = bool(rec['has_emb'])
                    team = rec['team']
                    if not has_emb:
                        emb_missing += 1
                    if not team:
                        plays_for_missing += 1
                    player_status.append({'player': name, 'has_embedding': has_emb, 'has_plays_for': bool(team)})

        print(f"\n✓ PLAYER VALIDATION (sample of {len(sample_players)}):")
        for ps in player_status:
            emb_icon = "✅" if ps['has_embedding'] else "❌"
            pf_icon = "✅" if ps['has_plays_for'] else "❌"
            print(f"   {ps['player']}: embedding {emb_icon}, PLAYS_FOR {pf_icon}")

        status = 'ok'
        notes = []

        if baseline_count == 0:
            notes.append('no baseline results')
            status = 'warning'
        if embedding_count == 0:
            notes.append('no embedding results')
            if status == 'ok':
                status = 'warning'
        if emb_missing > 0:
            notes.append(f'embedding missing: {emb_missing}')
        if plays_for_missing > 0:
            notes.append(f'PLAYS_FOR missing: {plays_for_missing}')

        results_summary[template_name] = {
            'status': status,
            'baseline_count': baseline_count,
            'embedding_count': embedding_count,
            'combined_count': combined_count,
            'sample_players_checked': len(sample_players),
            'embedding_missing': emb_missing,
            'plays_for_missing': plays_for_missing,
            'notes': notes
        }

        print(f"\n   Status: {status}, Notes: {notes}")

    retriever.close()

    # Print final summary
    print("\n" + "=" * 80)
    print("FINAL SUMMARY - ALL 12 TEMPLATES")
    print("=" * 80)
    print(f"\n{'Template':<35} {'Status':<10} {'Baseline':<10} {'Embedding':<10} {'Combined':<10}")
    print("-" * 95)
    for template, res in results_summary.items():
        status = res['status']
        baseline = res['baseline_count']
        embedding = res['embedding_count']
        combined = res['combined_count']
        print(f"{template:<35} {status:<10} {baseline:<10} {embedding:<10} {combined:<10}")

    print(f"\n{'Notes:'}")
    for template, res in results_summary.items():
        if res['notes']:
            print(f"  {template}: {', '.join(res['notes'])}")

    print("\n✅ TEMPLATE TEST COMPLETE")


if __name__ == "__main__":
    NEO4J_URI = "neo4j+s://1da86c19.databases.neo4j.io"
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD = "HA4iunTOGen7RYpeISs3ZRhcWjpcokqam9przCqCuQ8"
    
    # Initialize retriever
    retriever = FPLGraphRetrieval(
        neo4j_uri=NEO4J_URI,
        neo4j_user=NEO4J_USER,
        neo4j_password=NEO4J_PASSWORD
    )
    
    print("=" * 80)
    print("STORING ALL EMBEDDINGS IN NEO4J")
    print("=" * 80)
    
    # Store ALL embedding types
    results = retriever.store_all_embeddings()
    
    print("\n✅ ALL EMBEDDINGS STORED")
    print(f"Numeric: {results['numeric'].get('embeddings_stored')} players")
    print(f"MiniLM: {results['minilm'].get('embeddings_stored')} players")
    print(f"MPNet: {results['mpnet'].get('embeddings_stored')} players")
    
    # Optional: Run the template tests after
    print("\n" + "=" * 80)
    print("RUNNING TEMPLATE TESTS")
    print("=" * 80)
    test_each_template_with_example_query()


Overwriting fpl_Task2.py


In [23]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("preprocesskey")
secret_value_1 = user_secrets.get_secret("preprocesskey")
# secret_value_2 = user_secrets.get_secret("j")
# secret_value_3 = user_secrets.get_secret("LLM Token")
secret_value_4 = user_secrets.get_secret("openrouterkey")
secret_value_5 = user_secrets.get_secret("openrouterkey")
secret_value_6 = user_secrets.get_secret("preprocesskey")




# ─────────────────────────────────────────────────────────────
# REQUIRED PACKAGES (install if needed)
# ─────────────────────────────────────────────────────────────
# !pip install neo4j sentence-transformers

# ─────────────────────────────────────────────────────────────
# IMPORTS
# ─────────────────────────────────────────────────────────────
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import Dict, List, Any

# Import the main class
from fpl_Task2 import FPLGraphRetrieval

# ─────────────────────────────────────────────────────────────
# NEO4J CONFIGURATION
# ─────────────────────────────────────────────────────────────
NEO4J_URI = "neo4j+s://1da86c19.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "HA4iunTOGen7RYpeISs3ZRhcWjpcokqam9przCqCuQ8"

# ─────────────────────────────────────────────────────────────
# USAGE
# ─────────────────────────────────────────────────────────────
# Initialize
retriever = FPLGraphRetrieval(
    neo4j_uri=NEO4J_URI,
    neo4j_user=NEO4J_USER,
    neo4j_password=NEO4J_PASSWORD,
    hf_token=secret_value_1,  # Optional, for LLM mode
    use_llm=True  # Set True to use LLM for entity extraction
)




from openai import OpenAI
from kaggle_secrets import UserSecretsClient
def call_llm(user_question: str, model: str, embedding_type: str = 'numeric') -> str:
    """
    Takes user question, handles retrieval with specified embedding type.
    """

    # 1️⃣ Get API key
    user_secrets = UserSecretsClient()
    openrouter_key = user_secrets.get_secret("openrouterkey")

    # 2️⃣ Initialize OpenRouter client
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=openrouter_key
    )

    
    result = retriever.retrieve(user_question, method="both", embedding_type=embedding_type)

    # print("="*50)
    # print("RETRIEVAL DEBUG:")
    # print(f"Intent detected: {result.get('baseline', {}).get('intent')}")
    # print(f"Query used: {result.get('baseline', {}).get('query_used')}")
    # print(f"Parameters: {result.get('baseline', {}).get('parameters')}")
    # print(f"Results: {result.get('baseline', {}).get('results', [])[:2]}")  # First 2 results
    # print("="*50)
    print(result)
    

    context_text = result

    # 5️⃣ Define persona
    persona_text = (
    "You are an FPL (Fantasy Premier League) expert. "
    "You can answer any questions related to FPL, including player stats, "
    "team performance, transfers, and gameweek strategies. "
    "You are also knowledgeable about the Premier League in general."
    )

    task_text = (
    "IMPORTANT: You MUST always provide a response. Never return empty text.\n\n"
    
    "=== QUERY TYPE INSTRUCTIONS ===\n\n"
    
    "1. COMPARISON QUERIES (e.g., 'compare X and Y'):\n"
    "   • First check if BOTH players are mentioned in the CONTEXT\n"
    "   • Extract ALL their stats from the CONTEXT\n"
    "   • Create a detailed comparison table or bullet points\n"
    "   • Include: goals, points, assists, clean sheets, bonus points\n"
    "   • Provide a clear conclusion about who performed better\n"
    "   EXAMPLE:\n"
    "   Comparison: Player A vs Player B\n"
    "   • Player A: X goals, Y points, Z assists, W bonus\n"
    "   • Player B: A goals, B points, C assists, D bonus\n"
    "   Conclusion: [Who was better and why]\n\n"
    
    "2. TOP/RANKING QUERIES (e.g., 'top 10 scorers', 'best defenders'):\n"
    "   • Extract the exact ranking from the CONTEXT\n"
    "   • List players in numbered order (1, 2, 3...)\n"
    "   • Include key stats: goals/points/assists next to each name\n"
    "   • Format: '1. Player Name - X goals, Y points'\n"
    "   • If asking for top N, provide EXACTLY N players\n\n"
    
    "3. PLAYER SUMMARY QUERIES (e.g., 'how did X perform', 'X's stats'):\n"
    "   • Extract ALL available stats for that player from CONTEXT\n"
    "   • Include: total points, goals, assists, minutes, clean sheets, bonus, cards\n"
    "   • Present in bullet format or table\n"
    "   • Add brief analysis if stats are exceptional\n\n"
    
    "4. TEAM/FIXTURE QUERIES (e.g., 'team fixtures', 'upcoming matches'):\n"
    "   • List fixtures chronologically by gameweek\n"
    "   • Format: 'GW X: Home Team vs Away Team (Date)'\n"
    "   • Include scores if available in CONTEXT\n"
    "   • Group by team if querying specific team\n\n"
    
    "5. GAMEWEEK PERFORMANCE QUERIES (e.g., 'GW 20 performance'):\n"
    "   • Focus on specific gameweek data from CONTEXT\n"
    "   • Include: points scored, goals, assists, minutes played\n"
    "   • Mention opponent if available\n"
    "   • Note if player didn't play (0 minutes)\n\n"
    
    "6. LIST QUERIES (e.g., 'players on Manchester City'):\n"
    "   • Provide complete list from CONTEXT\n"
    "   • Group by position if relevant (GK, DEF, MID, FWD)\n"
    "   • Use bullet points or numbered list\n"
    "   • Include position labels\n\n"
    
    "7. STATISTICAL QUERIES (e.g., 'most cards', 'clean sheet leaders'):\n"
    "   • Focus on the specific statistic asked\n"
    "   • Rank players by that stat\n"
    "   • Show the actual numbers clearly\n"
    "   • Format: 'Player Name: X [stat]'\n\n"
    
    "=== GENERAL RULES ===\n"
    "• ALWAYS extract data from CONTEXT - never make up stats\n"
    "• If data is not in CONTEXT, say 'Information not available in context'\n"
    "• Use clear formatting: bullet points, numbering, or tables\n"
    "• Include units (points, goals, assists) with numbers\n"
    "• Be concise but complete - include all relevant data\n"
    "• For numerical data, preserve exact values from CONTEXT\n"
    "• If multiple entries exist (e.g., player played multiple GWs), show all\n\n"

    
    )

    # 7️⃣ Build the prompt
    prompt = f"""
    CONTEXT:
    {context_text}

    PERSONA:
    {persona_text}

    TASK:
    {task_text}
    """

    # 8️⃣ Call the LLM
    completion = client.chat.completions.create(
        model= model,
        messages=[
            {"role": "system", "content": persona_text},
            {"role": "user", "content": f"{prompt}\n\nQuestion: {user_question}"}
        ],
        temperature=0.2,
        max_tokens=500
    )

    # 9️⃣ Return the LLM answer

    answer = completion.choices[0].message.content

    # ✅ Token usage (access as attributes, not dict)
    prompt_tokens = completion.usage.prompt_tokens
    completion_tokens = completion.usage.completion_tokens
    total_tokens = completion.usage.total_tokens





    return completion.choices[0].message.content, total_tokens















def compare_models_human_evaluation():
    """
    Run 3 models on test queries and perform human evaluation with comparison.
    Self-contained function - no inputs required. Models and queries defined internally.
    
    Returns:
        Dictionary with evaluations and comparison table
    """
    import json
    import os
    import time
    
    # =========================================================================
    # CONFIGURATION - Edit these to customize your evaluation
    # =========================================================================
    
    models_dict = {
        "1": "mistralai/devstral-2512:free",
        "2":  "meta-llama/llama-3.3-70b-instruct:free",
        "3": "nvidia/nemotron-3-nano-30b-a3b:free",
    }
    
    test_queries = [
        {
            "question": "Who are the top 10 forwards in the 2022-23 season?",
            "ground_truth": """
Erling Haaland,272
Harry Kane,263
Ivan Toney,182
Ollie Watkins,175
Callum Wilson,157
Bryan Mbeumo,150
Dominic Solanke,130
Gabriel Fernando de Jesus,125
Brennan Johnson,122
Aleksandar Mitrović,107"""
        },
        {
            "question": "How did Erling Haaland perform in gameweek 20 of the 2022-23 season?",
            "ground_truth": """
Erling Haaland,points 2,goals 0,assists 0,minutes 90
Erling Haaland,points 6,goals 1, assists 0,minutes 89"""
        },
        {
            "question": "Compare Erling Haaland and Harry Kane's performance in the 2022-23 season",
            "ground_truth": """player,total_points,goals,assists
Erling Haaland,total_points 272,goals 36,assists 9
Harry Kane,total_points 263,goals 30,assists 9"""
        },
        {
            "question": "What were Manchester City's fixtures in the 2022-23 season?",
            "ground_truth": """
1,West Ham,Man City,2022-08-07 15:30:00+00:00
3,Newcastle,Man City,2022-08-21 15:30:00+00:00
6,Aston Villa,Man City,2022-09-03 16:30:00+00:00
8,Wolves,Man City,2022-09-17 11:30:00+00:00
11,Liverpool,Man City,2022-10-16 15:30:00+00:00
14,Leicester,Man City,2022-10-29 11:30:00+00:00
17,Leeds,Man City,2022-12-28 20:00:00+00:00
19,Chelsea,Man City,2023-01-05 20:00:00+00:00
20,Man Utd,Man City,2023-01-14 12:30:00+00:00
22,Spurs,Man City,2023-02-05 16:30:00+00:00
23,Arsenal,Man City,2023-02-15 19:30:00+00:00
24,Nott'm Forest,Man City,2023-02-18 15:00:00+00:00
25,Bournemouth,Man City,2023-02-25 17:30:00+00:00
27,Crystal Palace,Man City,2023-03-11 17:30:00+00:00
30,Southampton,Man City,2023-04-08 16:30:00+00:00
34,Fulham,Man City,2023-04-30 13:00:00+00:00
36,Everton,Man City,2023-05-14 13:00:00+00:00
37,Brighton,Man City,2023-05-24 19:00:00+00:00
38,Brentford,Man City,2023-05-28 15:30:00+00:00"""
        },
        {
            "question": "Which players are on Manchester City?",
            "ground_truth": """
player,position
Alex Robertson,MID
Aymeric Laporte,DEF
Aymeric Laporte,DEF
Ben Knight,MID
Benjamin Mendy,DEF
Bernardo Mota Veiga de Carvalho e Silva,MID
Bernardo Veiga de Carvalho e Silva,MID
Cieran Slicker,GK
Claudio Gomes,MID
Cole Palmer,MID
Cole Palmer,MID
Conrad Egan-Riley,DEF
Ederson Santana de Moraes,GK
Ederson Santana de Moraes,GK
Erling Haaland,FWD
Fernando Luiz Rosa,MID
Ferran Torres,MID
Gabriel Fernando de Jesus,FWD
Ilkay Gündogan,MID
Ilkay Gündogan,MID
Jack Grealish,MID
Jack Grealish,MID
James McAtee,MID
James McAtee,MID
John Stones,DEF
John Stones,FWD
John Stones,DEF
Josh Wilson-Esbrand,DEF
Joshua Wilson-Esbrand,DEF
João Cancelo,DEF
João Pedro Cavaco Cancelo,DEF
Julián Álvarez,FWD
Kalvin Phillips,MID
Kayky da Silva Chagas,FWD
Kevin De Bruyne,MID
Kevin De Bruyne,MID
Kyle Walker,DEF
Kyle Walker,DEF
Liam Delap,FWD
Liam Delap,FWD
Luke Mbete,DEF
Luke Mbete-Tabu,DEF
Manuel Akanji,DEF
Máximo Perrone,MID
Nathan Aké,DEF
Nathan Aké,DEF
Nico O'Reilly,MID
Oleksandr Zinchenko,DEF
Phil Foden,MID
Phil Foden,MID
Raheem Sterling,MID
Rico Lewis,DEF
Riyad Mahrez,MID
Riyad Mahrez,MID
Rodrigo Hernandez,MID
Rodrigo Hernandez,MID
Romeo Lavia,MID
Rúben Gato Alves Dias,DEF
Rúben Santos Gato Alves Dias,DEF
Samuel Edozie,MID
Scott Carson,GK
Scott Carson,GK
Sergio Gómez,DEF
Shea Charles,MID
Stefan Ortega Moreno,GK
Tommy Doyle,MID
Zack Steffen,GK
Zack Steffen,GK"""
        },
        {
            "question": "Who were the top 10 goal scorers in the 2022-23 season?",
            "ground_truth": """
Erling Haaland,36
Harry Kane,30
Ivan Toney,20
Mohamed Salah,19
Callum Wilson,18
Marcus Rashford,17
Ollie Watkins,15
Gabriel Martinelli Silva,15
Martin Ødegaard,15
Aleksandar Mitrović,14"""
        },
        {
            "question": "Who provided the most assists in the 2022-23 season? Show me the top 10?",
            "ground_truth": """
Kevin De Bruyne,18
Mohamed Salah,13
Morgan Gibbs-White,12
Riyad Mahrez,12
Bukayo Saka,12
Michael Olise,11
Trent Alexander-Arnold,11
Jack Grealish,10
Andreas Hoelgebaum Pereira,10
Solly March,10"""
        },
        {
            "question": "Which players had the most clean sheets in the 2022-23 season?",
            "ground_truth": """
Bruno Borges Fernandes,18
David De Gea Quintana,17
Kieran Trippier,16
Fabian Schär,15
Miguel Almirón Rejala,15
Benjamin White,15
Dan Burn,14
Alisson Ramses Becker,14
Gabriel Martinelli Silva,14
Aaron Ramsdale,14"""
        },
        {
            "question": "Who are the top 10 players in form as of gameweek 20 in the 2022-23 season?",
            "ground_truth": """
Martin Ødegaard,51
Solly March,46
Kieran Trippier,44
Harry Kane,43
Marcus Rashford,41
Bruno Borges Fernandes,40
Riyad Mahrez,39
Ivan Toney,39
Luke Shaw,38
Christian Eriksen,35"""
        },
       {
            "question": "Give me a complete summary of Erling Haaland's 2022-23 season?",
            "ground_truth": """player,total_points,goals,assists,clean_sheets,minutes,bonus_points
Erling Haaland,272,36,9,13,2767,40"""
        },
       {
            "question": "Who earned the most bonus points in the 2022-23 season?",
            "ground_truth": """player,total_bonus
Harry Kane,48
Erling Haaland,40
Kieran Trippier,39
Ivan Toney,35
Martin Ødegaard,30
Kevin De Bruyne,26
Ollie Watkins,25
Mohamed Salah,23
Bruno Borges Fernandes,23
Trent Alexander-Arnold,21"""
        },
      
        {
            "question": "Which players received the most yellow and red cards in the 2022-23 season?",
            "ground_truth": """player,total_yellow,total_red,total_cards
João Palhinha Gonçalves,total_yellow 14,total_red 0,total_cards 14
Rúben da Silva Neves,total_yellow 12,total_red 0,total_cards 12
Nélson Cabral Semedo,total_yellow 11,total_red 1,12
Joelinton Cássio Apolinário de Lira,total_yellow 12,total_red 0,total_cards 12
Fabio Henrique Tavares,total_yellow 11,total_red 0,total_cards 11
Adam Smith,total_yellow 11,total_red 0,total_cards 11
James Maddison,total_yellow 10,total_red 0,total_cards 10
Moisés Caicedo Corozo,total_yellow 10,total_red 0,total_cards 10
Conor Gallagher,total_yellow 9,total_red 1,total_cards 10
Cristian Romero,total_yellow 9,total_red 1,total_cards 10"""
        }
    ]
    
    # =========================================================================
    # MAIN EXECUTION
    # =========================================================================
    
    models = list(models_dict.values())
    model_labels = list(models_dict.keys())
    
    if len(models) != 3:
        raise ValueError("Please provide exactly 3 models for comparison")
    
    print("\n" + "="*80)
    print("          RUNNING MODELS ON TEST QUERIES")
    print("="*80 + "\n")
    
    # Step 1: Run all models and collect results
    all_model_results = {}
    
    for model_id, model_name in models_dict.items():
        display_name = f"Model {model_id} ({model_name.split('/')[1].split(':')[0]})"
        print(f"🔄 Running {display_name}...")
        model_results = []
        
        for query in test_queries:
            question = query["question"]
            ground_truth = query.get("ground_truth")
            
            try:
                answer, tokens_used = call_llm(question, model_name)
                
                model_results.append({
                    "question": question,
                    "ground_truth": ground_truth,
                    "answer": answer,
                    "tokens_used": tokens_used
                })
            except Exception as e:
                model_results.append({
                    "question": question,
                    "ground_truth": ground_truth,
                    "answer": f"ERROR: {str(e)}",
                    "tokens_used": 0
                })
        
        all_model_results[display_name] = model_results
        print(f"✓ {display_name} completed\n")
    
    model_display_names = list(all_model_results.keys())
    
    # Step 2: Human evaluation for each query across all models
    print("\n" + "="*80)
    print("          HUMAN EVALUATION SESSION")
    print("="*80 + "\n")
    
    evaluations = {model: [] for model in model_display_names}
    total_queries = len(test_queries)
    
    for query_idx in range(total_queries):
        os.system('clear' if os.name == 'posix' else 'cls')
        
        print(f"\n📊 Progress: Query {query_idx + 1}/{total_queries} ({(query_idx + 1)/total_queries*100:.0f}%)")
        print("="*80)
        
        # Display question and ground truth
        query_data = test_queries[query_idx]
        print(f"\n❓ Question: {query_data['question']}")
        if query_data.get('ground_truth'):
            print(f"✓ Ground Truth: {query_data['ground_truth']}")
        
        print("\n" + "-"*80)
        
        # Evaluate each model's answer for this query
        for model_name in model_display_names:
            result = all_model_results[model_name][query_idx]
            
            print(f"\n🤖 {model_name}")
            print(f"💬 Answer: {result['answer']}")
            print(f"🔢 Tokens Used: {result['tokens_used']}")
            print("-"*80)
            
            # Collect ratings
            print(f"\nRate this answer (1-5):")
            ratings = {}
            metrics = {
                'relevance': '🎯 Relevance',
                'correctness': '✅ Correctness',
                'naturalness': '🗣️  Naturalness',
                'completeness': '📝 Completeness',
                'overall': '⭐ Overall Quality'
            }
            
            for key, label in metrics.items():
                while True:
                    try:
                        rating = int(input(f"  {label} (1-5): "))
                        if 1 <= rating <= 5:
                            ratings[key] = rating
                            break
                        else:
                            print("    ⚠️  Please enter 1-5")
                    except ValueError:
                        print("    ⚠️  Please enter a valid number")
            
            comments = input(f"\n💭 Comments (optional): ")
            
            # Store evaluation
            evaluations[model_name].append({
                **result,
                "human_eval": {
                    **ratings,
                    "comments": comments
                }
            })
            
            print()
        
        # Save progress
        progress_data = {
            "models": model_display_names,
            "evaluations": evaluations,
            "queries_completed": query_idx + 1
        }
        with open("model_comparison_progress.json", "w") as f:
            json.dump(progress_data, f, indent=2)
        
        print(f"✓ Progress saved ({query_idx + 1}/{total_queries})")
        
        if query_idx < total_queries - 1:
            input("\nPress Enter to continue to next query...")
    
    # Step 3: Calculate statistics and create comparison table
    os.system('clear' if os.name == 'posix' else 'cls')
    
    print("\n" + "="*80)
    print("          EVALUATION COMPLETE - COMPARISON RESULTS")
    print("="*80 + "\n")
    
    metrics_list = ['relevance', 'correctness', 'naturalness', 'completeness', 'overall']
    comparison_data = {}
    
    for model_name in model_display_names:
        model_evals = evaluations[model_name]
        
        avg_metrics = {}
        for metric in metrics_list:
            avg = sum(e["human_eval"][metric] for e in model_evals) / len(model_evals)
            avg_metrics[metric] = round(avg, 2)
        
        avg_tokens = sum(e["tokens_used"] for e in model_evals) / len(model_evals)
        
        comparison_data[model_name] = {
            "averages": avg_metrics,
            "avg_tokens": round(avg_tokens, 1),
            "evaluations": model_evals
        }
    
    # Print comparison table
    print_comparison_table(model_display_names, comparison_data, metrics_list)
    
    # Save final results
    final_results = {
        "models": model_display_names,
        "total_queries": total_queries,
        "comparison": comparison_data,
        "config": {
            "models_tested": models_dict,
            "queries": test_queries
        }
    }
    
    with open("model_comparison_final.json", "w") as f:
        json.dump(final_results, f, indent=2)
    
    print(f"\n💾 Full results saved to: model_comparison_final.json")
    
    return final_results


def print_comparison_table(models, comparison_data, metrics):
    """Print a formatted comparison table"""
    
    # Header
    print("📊 COMPARISON TABLE")
    print("="*100)
    
    col_width = 28
    print(f"{'Metric':<{col_width}}", end="")
    for model in models:
        # Truncate long model names for display
        display_name = model if len(model) <= 25 else model[:22] + "..."
        print(f"{display_name:<{col_width}}", end="")
    print()
    print("-"*100)
    
    # Metrics rows
    metric_labels = {
        'relevance': '🎯 Relevance',
        'correctness': '✅ Correctness',
        'naturalness': '🗣️  Naturalness',
        'completeness': '📝 Completeness',
        'overall': '⭐ Overall'
    }
    
    for metric in metrics:
        label = metric_labels.get(metric, metric.capitalize())
        print(f"{label:<{col_width}}", end="")
        
        scores = [comparison_data[model]["averages"][metric] for model in models]
        max_score = max(scores) if scores else 0
        
        for model in models:
            score = comparison_data[model]["averages"][metric]
            is_winner = (score == max_score and score > 0)
            score_str = f"{score:.2f}/5.00"
            if is_winner:
                print(f"{score_str} 🏆{'':<{col_width-len(score_str)-3}}", end="")
            else:
                print(f"{score_str}{'':<{col_width-len(score_str)}}", end="")
        print()
    
    print("-"*100)
    
    # Token usage
    print(f"{'🔢 Avg Tokens Used':<{col_width}}", end="")
    for model in models:
        tokens = comparison_data[model]["avg_tokens"]
        tokens_str = f"{tokens:.0f}"
        print(f"{tokens_str}{'':<{col_width-len(tokens_str)}}", end="")
    print()
    
    print("="*100)
    
    # Determine winner
    overall_scores = [(model, comparison_data[model]["averages"]["overall"]) for model in models]
    winner = max(overall_scores, key=lambda x: x[1])
    
    print(f"\n🏆 WINNER: {winner[0]}")
    print(f"   Overall Score: {winner[1]:.2f}/5.00\n")


    # =========================================================================
    # USAGE EXAMPLE
    # =========================================================================    
    # Results are automatically saved to:
    # - model_comparison_progress.json (during evaluation)
    # - model_comparison_final.json (after completion)
    
    print("\n✅ Evaluation complete!")
    print(f"📊 {len(results['comparison'])} models evaluated")
    print(f"❓ {results['total_queries']} queries tested")




def extract_entities_and_numbers(text: str) -> dict:
    """
    Extract player names, team names, and numbers from text.
    Returns dictionary with entities and numbers.
    """
    import re
    
    text_lower = text.lower()
    
    # Extract numbers (integers and decimals)
    numbers = set(re.findall(r'\b\d+\.?\d*\b', text))
    
    # Extract entities more aggressively
    entities = set()
    
    # Method 1: Extract from lines (handles lists)
    lines = text.split('\n')
    for line in lines:
        # Remove common prefixes (numbers, bullets, etc)
        clean_line = re.sub(r'^\s*[\d\.\-\*]+\.?\s*', '', line)
        clean_line = re.sub(r'\([^)]*\)', '', clean_line)  # Remove parentheses content
        
        # Extract capitalized word sequences
        words = clean_line.split()
        current_name = []
        
        for word in words:
            clean_word = re.sub(r'[^\w\s\-\']', '', word)
            
            # Stop at position markers or common words
            if clean_word.lower() in ['gk', 'def', 'mid', 'fwd', 'goalkeeper', 'defender', 'midfielder', 'forward']:
                if current_name:
                    entities.add(' '.join(current_name).lower())
                    current_name = []
                continue
            
            # Check if looks like a name part
            if clean_word and len(clean_word) > 1:
                # Add if capitalized or part of ongoing name
                if clean_word[0].isupper() or current_name:
                    current_name.append(clean_word.lower())
                elif current_name:
                    # End of name
                    entities.add(' '.join(current_name).lower())
                    current_name = []
        
        if current_name:
            entities.add(' '.join(current_name).lower())
    
    # Method 2: Also extract last names (single capitalized words > 3 chars)
    all_words = re.findall(r'\b[A-Z][a-z]{3,}\b', text)
    for word in all_words:
        entities.add(word.lower())
    
    # Method 3: Extract hyphenated names
    hyphenated = re.findall(r'\b[A-Z][a-z]+(?:-[A-Z][a-z]+)+\b', text)
    for name in hyphenated:
        entities.add(name.lower())
    
    return {
        'entities': entities,
        'numbers': numbers
    }


def calculate_accuracy(answer: str, ground_truth: str) -> dict:
    """
    Calculate accuracy by comparing entities and numbers.
    More lenient matching - focuses on whether entities are present.
    Returns dict with accuracy score and details.
    """
    import re
    
    # Check if model refused to answer or has no information
    refusal_patterns = [
        'does not contain', 'not contain', 'no information', 'not available',
        'cannot find', 'unable to find', 'unfortunately', 'i don\'t have',
        'not provided', 'insufficient information', 'no data'
    ]
    
    answer_lower = answer.lower()
    if any(pattern in answer_lower for pattern in refusal_patterns):
        # Model refused to answer - automatic fail
        return {
            'accuracy': 0.0,
            'is_correct': False,
            'entity_precision': 0.0,
            'entity_recall': 0.0,
            'entity_f1': 0.0,
            'number_accuracy': 0.0,
            'entity_matches': "0/0",
            'number_matches': "0/0",
            'details': 'Model refused to answer or stated no information available'
        }
    
    # Parse ground truth (CSV format)
    truth_lines = [line.strip() for line in ground_truth.strip().split('\n') if line.strip()]
    
    if len(truth_lines) <= 1:
        return {
            'accuracy': 0.0,
            'entity_precision': 0.0,
            'entity_recall': 0.0,
            'number_accuracy': 0.0,
            'details': 'No ground truth data'
        }
    
    # Extract expected entities (skip header)
    expected_entities = set()
    expected_numbers = set()
    
    for line in truth_lines[1:]:
        parts = [p.strip() for p in line.split(',')]
        if parts:
            # First column is usually the entity name
            entity_name = parts[0].lower()
            expected_entities.add(entity_name)
            
            # Extract last name (most important for matching)
            name_parts = entity_name.split()
            if name_parts:
                # Add last name
                expected_entities.add(name_parts[-1])
                # Add first name if exists
                if len(name_parts) > 1:
                    expected_entities.add(name_parts[0])
                # Add middle parts
                for part in name_parts:
                    if len(part) > 3:
                        expected_entities.add(part)
            
            # Extract numbers
            for part in parts[1:]:
                if re.match(r'^\d+\.?\d*$', part):
                    expected_numbers.add(part)
    
    # Remove duplicates from expected entities (since ground truth has dupes)
    unique_expected = set()
    for entity in expected_entities:
        # Only keep substantial entities
        if len(entity) > 2:
            unique_expected.add(entity)
    
    # Extract entities from answer
    answer_data = extract_entities_and_numbers(answer)
    found_entities = answer_data['entities']
    found_numbers = answer_data['numbers']
    
    # Calculate entity matching with flexible approach
    matched_entities = set()
    for expected in unique_expected:
        for found in found_entities:
            # Exact match
            if expected == found:
                matched_entities.add(expected)
                break
            # Substring match (for partial names)
            elif expected in found or found in expected:
                matched_entities.add(expected)
                break
            # Check if all words of expected are in found
            expected_words = set(expected.split())
            found_words = set(found.split())
            if expected_words and expected_words.issubset(found_words):
                matched_entities.add(expected)
                break
    
    entity_matches = len(matched_entities)
    
    # For lists without numbers, don't penalize
    has_numbers = len(expected_numbers) > 0
    
    if has_numbers:
        # Calculate entity precision/recall
        entity_precision = entity_matches / len(found_entities) if found_entities else 0
        entity_recall = entity_matches / len(unique_expected) if unique_expected else 0
        entity_f1 = 2 * (entity_precision * entity_recall) / (entity_precision + entity_recall) if (entity_precision + entity_recall) > 0 else 0
        
        # Calculate number matching
        number_matches = len(expected_numbers & found_numbers)
        number_accuracy = number_matches / len(expected_numbers) if expected_numbers else 0
        
        # Combined accuracy (70% entities, 30% numbers)
        final_accuracy = (entity_f1 * 0.7) + (number_accuracy * 0.3)
    else:
        # For pure list queries (like "players on team"), use recall only
        # If we found at least 50% of expected entities, it's good
        entity_recall = entity_matches / len(unique_expected) if unique_expected else 0
        entity_precision = 1.0  # Don't penalize for finding extra entities
        entity_f1 = entity_recall  # Just use recall
        number_accuracy = 1.0  # No numbers expected
        
        # For list queries, lower threshold
        final_accuracy = entity_recall
    
    # Determine if correct (lower threshold for list queries)
    threshold = 0.3 if not has_numbers else 0.5
    is_correct = final_accuracy > threshold
    
    return {
        'accuracy': final_accuracy,
        'is_correct': is_correct,
        'entity_precision': entity_precision,
        'entity_recall': entity_recall,
        'entity_f1': entity_f1,
        'number_accuracy': number_accuracy,
        'entity_matches': f"{entity_matches}/{len(unique_expected)}",
        'number_matches': f"{len(expected_numbers & found_numbers)}/{len(expected_numbers)}" if expected_numbers else "N/A",
        'details': f"Entities: {entity_matches}/{len(unique_expected)}" + (f", Numbers: {len(expected_numbers & found_numbers)}/{len(expected_numbers)}" if expected_numbers else " (list query)")
    }


def evaluate_all_models():
    """
    Evaluate multiple models on predefined test queries.
    
    Measures:
    - Accuracy (NER-based entity and number matching)
    - Response time (seconds)
    - Tokens used
    
    Returns:
        Dictionary with individual results and comparison table
    """
    import time
    import pandas as pd
    
    # =========================================================================
    # MODEL CONFIGURATION
    # =========================================================================
    
    models = {
        "Llama 3.3 70B": "meta-llama/llama-3.3-70b-instruct:free",
        "Devstral 2512": "mistralai/devstral-2512:free",
        "Nemotron Nano 30B": "nvidia/nemotron-3-nano-30b-a3b:free",
    }
    
    # =========================================================================
    # TEST QUERIES CONFIGURATION
    # =========================================================================
    
    queries_with_truths = queries_with_truths = [
    {
        "question": "Who are the top 10 forwards in the 2022-23 season?",
        "ground_truth": """player,season_points
Erling Haaland,272
Harry Kane,263
Ivan Toney,182
Ollie Watkins,175
Callum Wilson,157
Bryan Mbeumo,150
Dominic Solanke,130
Gabriel Fernando de Jesus,125
Brennan Johnson,122
Aleksandar Mitrović,107"""
    },
    {
        "question": "How did Erling Haaland perform in gameweek 20 of the 2022-23 season?",
        "ground_truth": """player,points,goals,assists,minutes
Erling Haaland,2,0,0,90
Erling Haaland,6,1,0,89"""
    },
    {
        "question": "Compare Erling Haaland and Harry Kane's performance in the 2022-23 season",
        "ground_truth": """player,total_points,goals,assists
Erling Haaland,272,36,9
Harry Kane,263,30,9"""
    },
    {
        "question": "What were Manchester City's fixtures in the 2022-23 season?",
        "ground_truth": """gameweek,home_team,away_team,kickoff_time
1,West Ham,Man City,2022-08-07 15:30:00+00:00
3,Newcastle,Man City,2022-08-21 15:30:00+00:00
6,Aston Villa,Man City,2022-09-03 16:30:00+00:00
8,Wolves,Man City,2022-09-17 11:30:00+00:00
11,Liverpool,Man City,2022-10-16 15:30:00+00:00
14,Leicester,Man City,2022-10-29 11:30:00+00:00
17,Leeds,Man City,2022-12-28 20:00:00+00:00
19,Chelsea,Man City,2023-01-05 20:00:00+00:00
20,Man Utd,Man City,2023-01-14 12:30:00+00:00
22,Spurs,Man City,2023-02-05 16:30:00+00:00
23,Arsenal,Man City,2023-02-15 19:30:00+00:00
24,Nott'm Forest,Man City,2023-02-18 15:00:00+00:00
25,Bournemouth,Man City,2023-02-25 17:30:00+00:00
27,Crystal Palace,Man City,2023-03-11 17:30:00+00:00
30,Southampton,Man City,2023-04-08 16:30:00+00:00
34,Fulham,Man City,2023-04-30 13:00:00+00:00
36,Everton,Man City,2023-05-14 13:00:00+00:00
37,Brighton,Man City,2023-05-24 19:00:00+00:00
38,Brentford,Man City,2023-05-28 15:30:00+00:00"""
    },
    {
        "question": "Which players are on Manchester City?",
        "ground_truth": """player,position
Alex Robertson,MID
Aymeric Laporte,DEF
Aymeric Laporte,DEF
Ben Knight,MID
Benjamin Mendy,DEF
Bernardo Mota Veiga de Carvalho e Silva,MID
Bernardo Veiga de Carvalho e Silva,MID
Cieran Slicker,GK
Claudio Gomes,MID
Cole Palmer,MID
Cole Palmer,MID
Conrad Egan-Riley,DEF
Ederson Santana de Moraes,GK
Ederson Santana de Moraes,GK
Erling Haaland,FWD
Fernando Luiz Rosa,MID
Ferran Torres,MID
Gabriel Fernando de Jesus,FWD
Ilkay Gündogan,MID
Ilkay Gündogan,MID
Jack Grealish,MID
Jack Grealish,MID
James McAtee,MID
James McAtee,MID
John Stones,DEF
John Stones,FWD
John Stones,DEF
Josh Wilson-Esbrand,DEF
Joshua Wilson-Esbrand,DEF
João Cancelo,DEF
João Pedro Cavaco Cancelo,DEF
Julián Álvarez,FWD
Kalvin Phillips,MID
Kayky da Silva Chagas,FWD
Kevin De Bruyne,MID
Kevin De Bruyne,MID
Kyle Walker,DEF
Kyle Walker,DEF
Liam Delap,FWD
Liam Delap,FWD
Luke Mbete,DEF
Luke Mbete-Tabu,DEF
Manuel Akanji,DEF
Máximo Perrone,MID
Nathan Aké,DEF
Nathan Aké,DEF
Nico O'Reilly,MID
Oleksandr Zinchenko,DEF
Phil Foden,MID
Phil Foden,MID
Raheem Sterling,MID
Rico Lewis,DEF
Riyad Mahrez,MID
Riyad Mahrez,MID
Rodrigo Hernandez,MID
Rodrigo Hernandez,MID
Romeo Lavia,MID
Rúben Gato Alves Dias,DEF
Rúben Santos Gato Alves Dias,DEF
Samuel Edozie,MID
Scott Carson,GK
Scott Carson,GK
Sergio Gómez,DEF
Shea Charles,MID
Stefan Ortega Moreno,GK
Tommy Doyle,MID
Zack Steffen,GK
Zack Steffen,GK"""
    },
    {
        "question": "Who were the top 10 goal scorers in the 2022-23 season?",
        "ground_truth": """player,total_goals
Erling Haaland,36
Harry Kane,30
Ivan Toney,20
Mohamed Salah,19
Callum Wilson,18
Marcus Rashford,17
Ollie Watkins,15
Gabriel Martinelli Silva,15
Martin Ødegaard,15
Aleksandar Mitrović,14"""
    },
    {
        "question": "Who provided the most assists in the 2022-23 season? Show me the top 10",
        "ground_truth": """player,total_assists
Kevin De Bruyne,18
Mohamed Salah,13
Morgan Gibbs-White,12
Riyad Mahrez,12
Bukayo Saka,12
Michael Olise,11
Trent Alexander-Arnold,11
Jack Grealish,10
Andreas Hoelgebaum Pereira,10
Solly March,10"""
    },
    {
        "question": "Which players had the most clean sheets in the 2022-23 season?",
        "ground_truth": """player,total_clean_sheets
Bruno Borges Fernandes,18
David De Gea Quintana,17
Kieran Trippier,16
Fabian Schär,15
Miguel Almirón Rejala,15
Benjamin White,15
Dan Burn,14
Alisson Ramses Becker,14
Gabriel Martinelli Silva,14
Aaron Ramsdale,14"""
    },
    {
        "question": "Who are the top 10 players in form as of gameweek 20 in the 2022-23 season?",
        "ground_truth": """player,form_points
Martin Ødegaard,51
Solly March,46
Kieran Trippier,44
Harry Kane,43
Marcus Rashford,41
Bruno Borges Fernandes,40
Riyad Mahrez,39
Ivan Toney,39
Luke Shaw,38
Christian Eriksen,35"""
    },
    {
        "question": "Give me a complete summary of Erling Haaland's 2022-23 season",
        "ground_truth": """player,total_points,goals,assists,clean_sheets,minutes,bonus_points
Erling Haaland,272,36,9,13,2767,40"""
    },
    {
        "question": "Who earned the most bonus points in the 2022-23 season?",
        "ground_truth": """player,total_bonus
Harry Kane,48
Erling Haaland,40
Kieran Trippier,39
Ivan Toney,35
Martin Ødegaard,30
Kevin De Bruyne,26
Ollie Watkins,25
Mohamed Salah,23
Bruno Borges Fernandes,23
Trent Alexander-Arnold,21"""
    },
    {
        "question": "Which players received the most yellow and red cards in the 2022-23 season?",
        "ground_truth": """player,total_yellow,total_red,total_cards
João Palhinha Gonçalves,14,0,14
Rúben da Silva Neves,12,0,12
Nélson Cabral Semedo,11,1,12
Joelinton Cássio Apolinário de Lira,12,0,12
Fabio Henrique Tavares,11,0,11
Adam Smith,11,0,11
James Maddison,10,0,10
Moisés Caicedo Corozo,10,0,10
Conor Gallagher,9,1,10
Cristian Romero,9,1,10"""
    }
]
    
    # =========================================================================
    # EVALUATE ALL MODELS
    # =========================================================================
    
    all_model_results = {}
    
    for model_name, model_id in models.items():
        print(f"\n{'='*80}")
        print(f"EVALUATING MODEL: {model_name}")
        print(f"{'='*80}\n")
        
        results = []
        total_response_time = 0
        total_tokens = 0
        total_accuracy = 0
        correct_count = 0
        evaluated_count = 0
        
        for idx, item in enumerate(queries_with_truths, 1):
            question = item["question"]
            ground_truth = item.get("ground_truth")
            
            print(f"{'─'*80}")
            print(f"Query {idx}/{len(queries_with_truths)}")
            print(f"{'─'*80}")
            print(f"❓ Question: {question}")
            
            start = time.time()
            try:
                answer, tokens_used = call_llm(question, model_id)
                answer_preview = str(answer)[:300] + "..." if len(str(answer)) > 300 else str(answer)
                print(f"🤖 Model Answer: {answer_preview}")
                
                # Print ground truth for comparison
                if ground_truth:
                    truth_preview = ground_truth[:200] + "..." if len(ground_truth) > 200 else ground_truth
                    print(f"📋 Ground Truth: {truth_preview}")
                    
            except Exception as e:
                print(f"❌ Error: {str(e)}")
                results.append({
                    "question": question,
                    "ground_truth": ground_truth,
                    "answer": None,
                    "error": str(e),
                    "response_time": None,
                    "tokens_used": 0,
                    "accuracy": None
                })
                print()
                continue
            
            end = time.time()
            response_time = end - start
            total_tokens += tokens_used
            total_response_time += response_time
            
            # Calculate accuracy
            accuracy_result = calculate_accuracy(str(answer), ground_truth)
            accuracy_score = accuracy_result['accuracy']
            is_correct = accuracy_result['is_correct']
            
            total_accuracy += accuracy_score
            if is_correct:
                correct_count += 1
            evaluated_count += 1
            
            status = "✅ CORRECT" if is_correct else "❌ INCORRECT"
            print(f"\n{status}")
            print(f"📊 Accuracy: {accuracy_score:.2%}")
            print(f"   Entity F1: {accuracy_result['entity_f1']:.2%}")
            print(f"   Number Match: {accuracy_result['number_accuracy']:.2%}")
            print(f"   Details: {accuracy_result['details']}")
            print(f"⏱️  Response Time: {response_time:.3f}s")
            print(f"🔢 Tokens Used: {tokens_used}")
            print()
            
            results.append({
                "question": question,
                "ground_truth": ground_truth,
                "answer": str(answer),
                "response_time": response_time,
                "tokens_used": tokens_used,
                "accuracy": accuracy_score,
                "is_correct": is_correct,
                "accuracy_details": accuracy_result
            })
        
        num_queries = len(queries_with_truths)
        avg_response_time = total_response_time / num_queries if num_queries > 0 else 0
        avg_tokens = total_tokens / num_queries if num_queries > 0 else 0
        avg_accuracy = (total_accuracy / evaluated_count) * 100 if evaluated_count > 0 else 0
        correct_percentage = (correct_count / evaluated_count) * 100 if evaluated_count > 0 else 0
        
        all_model_results[model_name] = {
            "model_id": model_id,
            "results": results,
            "average_response_time": avg_response_time,
            "average_tokens_used": avg_tokens,
            "average_accuracy": avg_accuracy,
            "correct_percentage": correct_percentage,
            "correct_count": correct_count,
            "total_queries": evaluated_count
        }
        
        print(f"{'='*80}")
        print(f"SUMMARY FOR {model_name}")
        print(f"{'='*80}")
        print(f"Total Queries: {num_queries}")
        print(f"Correct Answers: {correct_count}/{evaluated_count} ({correct_percentage:.1f}%)")
        print(f"Average Accuracy Score: {avg_accuracy:.1f}%")
        print(f"Average Response Time: {avg_response_time:.3f}s")
        print(f"Average Tokens Used: {avg_tokens:.1f}")
        print(f"{'='*80}\n")
    
    # =========================================================================
    # CREATE COMPARISON TABLE
    # =========================================================================
    
    print(f"\n{'='*80}")
    print(f"MODEL COMPARISON TABLE")
    print(f"{'='*80}\n")
    
    comparison_data = []
    for model_name, metrics in all_model_results.items():
        comparison_data.append({
            "Model": model_name,
            "Correct (%)": f"{metrics['correct_percentage']:.1f}",
            "Avg Accuracy": f"{metrics['average_accuracy']:.1f}%",
            "Avg Time (s)": f"{metrics['average_response_time']:.3f}",
            "Avg Tokens": f"{metrics['average_tokens_used']:.0f}"
        })
    
    df = pd.DataFrame(comparison_data)
    print(df.to_string(index=False))
    print(f"\n{'='*80}\n")
    
    # Find best model for each metric
    print("🏆 BEST PERFORMERS:")
    print(f"{'─'*80}")
    
    best_accuracy = max(all_model_results.items(), key=lambda x: x[1]['average_accuracy'])
    print(f"Best Accuracy: {best_accuracy[0]} ({best_accuracy[1]['average_accuracy']:.1f}%)")
    
    best_correct = max(all_model_results.items(), key=lambda x: x[1]['correct_percentage'])
    print(f"Most Correct: {best_correct[0]} ({best_correct[1]['correct_count']}/{best_correct[1]['total_queries']})")
    
    best_speed = min(all_model_results.items(), key=lambda x: x[1]['average_response_time'])
    print(f"Fastest: {best_speed[0]} ({best_speed[1]['average_response_time']:.3f}s)")
    
    best_efficiency = min(all_model_results.items(), key=lambda x: x[1]['average_tokens_used'])
    print(f"Most Efficient: {best_efficiency[0]} ({best_efficiency[1]['average_tokens_used']:.0f} tokens)")
    
    print(f"{'='*80}\n")
    
    return {
        "individual_results": all_model_results,
        "comparison_table": df,
        "summary": {
            "total_models_evaluated": len(models),
            
        }
    }







🚀 Initializing FPL Input Preprocessor (Enhanced)...
✅ Connected to LLM: google/gemma-2-2b-it
✅ Loaded embedding model: sentence-transformers/all-MiniLM-L6-v2
✅ System initialized



In [22]:

def main():
    user_question = input("Hey manager, what do you need to know?: ")
    
    models = {
    "1": "mistralai/devstral-2512:free",
    "2":  "meta-llama/llama-3.3-70b-instruct:free",
    "3": "nvidia/nemotron-3-nano-30b-a3b:free",
    }
    
    embedding_types = {
        "1": "numeric",
        "2": "minilm", 
        "3": "mpnet"
    }
    
    # Ask for model choice
    model_choice = input("Choose a model (1-3): ").strip()
    
    # Ask for embedding type choice
    print("\nEmbedding types:")
    print("1. Numeric (fast, stat-based)")
    print("2. MiniLM (semantic, good balance)")
    print("3. MPNet (most accurate semantic)")
    embedding_choice = input("Choose embedding type (1-3): ").strip()
    
    if model_choice in models and embedding_choice in embedding_types:
        model = models[model_choice]
        embedding_type = embedding_types[embedding_choice]
        
        print(f"You selected Model {model_choice}: {model}")
        print(f"Using {embedding_type} embeddings")
        
        answer = call_llm(user_question, model, embedding_type)
        print("\nLLM Answer:\n", answer[0])
    else:
        print("Invalid choice.")

if __name__ == "__main__":
    main()

Hey manager, what do you need to know?:  Salah performance gameweek 20 season 22-23
Choose a model (1-3):  1



Embedding types:
1. Numeric (fast, stat-based)
2. MiniLM (semantic, good balance)
3. MPNet (most accurate semantic)


Choose embedding type (1-3):  1


You selected Model 1: mistralai/devstral-2512:free
Using numeric embeddings
{'user_input': 'Salah performance gameweek 20 season 22-23', 'baseline': {'method': 'baseline', 'intent': 'performance_query', 'query_used': 'player_gameweek_performance', 'parameters': {'intent': 'performance_query', 'intent_confidence': 0.99, 'gameweek': 20, 'player_name': 'Mohamed Salah', 'threshold': 22.0, 'thresholds': [22.0, 23.0], 'limit': 10, 'season': '2022-23'}, 'results': [], 'cypher': '\n                // Player performance in specific gameweek\n                MATCH (gw:Gameweek {season: $season, GW_number: $gameweek})<-[:HAS_FIXTURE]-(f:Fixture)\n                MATCH (p:Player {player_name: $player_name})-[played:PLAYED_IN]->(f)\n                MATCH (p)-[:PLAYS_FOR]->(t:Team)\n                RETURN p.player_name AS player,\n                    t.name AS team,\n                    played.total_points AS points,\n                    played.goals_scored AS goals,\n                    played.assi

In [ ]:
# compare_models_human_evaluation()


In [ ]:
 # evaluate_all_models()

In [20]:
# ============================================
# CELL 1: INSTALL DEPENDENCIES
# ============================================

!pip install streamlit pyngrok -q
!ngrok authtoken "36ulLRiXuNcPFvvcjBqzMY4fzzD_2YrjyBDEVMznrTiSTi8j"

# ============================================
# CELL 2: COPY Task3.py FUNCTIONS TO WORKING DIR
# ============================================

# Since Task3.py code is in another cell in your Kaggle notebook,
# we need to create a standalone Task3.py file for Streamlit to import

task3_code = """
from kaggle_secrets import UserSecretsClient
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import Dict, List, Any
from fpl_Task2 import FPLGraphRetrieval
from openai import OpenAI

# Get secrets
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("openrouterkey")
secret_value_1 = user_secrets.get_secret("preprocesskey")

# Neo4j Configuration
NEO4J_URI = "neo4j+s://1da86c19.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "HA4iunTOGen7RYpeISs3ZRhcWjpcokqam9przCqCuQ8"

# Initialize retriever
retriever = FPLGraphRetrieval(
    neo4j_uri=NEO4J_URI,
    neo4j_user=NEO4J_USER,
    neo4j_password=NEO4J_PASSWORD,
    hf_token=secret_value_1,
    use_llm=True
)

def call_llm(user_question: str, model: str, embedding_type: str = 'numeric'):
    openrouter_key = secret_value_0
    
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=openrouter_key
    )
    
    result = retriever.retrieve(user_question, method="both", embedding_type=embedding_type)
    context_text = result
    
    persona_text = (
        "You are an FPL (Fantasy Premier League) expert. "
        "You can answer any questions related to FPL, including player stats, "
        "team performance, transfers, and gameweek strategies. "
        "You are also knowledgeable about the Premier League in general."
    )
    
    task_text = (
        "IMPORTANT: You MUST always provide a response. Never return empty text.\\n\\n"
        "SPECIAL INSTRUCTIONS FOR COMPARISON QUERIES:\\n"
        "When the user asks to compare players (e.g., 'compare X and Y'):\\n"
        "1. First check if BOTH players are mentioned in the CONTEXT\\n"
        "2. If yes, extract ALL their stats from the CONTEXT\\n"
        "3. Create a detailed comparison table or bullet points\\n"
        "4. Include: goals, points, assists, clean sheets (if relevant), bonus points\\n"
        "5. Provide a clear conclusion about who performed better overall\\n\\n"
        "CONTEXT:\\n{context}\\n\\n"
        "QUESTION: {question}\\n\\n"
        "ANSWER (remember: never leave this blank):"
    )
    
    prompt = f\"\"\"
    CONTEXT:
    {context_text}

    PERSONA:
    {persona_text}

    TASK:
    {task_text}
    \"\"\"
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": persona_text},
            {"role": "user", "content": f"{prompt}\\n\\nQuestion: {user_question}"}
        ],
        temperature=0.2,
        max_tokens=500
    )
    
    answer = completion.choices[0].message.content
    total_tokens = completion.usage.total_tokens
    
    return answer, total_tokens
"""

# Write Task3.py
with open('Task3.py', 'w') as f:
    f.write(task3_code)

print("✅ Task3.py created successfully!")

# ============================================
# CELL 3: CREATE STREAMLIT APP FILE
# ============================================

app_code = """
import streamlit as st
import pandas as pd
import sys
import os

# Add current directory to Python path
sys.path.insert(0, os.getcwd())

# Import from Task3
try:
    from Task3 import call_llm, retriever
except ImportError as e:
    st.error(f"❌ Could not import Task3: {e}")
    st.stop()

# =========================
# PAGE CONFIG
# =========================
st.set_page_config(
    page_title="FPL Graph-RAG Assistant",
    page_icon="⚽",
    layout="wide"
)

# Custom CSS for FPL theme
st.markdown('''
    <style>
    .manager-header {
        background: linear-gradient(90deg, #1e3c72 0%, #2a5298 100%);
        color: white;
        padding: 25px;
        border-radius: 15px;
        margin-bottom: 25px;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    }
    .fpl-stats {
        background-color: #f8f9fa;
        border-left: 5px solid #28a745;
        padding: 20px;
        border-radius: 8px;
        margin: 15px 0;
        box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }
    .stButton button {
        background: linear-gradient(90deg, #FF6B6B 0%, #FF8E53 100%);
        color: white;
        font-weight: bold;
        border: none;
    }
    .stButton button:hover {
        background: linear-gradient(90deg, #FF8E53 0%, #FF6B6B 100%);
        color: white;
    }
    .sidebar-header {
        background: linear-gradient(90deg, #2c3e50 0%, #4ca1af 100%);
        color: white;
        padding: 15px;
        border-radius: 10px;
        margin-bottom: 20px;
    }
    </style>
''', unsafe_allow_html=True)

# =========================
# SIDEBAR CONTROLS
# =========================
with st.sidebar:
    st.markdown('<div class="sidebar-header">', unsafe_allow_html=True)
    st.title("⚙ FPL Manager Dashboard")
    st.caption("Configure your AI Assistant")
    st.markdown('</div>', unsafe_allow_html=True)
    
    st.markdown("### 🧠 1. Generation Model")
    st.caption("Select the LLM that generates the final answer.")
    llm_map = {
        "Devstral": "mistralai/devstral-2512:free",
        "Llama 3 70B": "meta-llama/llama-3.3-70b-instruct:free",
        "Nemotron 3": "nvidia/nemotron-3-nano-30b-a3b:free"
    }


    llm_choice = st.selectbox("LLM Model", list(llm_map.keys()), index=0)
    selected_llm_id = llm_map[llm_choice]

    st.markdown("---")

    st.markdown("### 📐 2. Embedding Model")
    st.caption("How the system finds similar players/stats.")
    embedding_map = {
        "Numeric (Stats-based)": "numeric",
        "MiniLM (Fast, 384d)": "minilm",
        "MPNet (Accurate, 768d)": "mpnet"
    }
    emb_choice = st.selectbox("Embedding Model", list(embedding_map.keys()), index=0)
    selected_emb_type = embedding_map[emb_choice]

    st.markdown("---")
    
    # FPL Tips Section
    with st.expander("💡 FPL Assistant Tips"):
        st.markdown('''
        **Best Query Examples:**
        - "Compare Haaland vs Salah"
        - "Top defenders for clean sheets"
        - "Which midfielders have most assists?"
        - "Erling Haaland gameweek 20 performance"
        - "Manchester City upcoming fixtures"
        
        **Pro Tips:**
        - Use **Numeric embeddings** for stat-based similarity
        - Use **Text embeddings** for semantic understanding
        ''')
    
    st.markdown("---")
    
    # System Status
    st.markdown("### 📊 System Status")
    col1, col2 = st.columns(2)
    with col1:
        st.metric("LLM", llm_choice.split()[0])
    with col2:
        st.metric("Embeddings", selected_emb_type[:6])
    
    if st.button("🔄 Reset Conversation", type="secondary", use_container_width=True):
        st.session_state.messages = []
        st.rerun()

# =========================
# MAIN INTERFACE
# =========================
st.markdown('<div class="manager-header">', unsafe_allow_html=True)
st.title("⚽ FPL Graph-RAG System")
st.markdown("### 👋 Welcome, Manager! Your AI Assistant for Fantasy Premier League Decisions")
st.caption(f"*Current Pipeline:* **{llm_choice}** → **{emb_choice}**")
st.markdown('</div>', unsafe_allow_html=True)

# Quick Stats Bar
col1, col2, col3, col4 = st.columns(4)
with col1:
    st.metric("LLM Models", "4")
with col2:
    st.metric("Embedding Types", "3")
with col3:
    st.metric("Knowledge Graph", "Active")
with col4:
    st.metric("Retrieval Method", "GraphRAG")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "assistant", "content": "Hello Manager! I'm your FPL AI Assistant. Ask me about players, teams, stats, or comparisons! ⚽"}
    ]

# Display history
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# Input handling
if prompt := st.chat_input("Ask about FPL players, teams, or stats... e.g., 'Compare Haaland and Salah'"):
    
    # User message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    # Assistant processing
    with st.chat_message("assistant"):
        # Create a container for the "Working..." status
        status_container = st.status("🏗 Processing Pipeline...", expanded=True)
        
        try:
            status_container.write(f"🤖 Using Model: **{llm_choice}**")
            status_container.write(f"📐 Using Embeddings: **{emb_choice}**")
            status_container.write("🔍 Retrieving data from Knowledge Graph...")
            
            # Call the LLM function from Task3.py
            answer, tokens_used = call_llm(
                user_question=prompt,
                model=selected_llm_id,
                embedding_type=selected_emb_type
            )
            
            status_container.write(f"✅ Generated response ({tokens_used} tokens used)")
            status_container.update(label="✅ Pipeline Complete", state="complete", expanded=False)
            
            # Display the answer
            st.markdown(answer)
            
            # Show token usage
            with st.expander("📊 Token Usage"):
                st.metric("Total Tokens", tokens_used)
            
            # Save to history
            st.session_state.messages.append({"role": "assistant", "content": answer})

        except Exception as e:
            status_container.update(label="❌ Error", state="error")
            error_msg = f"An error occurred: {e}"
            st.error(error_msg)
            st.info("Try simplifying your query or checking your configuration")
            
            # Save error to history
            st.session_state.messages.append({"role": "assistant", "content": f"❌ {error_msg}"})

# Footer
st.markdown("---")
col1, col2, col3 = st.columns(3)
with col1:
    st.caption("⚽ **FPL Graph-RAG System**")
with col2:
    st.caption("📊 Powered by Neo4j Knowledge Graph")
with col3:
    st.caption("🤖 Enhanced with LLM + Embeddings")
"""

# Write the app code to file
with open('app.py', 'w') as f:
    f.write(app_code)

print("✅ app.py created successfully!")

# ============================================
# CELL 4: LAUNCH STREAMLIT WITH NGROK
# ============================================

import time
from pyngrok import ngrok
import os

# Kill any existing Streamlit processes
os.system('pkill -f streamlit')

# Start Streamlit in the background
os.system('streamlit run app.py --server.port 8501 &')

# Wait for Streamlit to start
print("⏳ Starting Streamlit server...")
time.sleep(8)

# Create ngrok tunnel
public_url = ngrok.connect(8501)

print("="*60)
print("🎉 STREAMLIT APP IS LIVE!")
print("="*60)
print(f"📱 Access your app at: {public_url}")
print("="*60)
print("\n⚠️  IMPORTANT:")
print("   - Keep this Kaggle notebook running")
print("   - The URL will stop working if you close this notebook")
print("="*60)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Task3.py created successfully!
✅ app.py created successfully!
🚀 Initializing FPL Input Preprocessor (Enhanced)...
✅ Connected to LLM: google/gemma-2-2b-it
✅ Loaded embedding model: sentence-transformers/all-MiniLM-L6-v2
✅ System initialized

  Stopping...
⏳ Starting Streamlit server...



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://34.148.67.200:8501

🎉 STREAMLIT APP IS LIVE!
📱 Access your app at: NgrokTunnel: "https://talia-nonpublishable-shelley.ngrok-free.dev" -> "http://localhost:8501"

⚠️  IMPORTANT:
   - Keep this Kaggle notebook running
   - The URL will stop working if you close this notebook


2025-12-16 07:11:38.170661: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765869098.202723     444 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765869098.212124     444 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.p